# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 1 - PARTE 1**

## 1.- IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings
from datetime import datetime, timedelta


In [2]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2.- LECTURA DE DATAFRAMES

### 2.1 LECTURA "LIBRO MAYOR"

In [3]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Libro%20Mayor%202023.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_libro_mayor = pd.DataFrame(data)

# Imprimir el dataframe
df_libro_mayor.sample(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia
3588,4101010-000,2023,4,2023-04-04T00:00:00-03:00,CIN,63,Mezclador PAU con Distribuidor F 4 salid,P-1723,C08712674,142410,0,50189
1381,4101010-000,2023,1,2023-01-19T00:00:00-03:00,CIN,751,"Patch Panel 24P CAT5E 19"" (generico)",P-1682,C02041659,14784,0,49009
5220,4101010-000,2023,5,2023-05-08T00:00:00-03:00,CCO,218,COSTOS DE PROYECTOS,P-1709,76385837-5,29412,0,31121
1830,4101010-000,2023,2,2023-02-02T00:00:00-03:00,CIN,38,Fijación lateral de 300mm para mastil de,P-1696,C06112842,11546,0,49257
583,3101030-000,2023,6,2023-06-12T00:00:00-04:00,CVT,392,Ep #6 (adicional) Viña Oulet,P-1718,76525290-3,0,1442209,13880


In [4]:
df_libro_mayor.to_csv('archivos_respaldo/df_libro_mayor_1.csv', index = False)

### 2.2 LECTURA "DESPACHOS REALIZADOS"

In [5]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Despachos%20Realizados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_desp_realizados = pd.DataFrame(data)

# Imprimir el dataframe
df_desp_realizados.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
6188,P-1688,VCO,2023-06-09T00:00:00-04:00,51723,1076,I15420015,300,Aislador de paso cilindrico con base,UN,151,45300
6319,P-1700,VCO,2022-05-25T00:00:00-04:00,45560,1061,I15412727,70,"Perno U 3/8"" x 3"" con tuercas y golillas",UN,1131,79170
5778,NaN,STT,2022-10-11T00:00:00-03:00,47548,None,O19500966,1,Flete Ida y Vuelta,UN,101119,101119
20076,P-1685,VCO,2022-08-02T00:00:00-04:00,46550,1163,I15410668,1,"Dado Magnetico 5/16"" (Punta o Vaso)",UN,1179,1179
6696,P-1684,VCO,2022-05-05T00:00:00-03:00,45300,1044,T14400828,1,Enchufe hembra volante 220v 10a,UN,1218,1218


In [6]:
df_desp_realizados.to_csv('archivos_respaldo/df_desp_realizados_1.csv', index = False)

### 2.3 LECTURA "CODIGOS DE PROYECTO"

In [7]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\codigos%20de%20proyectos.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_cod_proyecto = pd.DataFrame(data)

# Imprimir el dataframe
df_cod_proyecto.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE
10,P-1617,Infante Videla,Constructora Grevia
144,P-544,Edif. Corporativo Armas Cabild,CONSTRUCTORA ARMAS LTDA
89,P-1667,Eco Placilla,Constructora ICF SPA
276,P-Comercial,Asignaciones asociadas al dpto. comercial,Grupo Schutz S.A.
70,P-1650,Marathon,EBCO S.A.


In [8]:
df_cod_proyecto.to_csv('archivos_respaldo/df_cod_proyecto_1.csv', index = False)

### 2.4 LECTURA "PROYECTADOS"

In [9]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso
import datetime

# Obtener el mes actual de acuerdo a la fecha actual, y el resultado se convierte en el nombre del MES para luego buscar aquel nombre en el archivo.
mes_actual = datetime.date.today().month

print(mes_actual)

if mes_actual == 1:
    mes_actual2 = "ENERO"
elif mes_actual == 2:
    mes_actual2 = "FEBRERO"
elif mes_actual == 3:
    mes_actual2 = "MARZO"
elif mes_actual == 4:
    mes_actual2 = "ABRIL"
elif mes_actual == 5:
    mes_actual2 = "MAYO"
elif mes_actual == 6:
    mes_actual2 = "JUNIO"
elif mes_actual == 7:
    mes_actual2 = "JULIO"
elif mes_actual == 8:
    mes_actual2 = "AGOSTO"
elif mes_actual == 9:
    mes_actual2 = "SEPTIEMBRE"
elif mes_actual == 10:
    mes_actual2 = "OCTUBRE"
elif mes_actual == 11:
    mes_actual2 = "NOVIEMBRE"
else:
    mes_actual2 = "DICIEMBRE"

print(mes_actual2)

6
JUNIO


In [10]:
ssl._create_default_https_context = ssl._create_unverified_context

# Lectura de presupuestos desde dropbox
df_proyectado = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_actual2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4-2 LECTURA "PROYECTADOS 2"

In [11]:
# Get the current date
import datetime

fecha_actual = datetime.datetime.now()

# Calculate the previous month
mes_anterior2 = (fecha_actual - timedelta(days=30)).month
print(mes_anterior2)

if mes_anterior2 == 1:
    mes_anterior2 = "ENERO"
elif mes_anterior2 == 2:
    mes_anterior2 = "FEBRERO"
elif mes_anterior2 == 3:
    mes_anterior2 = "MARZO"
elif mes_anterior2 == 4:
    mes_anterior2 = "ABRIL"
elif mes_anterior2 == 5:
    mes_anterior2 = "MAYO"
elif mes_anterior2 == 6:
    mes_anterior2 = "JUNIO"
elif mes_anterior2 == 7:
    mes_anterior2 = "JULIO"
elif mes_anterior2 == 8:
    mes_anterior2 = "AGOSTO"
elif mes_anterior2 == 9:
    mes_anterior2 = "SEPTIEMBRE"
elif mes_anterior2 == 10:
    mes_anterior2 = "OCTUBRE"
elif mes_anterior2 == 11:
    mes_anterior2 = "NOVIEMBRE"
else:
    mes_anterior2 = "DICIEMBRE"

print(mes_anterior2)

# Lectura de presupuestos desde dropbox
df_proyectado_1M = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_anterior2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado_1M.head(20)

5
MAYO


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.5 LECTURA "LIBRO MAYOR AÑOS ANTERIORES"

In [12]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

In [13]:
spr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Fx1nerTu10PZPOwknLcQuri_I6ARat5h7zuq4P3vypg/edit#gid=1097918290')
recipients = spr.sheet1.get_all_values()

# Se el string creado en un dataframe
df_libro_mayor_old = pd.DataFrame(recipients)

# se reasignan los headers en el dataframe.
new_header = df_libro_mayor_old.iloc[0] #grab the first row for the header
df_libro_mayor_old = df_libro_mayor_old[1:] #take the data less the header row
df_libro_mayor_old.columns = new_header #set the header row as the df header

# Se conservan solo las columnas de interés
#df_seguridad_v3 = df_seguridad_v3[["Nº Cédula","Nombres y apellidos","APRUEBA FASE DE CALIFICACIÓN ?","PUNTOS VIGENTES","APRUEBA FASE DE EVALUACIÓN ?","RESULTADO FINAL"]]

print("Las dimensiones del dataframe son :",df_libro_mayor_old.shape)
df_libro_mayor_old.sample(5)

Las dimensiones del dataframe son : (17368, 13)


,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
122,3101030-000,2021,2,2/22/21 0:00,CVT,404,CENTRO DE COSTO: 019 - PROV.E INST.CCTV-CITOFO...,P-1661,78056380-K,10925,0,0,
13551,4101010-000,2022,9,9/26/22 0:00,CIN,700,Guia SC/APC SM,P-1701,O19501068,47317,138560,0,
1617,4101010-000,2021,1,1/26/21 0:00,CIN,703,"Tornillo roscalata 6 x 1/2""",P-1664,I17440515,40454,5000,0,
10125,4101010-000,2022,4,4/26/22 0:00,CIN,374,"Soporte pera 3"" UL/FM",P-1693,P08202222,45127,3990,0,
17132,4201001-000,2022,1,1/31/22 0:00,AJC,1233,REMUNERACIONES POR DISTRIBUIR,P-Oficina,P-Oficina,17250657,0,43718,


In [14]:
df_libro_mayor_old.to_csv('archivos_respaldo/df_libro_mayor_old.csv', mode='a')

### 2.6 LECTURA "PLANILLA RESUMEN - PRESPUESTOS"

In [15]:
# Lectura de presupuestos desde dropbox
df_presupuesto = pd.read_excel("https://www.dropbox.com/s/miow7p9wl7t3jsr/PLANILLA%20RESUMEN.xlsb.xlsx?dl=1",sheet_name= "PROYECTOS" ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_presupuesto.head(5)

,PROYECTO,OBRA,RUT,CONSTRUCTORA,FECHA INICIO,FECHA TERMINO,ULTIMO EP PRESENTADO,AVANCE EN OBRA,MONTO EN $,$ POR FACTURAR,...,217465.036046312,217465.036046312.1,217465.036046312.2,217465.036046312.3,0,0.1,0.2,0.3,0.4,0.5
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,8113107696.86,3256288934.71,...,1,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00
1,P-1632,Chorrillos Clorox,76.108.978-1,SuKsa S.A,2019-02-04 00:00:00,2019-05-31,NaN,0.99,138107706.03,915642.26,...,217465.04,217465.04,217465.04,217465.04,0.00,0.00,0.00,0.00,0.00,0.00
2,P-1638,Colon,78134800-7,Grevia,2019-08-13 00:00:00,2020-03-27,NaN,1.00,43206282.00,2177592.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-1645,Los 3 Antonios,77.710.830-1,EDICASA,2019-08-31 00:00:00,NaT,NaN,0.98,18150000.00,208732.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-1647,Bodega corrosivos TW,76.564.630-8,Inmobiliaria y Rentas Logisticas S.A,NaN,NaT,NaN,0.58,29482131.00,12437344.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_presupuesto.to_csv('archivos_respaldo/df_presupuesto.csv', mode='a')

### 2.7 LECTURA "TABLA- PRESPUESTOS"

In [17]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Presupuesto Proyectos.txt'
response = requests.get(url)
query_result = response.text
soup = BeautifulSoup(response.content, 'xml')
rows = soup.find_all('Table')


data = []
for row in rows:
    values = [cell.text for cell in row.find_all()]
    data.append(values)

df_tabla_prespuesto = pd.DataFrame(data, columns=['CODIGO_PROYECTO','PARTIDA_PRESUPUESTARIA','PRODUCTO','UNIDAD','CANTIDAD','COSTO_UNITARIO','COSTO_TOTAL'])

# Exportación a excel
df_tabla_prespuesto.to_csv('df_tabla_prespuesto.csv', index = False)

df_tabla_prespuesto.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490,428490
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866,1022434
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846,95384
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608,138224
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200,353600


## 3.- TRATAMIENTO DE DATAFRAMES

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR

#### 3.1.1 - Exploración inicial de los datos

In [18]:
df_libro_mayor.shape

(8185, 12)

In [19]:
df_libro_mayor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8185 entries, 0 to 8184
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cta_Contable     8185 non-null   object
 1   Año              8185 non-null   object
 2   Mes              8185 non-null   object
 3   Fecha            8160 non-null   object
 4   Tipo_Comp        8185 non-null   object
 5   Num_Comp         8185 non-null   object
 6   Glosa            8185 non-null   object
 7   Primer_Analisis  8182 non-null   object
 8   Referencia       8185 non-null   object
 9   Debito           8185 non-null   object
 10  Credito          8185 non-null   object
 11  Num_Referencia   6594 non-null   object
dtypes: object(12)
memory usage: 767.5+ KB


#### 3.1.2 - Estandarización campo "Debito" y Crédito

In [20]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor['Debito_2'] = pd.to_numeric(df_libro_mayor['Debito'], errors='coerce')
df_libro_mayor['Credito_2'] = pd.to_numeric(df_libro_mayor['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor['Credito_2'] = df_libro_mayor['Credito_2'].fillna(0)

df_libro_mayor.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.3 - Creación campo Codigo_proyecto

In [21]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Primer_Analisis']

cols = df_libro_mayor.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('Codigo_proyecto'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor = df_libro_mayor.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Codigo_proyecto'].str.upper()

df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.4 - Creación de campo total

In [22]:
df_libro_mayor['TOTAL'] = df_libro_mayor['Credito_2'] - df_libro_mayor['Debito_2']
df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0,-4081983
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0,-6464848
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0,-4673685
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0,-1570055
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0,-3613652


#### 3.1.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [23]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return 'INGRESO'
    elif cta_contable == "4101010-000":
        return 'COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return 'COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return 'COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor['Tipo_cuenta'] = df_libro_mayor['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
4700,P-1725,4101010-000,2023,4,2023-04-26T00:00:00-03:00,CIN,1218,"Magnetico Sobrepuesto 7/8"" Blanco",P-1725,S11292446,65625,0,50735,65625,0,-65625,COSTO MATERIALES
3918,P-1697,4101010-000,2023,4,2023-04-12T00:00:00-03:00,CIN,382,Conector RG11 para crimpar,P-1697,C08732769,53160,0,50353,53160,0,-53160,COSTO MATERIALES
5860,P-1718,4101010-000,2023,5,2023-05-27T00:00:00-04:00,CCO,1633,COSTOS DE PROYECTOS,P-1718,76134941-4,21471,0,31337,21471,0,-21471,COSTO MATERIALES
5785,P-1721,4101010-000,2023,5,2023-05-23T00:00:00-04:00,CIN,1227,Magnético Blindado N.c Vc-52m Vconn,P-1721,S11290723,12596,0,51377,12596,0,-12596,COSTO MATERIALES
5420,P-1680,4101010-000,2023,5,2023-05-12T00:00:00-03:00,CIN,643,DERIVADOR 5-2400 MHZ RIT INALAMBRICA 4D,P-1680,C08713720,19764,0,51135,19764,0,-19764,COSTO MATERIALES


In [24]:
# EXPLORACIÓN DE DATOS

df_libro_mayor_piv = pd.pivot_table(df_libro_mayor, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_piv = pd.DataFrame(df_libro_mayor_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
25,OTRA,2023,4,3,29982233,-29982230
0,COSTO MANO DE OBRA,2023,1,0,72542885,-72542885
12,COSTO SUPERVISIÓN,2023,2,0,4016206,-4016206
6,COSTO MATERIALES,2023,2,3348963,83938903,-80589940
1,COSTO MANO DE OBRA,2023,2,7,82645544,-82645537
7,COSTO MATERIALES,2023,3,2764412,90084966,-87320554
2,COSTO MANO DE OBRA,2023,3,0,96802066,-96802066
10,COSTO MATERIALES,2023,6,4291793,151223128,-146931335
23,OTRA,2023,2,0,22347286,-22347286
11,COSTO SUPERVISIÓN,2023,1,0,4336206,-4336206


#### 3.1.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [25]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor2 = df_libro_mayor.copy()

In [26]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-PROYECTOS' 'P-1669'
 'P-1667' 'P-1668' 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658'
 'P-1660' 'P-1644' 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653'
 'P-1673' 'P-1683' 'P-1661' 'P-1612' 'P-1728' 'P-SERVICIOS' 'P-1639'
 'P-1674' 'P1718' 'P-1620' 'P-1603' 'P-1626' 'P-1649' 'P-1685' 'P-1632'
 'P-1643' 'P-1666' 'P-1656' '15314868-6' 'P-1717' 'P-1663' '78540190-5'
 '5555555-9' 'P-1729' 'P-1636' '77092384-0' '76107516-0' '96670840-9'
 'P-OFICINA' 'P-ANALISIS PROYE' 'P-1557' 'P-1591' '18800250-1'
 '13056802-5' '16411600-K' 

##### 3.1.6.1 - Se Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [27]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor2_2 = df_libro_mayor2.copy()

# Filtrar el DataFrame
df_libro_mayor2_2 = df_libro_mayor2_2[(df_libro_mayor2_2['Codigo_proyecto'] == 'P-OFICINA') & (df_libro_mayor2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor2_2

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
7868,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN
7869,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7870,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN
7871,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7872,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292504,0,NaN,2292504,0,-2292504,COSTO SUPERVISIÓN
7873,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7874,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN
7875,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN
7876,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292357,0,NaN,2292357,0,-2292357,COSTO SUPERVISIÓN
7877,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2350904,0,NaN,2350904,0,-2350904,COSTO SUPERVISIÓN


In [28]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor2 = df_libro_mayor2.dropna(subset=['Codigo_proyecto'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask1 = df_libro_mayor2['Codigo_proyecto'].isin(valores_a_eliminar)

# Crea una máscara booleana que identifica las filas que comienzan con un número
mask2 = df_libro_mayor2['Codigo_proyecto'].str[0].str.isnumeric()

# Une las dos máscaras con el operador OR
mask = mask1 | mask2

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor2 = df_libro_mayor2[~mask]


# Verifica si el valor 'P1718' o valore similares están presentes en la columna Codigo_proyecto
if 'P1718' in df_libro_mayor2['Codigo_proyecto'].values:
    # Reemplaza el valor 'P1718' por 'P-1718' en la columna Codigo_proyecto
    df_libro_mayor2['Codigo_proyecto'] = df_libro_mayor2['Codigo_proyecto'].replace('P1718', 'P-1718')


In [29]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-1669' 'P-1667' 'P-1668'
 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658' 'P-1660' 'P-1644'
 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653' 'P-1673' 'P-1683'
 'P-1661' 'P-1612' 'P-1728' 'P-1639' 'P-1674' 'P-1620' 'P-1603' 'P-1626'
 'P-1649' 'P-1685' 'P-1632' 'P-1643' 'P-1666' 'P-1656' 'P-1717' 'P-1663'
 'P-1729' 'P-1636' 'P-1557' 'P-1591']
Dimensiones del dataframe antes de la reducción : 
(7663, 17)


#### 3.1.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [30]:
df_libro_mayor2['Fecha2'] = pd.to_datetime(df_libro_mayor2['Año'].astype(str) + '-' + df_libro_mayor2['Mes'].astype(str) + '-1')
df_libro_mayor2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
6393,P-1704,4101010-000,2023,6,2023-06-08T00:00:00-04:00,CIN,251,"Codo 90° 4"" Ranurado UL A-536",P-1704,P08200561,10026,0,51640,10026,0,-10026,COSTO MATERIALES,2023-06-01
7624,P-1699,4150002-000,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES DIRECTAS PROYECTO,P-1699,8606789,828165,0,NaN,828165,0,-828165,COSTO MANO DE OBRA,2023-05-01
1356,P-1699,4101010-000,2023,1,2023-01-19T00:00:00-03:00,CIN,744,Conector Conico F RG6 para crimpar,P-1699,S09242300,55200,0,49002,55200,0,-55200,COSTO MATERIALES,2023-01-01
6044,P-1694,4101010-000,2023,6,2023-06-02T00:00:00-04:00,CIN,59,REPARTIDOR 5-2400 MHZ RIT INALAMBRICA 2D,P-1694,C08712960,12176,0,51488,12176,0,-12176,COSTO MATERIALES,2023-06-01
5504,P-1704,4101010-000,2023,5,2023-05-15T00:00:00-04:00,CIN,751,"Strap 2"" x 1"" 150# NPT UL/FM",P-1704,P08200274,129300,0,51196,129300,0,-129300,COSTO MATERIALES,2023-05-01


##### 3.1.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [31]:
df_libro_mayor2_2['Fecha2'] = pd.to_datetime(df_libro_mayor2_2['Año'].astype(str) + '-' + df_libro_mayor2_2['Mes'].astype(str) + '-1')
df_libro_mayor2_2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
7874,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN,2023-04-01
7873,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-03-01
7876,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292357,0,NaN,2292357,0,-2292357,COSTO SUPERVISIÓN,2023-05-01
7871,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-02-01
7875,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN,2023-04-01


#### 3.1.8 - Se realiza pivote final del dataframe

In [32]:
# Se exploran las columnas:
df_libro_mayor2.columns

Index(['Codigo_proyecto', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Debito',
       'Credito', 'Num_Referencia', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object')

In [33]:
# crear pivot table sin margenes

df_libro_mayor2_piv = pd.pivot_table(df_libro_mayor2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_piv = pd.DataFrame(df_libro_mayor2_piv.to_records())

df_libro_mayor2_piv.sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
4,P-1592,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,45357,0,45357
486,P-1720,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,65665,-65665
14,P-1632,COSTO MATERIALES,2023-06-01 00:00:00,2023,6,22680,21840,840
8,P-1603,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,0,30000,-30000
172,P-1689,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,26522,0,26522


In [34]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor2_piv.shape

(559, 8)

##### 3.1.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [35]:
# crear pivot table sin margenes

df_libro_mayor2_2_piv = pd.pivot_table(df_libro_mayor2_2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_2_piv = pd.DataFrame(df_libro_mayor2_2_piv.to_records())

df_libro_mayor2_2_piv = df_libro_mayor2_2_piv[df_libro_mayor2_2_piv['Codigo_proyecto'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor2_2_piv['Codigo_proyecto'] = df_libro_mayor2_2_piv['Codigo_proyecto'].replace('P-OFICINA', 'TOTAL')


df_libro_mayor2_2_piv

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023,1,0,4336206,-4336206
1,TOTAL,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023,2,0,4016206,-4016206
2,TOTAL,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023,3,0,4474872,-4474872
3,TOTAL,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023,4,0,5096121,-5096121
4,TOTAL,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023,5,0,4643261,-4643261


#### 3.1.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta

In [36]:
df_libro_mayor_piv2 = pd.pivot_table(df_libro_mayor2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_piv2 = pd.DataFrame(df_libro_mayor_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_piv2.insert(loc=0, column='Codigo_proyecto', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != '']

df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
2,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
3,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
4,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
5,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,114373715,-114373715
6,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
7,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
8,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88061320,-85296908
9,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,149731515,-146484271
10,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,178069095,-169535388


##### 3.1.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [37]:
df_libro_mayor_piv2 = pd.concat([df_libro_mayor_piv2, df_libro_mayor2_2_piv], ignore_index=True)
df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88061320,-85296908
8,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,149731515,-146484271
9,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,178069095,-169535388


#### 3.1.10 - Se concatenan los 2 dataframes resultantes

In [38]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor2_piv.shape)

Dimensiones del dataframe antes de la concatenación : 
(559, 8)


In [39]:
df_libro_mayor3 = pd.concat([df_libro_mayor2_piv, df_libro_mayor_piv2])
df_libro_mayor3.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
220,P-1694,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,0,7641345,-7641345
342,P-1705,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,76065,5196251,-5120186
413,P-1713,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,1005445,-1005445
460,P-1716,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,719943,1750152,-1030209
223,P-1694,INGRESO,2023-02-01 00:00:00,2023,2,7211400,0,7211400


In [40]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe después de la concatenación : 
(581, 8)


#### 3.1.11 - Se crean las filas que permiten tener siempre todas las opciones de Tipo_Cuenta por cada Codigo_proyecto

In [41]:
combinaciones = []
for proyecto in df_libro_mayor3['Codigo_proyecto'].unique():
    for fecha in df_libro_mayor3['Fecha2'].unique():
        mes = pd.to_datetime(fecha).month
        año = pd.to_datetime(fecha).year
        for cuenta in ['INGRESO', 'COSTO MATERIALES', 'COSTO MANO DE OBRA', 'COSTO SUPERVISIÓN']:
            combinaciones.append((proyecto, fecha, mes, año, cuenta))

# crear un nuevo DataFrame con las combinaciones obtenidas
combinaciones_df = pd.DataFrame(combinaciones, columns=['Codigo_proyecto', 'Fecha2', 'Mes', 'Año', 'Tipo_cuenta'])

# agregar las demás columnas con valores nulos
combinaciones_df['Credito_2'] = 0
combinaciones_df['Debito_2'] = 0
combinaciones_df['TOTAL'] = 0

# Se eliminan filas con fecha en blanco.
combinaciones_df = combinaciones_df.dropna(subset=['Mes'])

combinaciones_df.sample(10)

,Codigo_proyecto,Fecha2,Mes,Año,Tipo_cuenta,Credito_2,Debito_2,TOTAL
945,P-1675,2023-06-01 00:00:00,6.00,2023.00,COSTO MATERIALES,0,0,0
294,P-1639,2023-02-01 00:00:00,2.00,2023.00,COSTO MANO DE OBRA,0,0,0
169,P-1620,2023-04-01 00:00:00,4.00,2023.00,COSTO MATERIALES,0,0,0
35,P-1591,2023-05-01 00:00:00,5.00,2023.00,COSTO SUPERVISIÓN,0,0,0
1403,P-1693,2023-04-01 00:00:00,4.00,2023.00,COSTO SUPERVISIÓN,0,0,0
274,P-1636,2023-06-01 00:00:00,6.00,2023.00,COSTO MANO DE OBRA,0,0,0
665,P-1663,2023-06-01 00:00:00,6.00,2023.00,COSTO MATERIALES,0,0,0
1708,P-1705,2023-04-01 00:00:00,4.00,2023.00,INGRESO,0,0,0
1553,P-1698,2023-02-01 00:00:00,2.00,2023.00,COSTO MATERIALES,0,0,0
536,P-1656,2023-05-01 00:00:00,5.00,2023.00,INGRESO,0,0,0


#### 3.1.12 - Se concatenan los 2 dataframes resultantes

In [42]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe antes de la concatenación : 
(581, 8)


In [43]:
df_libro_mayor4 = pd.concat([df_libro_mayor3, combinaciones_df])
df_libro_mayor4.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
2206,P-1724,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0
1913,P-1714,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0
1741,P-1706,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0
2212,P-1725,INGRESO,2023-04-01 00:00:00,2023.00,4.00,0,0,0
1637,P-1701,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0


In [44]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor4.shape)

Dimensiones del dataframe después de la concatenación : 
(2621, 8)


#### 3.1.13 - Estandarización de campos 

In [45]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor4 = df_libro_mayor4.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor4['AÑO'] = pd.to_numeric(df_libro_mayor4['AÑO'], downcast='integer')
df_libro_mayor4['MES'] = pd.to_numeric(df_libro_mayor4['MES'], downcast='integer')

df_libro_mayor4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
171,P-1688,INGRESO,2023-05-01 00:00:00,2023.00,5.00,9966497,5436746,4529751
333,P-1704,INGRESO,2023-02-01 00:00:00,2023.00,2.00,23544607,0,23544607
1433,P-1694,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0
324,P-1641,INGRESO,2023-03-01 00:00:00,2023.00,3.00,0,0,0
1968,P-1716,INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0
2371,TOTAL,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0
1966,P-1716,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0
133,P-1611,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0
387,P-1643,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0
2080,P-1720,INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0


In [46]:
#df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
558,TOTAL,,,NaN,NaN,2293294225,1935963462,357330763
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88061320,-85296908
8,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,149731515,-146484271


#### 3.1.14 Creación de columna MARGEN

In [47]:
df_libro_mayor4_2 = df_libro_mayor4.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_libro_mayor4_2.groupby(['CODIGO_PROYECTO','MES']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

    
df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(5)

,CODIGO_PROYECTO,MES,MARGEN
294,P-1692,1.00,0.00
232,P-1681,5.00,0.00
417,P-1715,4.00,0.00
114,P-1656,1.00,0.00
458,P-1722,3.00,0.00


In [48]:
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  : ", df_libro_mayor4.shape)
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          : ", df_margen.shape)
print(df_margen.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  :  (2621, 8)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          :  (510, 3)
(510, 3)


In [49]:
# CONCATENACIÓN DE DATAFRAMES

df_libro_mayor5 = pd.merge(df_libro_mayor4, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor5['CODIGO_PROYECTO'] = df_libro_mayor5['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  : ", df_libro_mayor5.shape)
print(df_libro_mayor5.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  :  (2621, 9)
(2621, 9)


In [50]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
558,1. TOTAL,,,NaN,NaN,2293294225,1935963462,357330763,NaN
559,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,0.00
560,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,0.00
561,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,0.00
562,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,0.00
563,1. TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,114373715,-114373715,0.00
564,1. TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040,0.00
565,1. TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234,0.00
566,1. TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88061320,-85296908,0.00
567,1. TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,149731515,-146484271,0.00


In [51]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
321,P-1704,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,7375493,-7375493,0.00
322,P-1704,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,10751091,-10751091,0.00
323,P-1704,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,12332472,-12332472,0.00
324,P-1704,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,14242535,-14242535,0.00
325,P-1704,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,12890883,-12890883,0.00
326,P-1704,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,20073319,-20073319,0.00
327,P-1704,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,139976,3388760,-3248784,0.00
328,P-1704,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,14768626,-14768626,0.00
329,P-1704,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,20031042,-20031042,0.00
330,P-1704,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,13967551,-13967551,0.00


In [52]:
df_libro_mayor5["CODIGO_PROYECTO"].unique()

array(['P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611', 'P-1612',
       'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639', 'P-1641',
       'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648', 'P-1649',
       'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661', 'P-1663',
       'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669', 'P-1670',
       'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676', 'P-1677',
       'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683', 'P-1684',
       'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689', 'P-1690',
       'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695', 'P-1696',
       'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701', 'P-1703',
       'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708', 'P-1709',
       'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716', 'P-1717',
       'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722', 'P-1723',
       'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728', 'P-1729',
       '1. TOTAL'], dtype=object)

In [53]:
df_libro_mayor5.to_csv('archivos_respaldo/df_libro_mayor5.csv', index = False)

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR SUBSETS

#### 3.1.16 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO"**

In [54]:
fecha_actual = datetime.date.today()
fecha_actual

datetime.date(2023, 6, 19)

In [55]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd = df_libro_mayor5.copy()
df_libro_mayor_ytd.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
358,P-1706,INGRESO,2023-02-01 00:00:00,2023.00,2.00,1096335,0,1096335,0.00
1637,P-1687,INGRESO,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
1995,P-1701,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
2528,P-1727,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
1740,P-1691,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00


In [56]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Filtrar los registros del dataframe que corresponden al mes actual y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month < mes_actual]

df_libro_mayor_ytd_closed.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1617,P-1686,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1494,P-1681,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
1423,P-1677,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
178,P-1690,COSTO MATERIALES,2023-02-01,2023.00,2.00,218974,4882316,-4663342,0.00
1839,P-1695,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,0,0,0.00
1594,P-1685,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00
162,P-1688,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,2657700,-2657700,0.00
1256,P-1669,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
2448,P-1723,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00
367,P-1707,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,2057186,-2057186,0.00


In [57]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_piv = pd.pivot_table(df_libro_mayor_ytd_closed, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)
#df_libro_mayor4_piv = pd.pivot_table(df_libro_mayor4, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta'], columns = ['Fecha2'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')

df_libro_mayor_ytd_closed_piv = pd.DataFrame(df_libro_mayor_ytd_closed_piv.to_records())

df_libro_mayor_ytd_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
186,P-1688,COSTO SUPERVISIÓN,0
273,P-1713,COSTO MATERIALES,-2358536
322,P-1725,COSTO SUPERVISIÓN,0
234,P-1700,COSTO SUPERVISIÓN,0
250,P-1705,COSTO SUPERVISIÓN,0
119,P-1669,INGRESO,0
251,P-1705,INGRESO,30287816
66,P-1645,COSTO SUPERVISIÓN,0
54,P-1642,COSTO SUPERVISIÓN,0
272,P-1713,COSTO MANO DE OBRA,-4283233


#### 3.1.16-1 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO 2"**
Este dataframe es para considerar como mes cerrado 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [58]:
# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Calcular el mes que es dos meses antes del actual
mes_filtrado = mes_actual - 2 if mes_actual > 2 else (12 + mes_actual - 2) 

# Filtrar los registros del dataframe que corresponden al mes filtrado y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed_2M = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month <= mes_filtrado]

df_libro_mayor_ytd_closed_2M.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1315,P-1672,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
2144,P-1709,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
2376,P-1720,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00
2587,P-1729,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
1387,P-1675,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
1686,P-1689,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
524,P-1725,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,449245,-449245,0.00
93,P-1680,COSTO MATERIALES,2023-03-01,2023.00,3.00,918990,2002485,-1083495,0.00
1679,P-1688,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,0.00
1422,P-1677,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00


In [59]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_2M = pd.pivot_table(df_libro_mayor_ytd_closed_2M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd_closed_2M = pd.DataFrame(df_libro_mayor_ytd_closed_2M.to_records())

df_libro_mayor_ytd_closed_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
208,P-1694,COSTO MANO DE OBRA,-12624130
144,P-1677,COSTO MANO DE OBRA,-1992711
136,P-1675,COSTO MANO DE OBRA,0
329,P-1727,COSTO MATERIALES,-3753865
193,P-1690,COSTO MATERIALES,-7192915


#### 3.1.17 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO**

In [60]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.
 
df_libro_mayor5_mec = df_libro_mayor5.copy()
df_libro_mayor5_mec.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  : ", df_libro_mayor5_mec.shape)
df_libro_mayor5_mec.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  :  (2621, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
257,P-1697,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,4485402,-4485402,0.00
1623,P-1686,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
1342,P-1673,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00


In [61]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
2418,P-1722,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00,LIBRO MAYOR
2425,P-1722,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
81,P-1677,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,24352,0,24352,0.00,LIBRO MAYOR
1788,P-1693,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1063,P-1658,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR


In [62]:
# Se seleccionan los meses que son utiles para crear la vista de mes en curso.
# Para el real se necesita: INGRESO, COSTO MATERIALES, Y SUPERVISIÓN (ÉSTE ÚLTIMO DEL MES ANTERIOR)
# Para el proyectado se necesita: SUPERVISIÓN (DEL MES ANTERIOR)

In [63]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta


df_libro_mayor5_mec['FECHA'] = pd.to_datetime(df_libro_mayor5_mec['FECHA'])

filtro_ingreso_materiales = ((df_libro_mayor5_mec['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec['FECHA'].dt.month == datetime.now().month)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_supervision_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_mo_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec2 = df_libro_mayor5_mec[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  : ", df_libro_mayor5_mec2.shape)

df_libro_mayor5_mec2.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  :  (426, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1850,P-1695,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2027,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1908,P-1698,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
376,P-1708,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,99013,-99013,0.00,LIBRO MAYOR
961,P-1645,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
890,P-1642,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1490,P-1680,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
769,P-1626,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
698,P-1611,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
697,P-1611,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [64]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
325,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,12890883,-12890883,0.00,LIBRO MAYOR
331,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,24160596,-24160596,0.00,LIBRO MAYOR
2027,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2028,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2041,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2042,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [65]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
563,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,114373715,-114373715,0.00,LIBRO MAYOR
569,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,4291793,149905154,-145613361,0.00,LIBRO MAYOR
575,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,65653580,58891059,6762521,0.00,LIBRO MAYOR
580,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,4643261,-4643261,0.00,LIBRO MAYOR
2603,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2604,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2617,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2618,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.17-1 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO 2**
Este dataframe es para considerar como mes en curso de 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [66]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.

df_libro_mayor5_mec_2M = df_libro_mayor5.copy()
df_libro_mayor5_mec_2M.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (2621, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2328,P-1718,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
401,P-1712,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,2944488,-2944488,0.00
1875,P-1696,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00


In [67]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec_2M["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
582,P-1557,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
469,P-1718,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,6094576,-6094576,0.00,LIBRO MAYOR
829,P-1639,INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR
277,P-1699,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,2971945,-2971945,0.00,LIBRO MAYOR
1885,P-1697,INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR


In [68]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta

df_libro_mayor5_mec_2M['FECHA'] = pd.to_datetime(df_libro_mayor5_mec_2M['FECHA'])

# Filtro para INGRESO Y COSTO DE MATERIALES
mes_anterior = (datetime.now() - timedelta(days=30)).month

filtro_ingreso_materiales = ((df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_supervision_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_mo_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec_2M = df_libro_mayor5_mec_2M[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)

df_libro_mayor5_mec_2M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
505,P-1723,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,7889395,-7889395,0.00,LIBRO MAYOR
848,P-1641,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
1498,P-1681,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1376,P-1675,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
1448,P-1679,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR


In [69]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
324,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00,LIBRO MAYOR
330,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00,LIBRO MAYOR
336,P-1704,INGRESO,2023-05-01,2023.00,5.00,132309997,53250000,79059997,0.00,LIBRO MAYOR
2023,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2024,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2025,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2026,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [70]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
562,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00,LIBRO MAYOR
568,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,178069095,-169535388,0.00,LIBRO MAYOR
574,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,631496823,170698310,460798513,0.00,LIBRO MAYOR
579,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,5096121,-5096121,0.00,LIBRO MAYOR
2599,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2600,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2601,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2602,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.18 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual)**

In [71]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd2 = df_libro_mayor5.copy()
df_libro_mayor_ytd2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
540,P-1726,INGRESO,2023-03-01 00:00:00,2023.00,3.00,27650696,27650696,0,0.00
2313,P-1718,INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
989,P-1649,INGRESO,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
2252,P-1715,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
21,P-1644,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,198000,0,198000,0.00
1164,P-1664,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
1941,P-1699,INGRESO,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
1952,P-1700,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
1847,P-1695,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
2128,P-1708,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00


In [72]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv = pd.pivot_table(df_libro_mayor_ytd2, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv = pd.DataFrame(df_libro_mayor_ytd2_closed_piv.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  : ", df_libro_mayor_ytd2_closed_piv.shape)

df_libro_mayor_ytd2_closed_piv.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  :  (341, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
182,P-1687,COSTO MATERIALES,-6360393
64,P-1644,INGRESO,0
46,P-1639,COSTO MATERIALES,20266
225,P-1698,COSTO MANO DE OBRA,0
280,P-1714,INGRESO,20583182
227,P-1698,COSTO SUPERVISIÓN,0
116,P-1668,INGRESO,0
159,P-1681,COSTO SUPERVISIÓN,0
52,P-1641,INGRESO,0
331,P-1727,COSTO SUPERVISIÓN,0


#### 3.1.18-2 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual) 2**

In [73]:
# Obtener el mes actual
mes_actual = datetime.now().month

# Filtrar los registros hasta el mes anterior al mes actual
df_libro_mayor_ytd2_1M = df_libro_mayor_ytd2[df_libro_mayor_ytd2['MES'] < mes_actual]

# Mostrar el nuevo DataFrame
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  : ", df_libro_mayor_ytd2_1M.shape)
df_libro_mayor_ytd2_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  :  (2230, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
521,P-1724,INGRESO,2023-04-01 00:00:00,2023.00,4.00,27099640,13549820,13549820,0.00
1872,P-1696,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
1733,P-1691,INGRESO,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
1459,P-1679,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
1176,P-1664,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00


In [74]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv_1M = pd.pivot_table(df_libro_mayor_ytd2_1M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv_1M = pd.DataFrame(df_libro_mayor_ytd2_closed_piv_1M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  : ", df_libro_mayor_ytd2_closed_piv_1M.shape)

df_libro_mayor_ytd2_closed_piv_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
195,P-1690,INGRESO,16662806
182,P-1687,COSTO SUPERVISIÓN,0
289,P-1717,COSTO MATERIALES,0
10,P-1591,COSTO SUPERVISIÓN,0
126,P-1672,COSTO SUPERVISIÓN,0
235,P-1700,INGRESO,16747965
88,P-1660,COSTO MANO DE OBRA,0
37,P-1632,COSTO MATERIALES,75965
154,P-1680,COSTO SUPERVISIÓN,0
188,P-1689,COSTO MANO DE OBRA,0


#### 3.1.19 - CREACIÓN DE DATAFRAME BASE LIBRO MAYOR 2023 PARA EL CRUCE OTROS DATAFRAMES

##### NOTA IMPORTANTE
##### Se considera que todos los dataframes que pintan los dashboards deben tener como base los proyectos abiertos en el Libro Mayor 2023. Para esto se crea el dataframe base con el cual debe comenzar cualquier cruce de datos y que considera el código del proyecto, todos los tipos de cuenta por cada proyecto y con valor en cero, además se cruza con la tabla codigo de proyecto.

In [75]:
df_libro_mayor5_base = df_libro_mayor5.copy()

In [76]:
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base = df_libro_mayor5_base[['CODIGO_PROYECTO','TIPO_CUENTA']].drop_duplicates()

# Se elimina fila de TOTAL
df_libro_mayor5_base = df_libro_mayor5_base[df_libro_mayor5_base['CODIGO_PROYECTO'] != '1. TOTAL']

# Se Reemplazan valores de la column TIPO_CUENTA
df_libro_mayor5_base['TIPO_CUENTA'] = df_libro_mayor5_base['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor5_base['TOTAL_LB']= 0

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(10)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   :  (2621, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (336, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
679,P-1611,3. COSTO MANO DE OBRA,0
2407,P-1722,3. COSTO MANO DE OBRA,0
520,P-1724,1. INGRESO,0
1448,P-1679,4. COSTO SUPERVISIÓN,0
776,P-1632,4. COSTO SUPERVISIÓN,0
848,P-1641,4. COSTO SUPERVISIÓN,0
553,P-1728,3. COSTO MANO DE OBRA,0
2335,P-1719,3. COSTO MANO DE OBRA,0
872,P-1642,4. COSTO SUPERVISIÓN,0
117,P-1683,2. COSTO MATERIALES,0


In [77]:
# Se Agrega COSTO DE SUPERVISIÓN

nueva_fila = pd.DataFrame({'CODIGO_PROYECTO': ['1. TOTAL'],
                           'TIPO_CUENTA': ['4. COSTO SUPERVISIÓN'],
                           'TOTAL_LB': [0]})

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, nueva_fila], ignore_index=True)

df_libro_mayor5_base[df_libro_mayor5_base["CODIGO_PROYECTO"]=="1. TOTAL"]
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())
print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)


In [78]:
# Crear DataFrame con TIPO_CUENTA y valor cero para cada combinación única

df_nuevo = pd.DataFrame({
    'CODIGO_PROYECTO': df_libro_mayor5_base['CODIGO_PROYECTO'].repeat(4),
    'TIPO_CUENTA': ['1. INGRESO', '2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'] * len(df_libro_mayor5_base),
    'TOTAL_LB': 0
})

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             : ", df_nuevo.shape)

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, df_nuevo], ignore_index=True)

# Se remueve cualquier duplicado.
df_libro_mayor5_base.drop_duplicates(inplace=True)

print("DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             :  (1348, 3)
DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
160,P-1726,2. COSTO MATERIALES,0
100,P-1701,3. COSTO MANO DE OBRA,0
45,P-1679,3. COSTO MANO DE OBRA,0
55,P-1684,3. COSTO MANO DE OBRA,0
116,P-1707,2. COSTO MATERIALES,0


In [79]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_base.sort_values(by=['CODIGO_PROYECTO','TIPO_CUENTA']).head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
1681,1. TOTAL,1. INGRESO,0
1682,1. TOTAL,2. COSTO MATERIALES,0
1683,1. TOTAL,3. COSTO MANO DE OBRA,0
336,1. TOTAL,4. COSTO SUPERVISIÓN,0
168,P-1557,1. INGRESO,0
169,P-1557,2. COSTO MATERIALES,0
0,P-1557,3. COSTO MANO DE OBRA,0
170,P-1557,4. COSTO SUPERVISIÓN,0
171,P-1591,1. INGRESO,0
172,P-1591,2. COSTO MATERIALES,0


In [80]:
# MERGE CON CODIGO_PROYECTO

# Se crea dataframe de codigos de proyecto y se aplica mayúscula
df_cod_proyecto_base = df_cod_proyecto.applymap(lambda s: s.upper() if isinstance(s, str) else s)

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2 = pd.merge(df_libro_mayor5_base, df_cod_proyecto_base, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
19,P-1653,2. COSTO MATERIALES,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.
140,P-1718,3. COSTO MANO DE OBRA,0,PATIO COMIDAS,EBCO S.A.
8,P-1632,2. COSTO MATERIALES,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.
245,P-1667,1. INGRESO,0,ECO PLACILLA,CONSTRUCTORA ICF SPA
250,P-1669,1. INGRESO,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.
220,P-1649,4. COSTO SUPERVISIÓN,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
276,P-1683,4. COSTO SUPERVISIÓN,0,CD BP-A05,EBCO S.A.
202,P-1641,3. COSTO MANO DE OBRA,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA
227,P-1658,1. INGRESO,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.
68,P-1689,2. COSTO MATERIALES,0,CASINO BUENAVENTURA,EBCO S.A.


In [81]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"

df_libro_mayor5_base2.loc[df_libro_mayor5_base2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_libro_mayor5_base2.sample(10) 

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.
20,P-1656,2. COSTO MATERIALES,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA
199,P-1639,3. COSTO MANO DE OBRA,0,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...
294,P-1696,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.
131,P-1714,2. COSTO MATERIALES,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
310,P-1714,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
226,P-1656,4. COSTO SUPERVISIÓN,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA
21,P-1658,2. COSTO MATERIALES,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.


In [82]:
# INSPECCIÓN VISUAL 1

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL


In [83]:
# INSPECCIÓN VISUAL 2

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.


### 3.2 - TRATAMIENTO DE DESPACHOS REALIZADOS

#### 3.2.1 - Creación de dataframe copia

In [84]:
df_desp_realizados2 = df_desp_realizados.copy()
df_desp_realizados2.head(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
0,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1033,P08202340,1,"Tee 4"" Ranurada A536",UN,6296,6296
1,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1026,P08201152,7,"Tee 3"" Ranurada UL/FM",UN,4190,29330
2,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1040,P08202739,1,"Tee Mecanica 4"" x 3"" ranurada A-536 UL/F",UN,4467,4467
3,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1040,P08202656,2,"Tee Mecanica 3"" x 1 1/4"" HI NPT",UN,3001,6002
4,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1070,P08200874,1,"Tee Mecanica 3"" x 1"" HI NPT",UN,2442,2442


#### 3.2.2 - Se eliminan proyectos relacionados a STT, P-Proyectos, P-Servicios, P-Mantención

In [85]:
df_desp_realizados2["COD_PROYECTO"].unique()

array(['P-1693', 'P-Servicios', 'P-1675', 'P-1672', 'P-1667', 'P-1698',
       'P-1685', 'P-Proyectos', 'P-1668', 'P-1677', nan, 'P-1700',
       'P-1690', 'P-1676', 'P-1683', 'P-ANALISIS PROYE', 'P-1703',
       'P-1615', 'P-1701', 'P-1704', 'P-1684', 'P-1697', 'P-1691',
       'P-1686', 'P-1680', 'P-Mantención', 'P-1661', 'P-1688', 'P-1648',
       'P-1625', 'P-1689', 'P-1702', 'P-1670', 'P-1692', 'P-1681',
       'P-1645', 'P-PROYECTOS', 'P-1696', 'P-1712', 'P-1695', 'P-1682',
       'P-1669', 'P-1707', 'P-1639', 'P-1694', 'P-1714', 'P-1718',
       'P-1623', 'P-1716', 'P-1715', 'P-1673', 'P-1687', 'P-1664',
       'P-1723', 'P-1721', 'P-1699', 'P-1705', 'P-1592', 'P-1724',
       'P-1726', 'P-1709', 'P-1728', 'P-1725', 'P-1620', 'P-1652',
       'P-Oficina', 'P-1713', 'P-1658', 'P-1660', 'P-1717', 'P-1654',
       'P-1629', 'P-1663', 'P-1706', 'P-1708', 'P-1671', 'P-1626',
       'P-1649', 'P-1653', 'P-1636', 'P-1727', 'P-1638', 'P-1656',
       'P-1637', 'S-1601', 'P-1678', 'P-164

In [86]:
valores_eliminar = ['STT', 'P-Proyectos', 'P-Servicios', 'P-Mantención','P-Oficina''P-ANALISIS PROYE','P-ANALISIS PROYE','P-PROYECTOS','P-Oficina','nan']

# Convertir los valores a eliminar y la columna 'PROYECTO' a mayúsculas
valores_eliminar_mayusculas = [valor.upper() for valor in valores_eliminar]
df_desp_realizados2['PROYECTO_MAYUS'] = df_desp_realizados2['COD_PROYECTO'].str.upper()

# Filtrar las filas que no contengan los valores a eliminar
df_desp_realizados3 = df_desp_realizados2[~df_desp_realizados2['PROYECTO_MAYUS'].isin(valores_eliminar_mayusculas)]

# Eliminar la columna auxiliar 'PROYECTO_MAYUS'
df_desp_realizados3 = df_desp_realizados3.drop(columns=['PROYECTO_MAYUS'])

df_desp_realizados3.sample(10)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
11933,P-1670,VCO,2022-03-14T00:00:00-03:00,44742,1045,T14370414,5,Conduit Flexible 20mm Metalico con PVC (,M,850,4250
2460,P-1686,VCO,2022-10-04T00:00:00-03:00,47446,1009,T14380918,305,Cable UTP CAT5E SFTP Multifilar Interior Blindado,M,324,98820
20631,P-1705,VCO,2023-05-05T00:00:00-03:00,50911,1060,C08712954,8,DERIVADOR 5-1200 MHZ RIT ALAMBRICA 4D 14,UN,4556,36448
18079,P-1697,VCO,2023-04-04T00:00:00-03:00,50203,1071,S12341022,1,Placa anclaje motor Faac 746 / 844,UN,22674,22674
15083,P-1696,VCO,2023-02-02T00:00:00-03:00,49259,1044,T14390176,1,Fuente 220VAC/12VDC Aiphone PS-12S,UN,40492,40492
16749,P-1667,VCO,2022-12-06T00:00:00-03:00,48351,1013,I16431048,1000,Amarras plasticas 300mm,UN,17,17000
16929,P-1715,VCO,2023-05-08T00:00:00-03:00,50945,1129,P08200319,12,"Union copla 4"" ranurada flexible",UN,2800,33600
5899,P-1693,VCO,2022-04-29T00:00:00-03:00,45194,1013,T14370122,8,"Conduit galvanizado EMT 20 x3000mm (1/2"")",T3M,3035,24280
7395,P-1685,VCO,2022-07-04T00:00:00-04:00,46064,1128,P08202600,2,"Codo 90° 1 1/4"" NPT HI 150# A197",UN,904,1808
18509,P-1691,VCO,2022-04-13T00:00:00-03:00,45022,1023,S11290256,14,Rayo Infrarrojo PIR generico,UN,6600,92400


#### 3.2.2 - Se eliminan Tipos de movimientos diferentes a VCO y GDP

In [87]:
# Reemplazar los valores NaN en la columna "TIPO MOVIMIENTO" con una cadena vacía
df_desp_realizados3['TIPO_x0020_MOVIMIENTO'] = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].fillna('')

# Filtrar las filas donde "TIPO MOVIMIENTO" sea igual a "VCO" o "GDP" y excluir otros valores

mask = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].isin(['VCO', 'GDP'])
df_desp_realizados3 = df_desp_realizados3[mask]

In [88]:
# Se realiza la evaluación de dimensiones después de la eliminación de filas
print("\nDataframe Original antes de los cambios:")
print(df_desp_realizados.shape)
print("Dataframe después de eliminar registros:")
print(df_desp_realizados3.shape)

df_desp_realizados3.to_csv('df_desp_realizados3.csv', index = False)


Dataframe Original antes de los cambios:
(20668, 11)
Dataframe después de eliminar registros:
(12280, 11)


In [89]:
# Convertir la columna 'FECHA GUIA' a objetos de fecha y hora, manejando campos en blanco o no válidos como fechas
df_desp_realizados3['FECHA GUIA 2'] = pd.to_datetime(df_desp_realizados3['FECHA_x0020_GUIA'], errors='coerce')

# Extraer el MES y el AÑO en nuevas columnas utilizando funciones lambda y apply()
df_desp_realizados3['MES'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.month if pd.notnull(x) else None)
df_desp_realizados3['AÑO'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.year if pd.notnull(x) else None)

df_desp_realizados3.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,FECHA GUIA 2,MES,AÑO
7384,P-1685,VCO,2022-07-04T00:00:00-04:00,46064,1163,P08200273,3,"Strap 2 1/2"" x 1"" 300# NPT UL/FM",UN,2464,7392,2022-07-04 00:00:00-04:00,7,2022
19001,P-1680,VCO,2022-11-18T00:00:00-03:00,48109,1087,I17452714,1,Vaselina Vegetal,UN,7479,7479,2022-11-18 00:00:00-03:00,11,2022
5696,P-1693,VCO,2022-04-26T00:00:00-03:00,45115,1086,I15412322,18,"Teflon 3/4"" X 10M",UN,209,3762,2022-04-26 00:00:00-03:00,4,2022
8862,P-1700,VCO,2022-08-19T00:00:00-03:00,46815,1017,T14372793,70,Conduit PVC 20 x3000mm libre de halogeno,T3M,3390,237300,2022-08-19 00:00:00-03:00,8,2022
9795,P-1667,VCO,2022-11-02T00:00:00-03:00,47823,1032,T14380059,915,Cable UTP CAT5E unifilar interior Trimer,M,184,168360,2022-11-02 00:00:00-03:00,11,2022


#### 3.2.4 - Se realizan tablas pivot

In [90]:
df_desp_realizados3['COSTO_x0020_TOTAL'] = df_desp_realizados3['COSTO_x0020_TOTAL'].astype(int)
df_desp_realizados3['CANTIDAD'] = df_desp_realizados3['CANTIDAD'].astype(float)

In [91]:
df_desp_realizados3.columns

Index(['COD_PROYECTO', 'TIPO_x0020_MOVIMIENTO', 'FECHA_x0020_GUIA',
       'NUMERO_x0020_GUIA', 'PARTIDA_x0020_PRESUPUESTARIA',
       'CODIGO_x0020_PARTE', 'CANTIDAD', 'DESCRIPCI_xFFFD_N_x0020_PRODUCTO',
       'UNIDAD_x0020_DE_x0020_MEDIDA', 'COSTO_x0020_UNITARIO',
       'COSTO_x0020_TOTAL', 'FECHA GUIA 2', 'MES', 'AÑO'],
      dtype='object')

In [92]:
# Crear el pivot dataframe
df_desp_realizados4 = pd.pivot_table(df_desp_realizados3, 
                          values=['CANTIDAD', 'COSTO_x0020_TOTAL'], 
                          index=['COD_PROYECTO', 'AÑO', 'MES'], 
                          aggfunc='sum', 
                          columns='TIPO_x0020_MOVIMIENTO')

df_desp_realizados4 = pd.DataFrame(df_desp_realizados4.to_records())

df_desp_realizados4.sample(10)

,COD_PROYECTO,AÑO,MES,"('CANTIDAD', 'GDP')","('CANTIDAD', 'VCO')","('COSTO_x0020_TOTAL', 'GDP')","('COSTO_x0020_TOTAL', 'VCO')"
218,P-1677,2022,8,NaN,7896.00,NaN,4976089.00
134,P-1667,2022,8,NaN,1236.00,NaN,346309.00
504,P-1707,2022,10,1000.00,9484.00,116000.00,2262318.00
279,P-1685,2022,5,NaN,415.00,NaN,139197.00
62,P-1648,2022,5,8.00,NaN,4704.00,NaN
135,P-1667,2022,9,NaN,7410.00,NaN,9321897.00
328,P-1690,2022,3,NaN,1672.00,NaN,606458.00
85,P-1654,2022,5,200.00,NaN,10400.00,NaN
272,P-1684,2023,3,400.00,1734.00,3200.00,717239.00
202,P-1676,2022,4,NaN,6325.00,NaN,2424218.00


#### 3.2.5 - Se renombran columnas 

In [93]:
df_desp_realizados4 = df_desp_realizados4.rename(columns={'COD_PROYECTO': 'CODIGO_PROYECTO'})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'GDP')": "CANT. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'VCO')": "CANT. VCO"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'GDP')": "TOTAL. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'VCO')": "TOTAL. VCO"})

df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO
545,P-1714,2023,4,NaN,630.00,NaN,3476784.00
197,P-1675,2022,12,NaN,921.00,NaN,199147.00
459,P-1701,2023,5,37.00,NaN,175639.00,NaN
50,P-1644,2023,1,1.00,NaN,198000.00,NaN
226,P-1677,2023,5,12.00,NaN,570000.00,NaN


#### 3.2.6 - Se reemplazas columnas NaN por ceros y se crea columna de TOTAL GPD-VCO

In [94]:
df_desp_realizados4.fillna(0, inplace=True)
df_desp_realizados4['TOTAL. VCO - GDP'] = df_desp_realizados4['TOTAL. VCO'] - df_desp_realizados4['TOTAL. GDP']
df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
151,P-1669,2022,1,326.00,0.00,550176.00,0.00,-550176.00
518,P-1709,2023,5,0.00,12.00,0.00,2999784.00,2999784.00
140,P-1667,2023,3,200.00,52.00,24600.00,436660.00,412060.00
398,P-1696,2022,6,0.00,7685.00,0.00,1923690.00,1923690.00
126,P-1666,2022,7,8.00,0.00,7816.00,0.00,-7816.00


#### 3.2.7 - Se seleccionan solamente los registros del mes y año en curso 

In [95]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

# Filtrar el dataframe por año y mes
df_desp_realizados5 = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == current_month)]
print("Dimensiones después del filtro", df_desp_realizados5.shape)
df_desp_realizados5.sample(5)

2023
6
Dimensiones antes del filtro (599, 8)
Dimensiones después del filtro (42, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
510,P-1707,2023,6,0.00,5132.00,0.00,3557418.00,3557418.00
388,P-1694,2023,6,0.00,15589.00,0.00,8964664.00,8964664.00
198,P-1675,2023,6,4.00,0.00,27469.00,0.00,-27469.00
512,P-1708,2023,6,0.00,2.00,0.00,45820.00,45820.00
559,P-1716,2023,6,0.00,8.00,0.00,2308672.00,2308672.00


#### 3.2.7-2 - Se seleccionan solamente los registros del mes y año en curso 2
Se considera esta línea para poblar la vista del mes anterior.

In [96]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

mes_anterior = (datetime.now() - timedelta(days=30)).month

# Filtrar el dataframe por año y mes
df_desp_realizados5_1M = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == mes_anterior)]
print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)
df_desp_realizados5_1M.sample(5)

2023
6
Dimensiones antes del filtro (599, 8)
Dimensiones después del filtro (46, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
588,P-1726,2023,5,0.00,2127.00,0.00,9232066.00,9232066.00
317,P-1688,2023,5,3.00,316.00,135337.00,217172.00,81835.00
585,P-1725,2023,5,0.00,8020.00,0.00,6708545.00,6708545.00
396,P-1695,2023,5,15.00,8976.00,981954.00,9619052.00,8637098.00
434,P-1699,2023,5,0.00,17026.00,0.00,8687386.00,8687386.00


#### 3.2.8 - Se agrega la fila TOTAL

In [97]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5 = pd.concat([df_desp_realizados5, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TOTAL"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TIPO_CUENTA"]= "COSTO MATERIALES"

df_desp_realizados5.sample()

print("Dimensiones después del filtro", df_desp_realizados5.shape)

df_desp_realizados5.sample(5)

Dimensiones antes del filtro (42, 8)
Dimensiones después del filtro (43, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
35,P-1721,2023,6,0.00,45.00,0.00,3488722.00,3488722.00,3488722.00,COSTO MATERIALES
26,P-1707,2023,6,0.00,5132.00,0.00,3557418.00,3557418.00,3557418.00,COSTO MATERIALES
17,P-1690,2023,6,0.00,657.00,0.00,474474.00,474474.00,474474.00,COSTO MATERIALES
12,P-1680,2023,6,0.00,12842.00,0.00,6905722.00,6905722.00,6905722.00,COSTO MATERIALES
0,P-1592,2023,6,0.00,2200.00,0.00,52800.00,52800.00,52800.00,COSTO MATERIALES


#### 3.2.8-2 - Se agrega la fila TOTAL 2

In [98]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5_1M.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5_1M['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5_1M['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5_1M['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5_1M['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5_1M['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5_1M['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5_1M['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5_1M = pd.concat([df_desp_realizados5_1M, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TOTAL"]= df_desp_realizados5_1M["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TIPO_CUENTA"]= "COSTO MATERIALES"

print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)

df_desp_realizados5_1M.sample(5)

Dimensiones antes del filtro (46, 8)
Dimensiones después del filtro (47, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
44,P-1726,2023,5,0.00,2127.00,0.00,9232066.00,9232066.00,9232066.00,COSTO MATERIALES
32,P-1707,2023,5,12.00,6235.00,257844.00,5892447.00,5634603.00,5634603.00,COSTO MATERIALES
19,P-1691,2023,5,8.00,14.00,194048.00,210200.00,16152.00,16152.00,COSTO MATERIALES
22,P-1695,2023,5,15.00,8976.00,981954.00,9619052.00,8637098.00,8637098.00,COSTO MATERIALES
20,P-1692,2023,5,250.00,7060.00,360000.00,1837143.00,1477143.00,1477143.00,COSTO MATERIALES


In [99]:
df_desp_realizados5.to_csv('archivos_respaldo/df_desp_realizados5.csv', mode='a')
df_desp_realizados5_1M.to_csv('archivos_respaldo/df_desp_realizados5_1M.csv', mode='a')

### 3.4 TRATAMIENTO "PROYECTADO"

#### 3.4.1 CREACIÓN NUEVO DATAFRAME DE COPIA.

In [100]:
df_proyectado2 = df_proyectado
df_proyectado2.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.1-2 CREACIÓN NUEVO DATAFRAME DE COPIA 2.

In [101]:
df_proyectado2_1M = df_proyectado_1M
df_proyectado2_1M.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES

In [102]:
df_proyectado2 = df_proyectado2.drop(range(0, 6))
new_header = df_proyectado2.iloc[0]
df_proyectado2 = df_proyectado2[1:]
df_proyectado2.columns = new_header
df_proyectado2 = df_proyectado2.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
89,NaN,NaN,0.26,NaN,NaN
9,P-1615,ECO SANTIAGO,NaN,NaN,NaN
84,NaN,NaN,130000000,NaN,0.32
37,P-1684,GOYCOLEA,25380000,12035806,NaN
28,P-1674,Beiersdorf,NaN,NaN,NaN
23,P-1668,QUILLAYES,NaN,NaN,NaN
38,1685,MAPA,NaN,NaN,NaN
56,P-1703,SANTA VICTORIA DANACORP,NaN,NaN,NaN
87,NaN,NaN,22000000,NaN,NaN
8,P-1601,CUATRE CASAS,NaN,NaN,NaN


#### 3.4.2-2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES 2

In [103]:
df_proyectado2_1M = df_proyectado2_1M.drop(range(0, 6))
new_header = df_proyectado2_1M.iloc[0]
df_proyectado2_1M = df_proyectado2_1M[1:]
df_proyectado2_1M.columns = new_header
df_proyectado2_1M = df_proyectado2_1M.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2_1M.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
20,P-1661,SAN VICENTE,NaN,NaN,NaN
52,P-1699,VILLA PANAMERICANA LOTE B,21598229,18000000,NaN
24,P-1669,Departamental II,NaN,NaN,NaN
32,P-1678,PIRER,NaN,NaN,NaN
67,P-1716,Astara,10082518,NaN,NaN
15,P-1645,LOS TRES ANTONIO,NaN,NaN,NaN
21,P-1663,Oficinas Equifax,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN
30,P-1676,PORTAL LO CAMPINO,NaN,NaN,NaN
40,P-1687,ROJAS MAGALLANES,NaN,NaN,NaN


In [104]:
# INSPECCIÓN VISUAL 1
df_proyectado2_1M[df_proyectado2_1M["PROYECTO"]=="P-1726"]

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
77,P-1726,LA GLORIA,25429159,10000000,NaN


#### 3.4.3 TRATAMIENTO DE DATOS NAN

In [105]:
# seleccionar las columnas deseadas
df_proyectado3 = df_proyectado2[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3 = df_proyectado3.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3 = df_proyectado3.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3['INGRESO'] = df_proyectado3['INGRESO'].fillna(0)
df_proyectado3['MATERIALES'] = df_proyectado3['MATERIALES'].fillna(0)
df_proyectado3['MANO DE OBRA'] = df_proyectado3['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3['CODIGO_PROYECTO'] = df_proyectado3['CODIGO_PROYECTO'].str.upper()
df_proyectado3['PROYECTO'] = df_proyectado3['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)

df_proyectado3.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (74, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
51,P-1705,LOS CLARINES,12109554.10,4500000,0
23,P-1676,PORTAL LO CAMPINO,0.00,0,0
55,P-1709,CC PASEO SAN PEDRO,0.00,0,0
62,P-1718,PATIO DE COMIDA,24886069.78,6000000,0
59,P-1715,LO ECHEVERS,11500000.00,6000000,0
37,P-1691,ROSARIO ROSALES,0.00,0,0
70,,LA GLORIA,12000000.00,2000000,0
67,P-1723,VILLA PANAMERICANA LOTE C,85000000.00,31000000,0
41,P-1695,ESTORIL,4230000.00,3000000,0
64,P-1720,CLARO DECOMBE,0.00,0,0


In [106]:
print(df_proyectado3.columns)

Index(['CODIGO_PROYECTO', 'PROYECTO', 'INGRESO', 'MATERIALES', 'MANO DE OBRA'], dtype='object', name=6)


In [107]:
# Eliminar la columna '6'
df_proyectado3 = df_proyectado3.rename_axis(None, axis=1)
df_proyectado3 = df_proyectado3.dropna(subset=['CODIGO_PROYECTO'])

print(df_proyectado3.shape)
df_proyectado3.sample(15)

(72, 5)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
47,P-1701,ZENTENO,0.00,0,0
14,P-1663,OFICINAS EQUIFAX,0.00,0,0
44,P-1698,PCI ANDES,0.00,0,0
73,P-1729,ESTACION CENTRAL,0.00,0,0
35,P-1689,CASINO,0.00,0,0
70,,LA GLORIA,12000000.00,2000000,0
53,P-1707,LAS TRANQUERAS,12240000.00,3000000,0
19,P-1671,BARRIO PARQUE 8 Y 9,0.00,0,0
36,P-1690,ONGOLMO,5000000.00,2000000,0
16,P-1668,QUILLAYES,0.00,0,0


#### 3.4.3-2 TRATAMIENTO DE DATOS NAN 2

In [108]:
# seleccionar las columnas deseadas
df_proyectado3_1M = df_proyectado2_1M[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3_1M = df_proyectado3_1M.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3_1M = df_proyectado3_1M.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3_1M['INGRESO'] = df_proyectado3_1M['INGRESO'].fillna(0)
df_proyectado3_1M['MATERIALES'] = df_proyectado3_1M['MATERIALES'].fillna(0)
df_proyectado3_1M['MANO DE OBRA'] = df_proyectado3_1M['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3_1M['CODIGO_PROYECTO'] = df_proyectado3_1M['CODIGO_PROYECTO'].str.upper()
df_proyectado3_1M['PROYECTO'] = df_proyectado3_1M['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)

df_proyectado3_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (73, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
63,P-1719,METALPARK,0.00,0,0
67,P-1723,VILLA PANAMERICANA LOTE C,68515939.00,53000000,0
34,P-1688,SUIZA,4529751.00,1500000,0
22,P-1675,PALENA,0.00,0,0
29,NaN,BPA05,0.00,0,0
55,P-1709,CC PASEO SAN PEDRO,11127770.00,500000,0
37,P-1691,ROSARIO ROSALES,0.00,0,0
35,P-1689,CASINO,0.00,0,0
59,P-1715,LO ECHEVERS,18230799.58,6000000,0
25,P-1678,PIRER,0.00,0,0


In [109]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
70,P-1726,LA GLORIA,25429159.00,10000000,0


#### 3.4.4 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO

In [110]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso = df_proyectado3['INGRESO'].sum()
total_materiales = df_proyectado3['MATERIALES'].sum()
total_mano_obra = df_proyectado3['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3 = pd.concat([df_proyectado3, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3["TIPO_CUENTA"] = "INGRESO"
df_proyectado3["MES"] = current_month 
df_proyectado3["AÑO"] = current_year 
df_proyectado3["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3 = df_proyectado3.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3['TOTAL'] = df_proyectado3[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)
df_proyectado3.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (73, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
46,P-1702,KARUN VITACURA,0.00,0,0,0.00,INGRESO,6,2023,PROYECTADO
52,P-1708,ECO ESPAÑA,0.00,-4200000,0,-4200000.00,INGRESO,6,2023,PROYECTADO
71,P-1729,ESTACION CENTRAL,0.00,0,0,0.00,INGRESO,6,2023,PROYECTADO
43,P-1699,VILLA PANAMERICANA LOTE B,40000000.00,-22000000,0,18000000.00,INGRESO,6,2023,PROYECTADO
58,P-1716,ASTARA,0.00,0,0,0.00,INGRESO,6,2023,PROYECTADO


In [111]:
df_proyectado3['INGRESO'].sum()

864624767.7641654

#### 3.4.4-2 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO 2

In [112]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
last_month = (fecha_actual - timedelta(days=30)).month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso    = df_proyectado3_1M['INGRESO'].sum()
total_materiales = df_proyectado3_1M['MATERIALES'].sum()
total_mano_obra  = df_proyectado3_1M['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3_1M = pd.concat([df_proyectado3_1M, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3_1M["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3_1M["TIPO_CUENTA"] = "INGRESO"
df_proyectado3_1M["MES"] = last_month 
df_proyectado3_1M["AÑO"] = current_year 
df_proyectado3_1M["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3_1M = df_proyectado3_1M.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3_1M['TOTAL'] = df_proyectado3_1M[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)
df_proyectado3_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (74, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
3,P-1621,RENAULT,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
59,P-1715,LO ECHEVERS,18230799.58,-6000000,0,12230799.58,INGRESO,5,2023,PROYECTADO
52,P-1706,CALIFORNIA,17637537.00,-5000000,0,12637537.00,INGRESO,5,2023,PROYECTADO
37,P-1691,ROSARIO ROSALES,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
25,P-1678,PIRER,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO


In [113]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
70,P-1726,LA GLORIA,25429159.00,-10000000,0,15429159.00,INGRESO,5,2023,PROYECTADO


#### 3.4.5 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS. 

In [114]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4 = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)

df_proyectado4.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (73, 10)
Dimensiones solo INGRESO (73, 7)
Dimensiones solo MANO DE OBRA (73, 7)
Dimensiones solo MATERIALES (73, 7)
Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
61,P-1719,METALPARK,0.00,6,2023,PROYECTADO,INGRESO
37,P-1693,VOLTA,0.00,6,2023,PROYECTADO,MATERIALES
6,P-1638,COLON,0.00,6,2023,PROYECTADO,MANO DE OBRA
45,P-1701,ZENTENO,0.00,6,2023,PROYECTADO,MANO DE OBRA
39,P-1695,ESTORIL,0.00,6,2023,PROYECTADO,MANO DE OBRA
7,P-1642,CORINTOS,0.00,6,2023,PROYECTADO,MANO DE OBRA
64,P-1722,ECO CERRILLO,0.00,6,2023,PROYECTADO,INGRESO
61,P-1719,METALPARK,0.00,6,2023,PROYECTADO,MANO DE OBRA
4,P-1652,ABDON CIFUENTES,0.00,6,2023,PROYECTADO,MATERIALES
25,P-1678,PIRER,0.00,6,2023,PROYECTADO,INGRESO


#### 3.4.5-2 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS 2  

In [115]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3_1M.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3_1M.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3_1M.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3_1M.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4_1M = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)

df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (74, 10)
Dimensiones solo INGRESO (74, 7)
Dimensiones solo MANO DE OBRA (74, 7)
Dimensiones solo MATERIALES (74, 7)
Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
25,P-1678,PIRER,0.00,5,2023,PROYECTADO,INGRESO
1,P-1601,CUATRE CASAS,0.00,5,2023,PROYECTADO,INGRESO
35,P-1689,CASINO,0.00,5,2023,PROYECTADO,MATERIALES
18,P-1670,MK SEGURIDAS,0.00,5,2023,PROYECTADO,MATERIALES
33,P-1687,ROJAS MAGALLANES,0.00,5,2023,PROYECTADO,MATERIALES
40,P-1694,ECO HUEMUL,0.00,5,2023,PROYECTADO,MANO DE OBRA
40,P-1694,ECO HUEMUL,-7000000.00,5,2023,PROYECTADO,MATERIALES
46,P-1700,ORIZON,0.00,5,2023,PROYECTADO,MATERIALES
50,P-1704,MERCADO LIBRE,68606659.28,5,2023,PROYECTADO,INGRESO
29,0,BPA05,0.00,5,2023,PROYECTADO,MATERIALES


In [116]:
# INSPECCIÓN VISUAL 1
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,MATERIALES


#### 3.4.6 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA

In [117]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4['TIPO_CUENTA'] = df_proyectado4['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)
df_proyectado4.sample(10)

Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
7,P-1642,CORINTOS,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES
25,P-1678,PIRER,0.00,6,2023,PROYECTADO,1. INGRESO
45,P-1701,ZENTENO,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
16,P-1668,QUILLAYES,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
59,P-1717,ECO QUILIN,-200000.00,6,2023,PROYECTADO,2. COSTO MATERIALES
57,P-1715,LO ECHEVERS,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
53,P-1709,CC PASEO SAN PEDRO,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES
52,P-1708,ECO ESPAÑA,0.00,6,2023,PROYECTADO,1. INGRESO
21,P-1674,BEIERSDORF,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES
46,P-1702,KARUN VITACURA,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES


In [118]:
# TEST TOTAL "INGRESO" 

df_proyectado4_test = df_proyectado4[df_proyectado4['TIPO_CUENTA'] == '1. INGRESO']
df_proyectado4_test                                     

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
0,P-1600,TERRANOVA,0.00,6,2023,PROYECTADO,1. INGRESO
1,P-1601,CUATRE CASAS,0.00,6,2023,PROYECTADO,1. INGRESO
2,P-1615,ECO SANTIAGO,0.00,6,2023,PROYECTADO,1. INGRESO
3,P-1621,RENAULT,0.00,6,2023,PROYECTADO,1. INGRESO
4,P-1652,ABDON CIFUENTES,0.00,6,2023,PROYECTADO,1. INGRESO
5,P-1632,CHORILLOS CLOROX,0.00,6,2023,PROYECTADO,1. INGRESO
6,P-1638,COLON,0.00,6,2023,PROYECTADO,1. INGRESO
7,P-1642,CORINTOS,0.00,6,2023,PROYECTADO,1. INGRESO
8,P-1645,LOS TRES ANTONIO,0.00,6,2023,PROYECTADO,1. INGRESO
9,P-1654,EXEQUIEL FERNANDEZ 1670,0.00,6,2023,PROYECTADO,1. INGRESO


#### 3.4.6-2 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA 2

In [119]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4_1M['TIPO_CUENTA'] = df_proyectado4_1M['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)
df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
34,P-1688,SUIZA,-1500000.00,5,2023,PROYECTADO,2. COSTO MATERIALES
42,P-1696,SANTA VICTORIA,0.00,5,2023,PROYECTADO,1. INGRESO
58,P-1714,DEPARTAMENTAL II,2436155.00,5,2023,PROYECTADO,1. INGRESO
48,P-1702,KARUN VITACURA,0.00,5,2023,PROYECTADO,1. INGRESO
21,P-1674,BEIERSDORF,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
33,P-1687,ROJAS MAGALLANES,0.00,5,2023,PROYECTADO,1. INGRESO
6,P-1638,COLON,0.00,5,2023,PROYECTADO,1. INGRESO
68,P-1724,185 VOLTA,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
67,P-1723,VILLA PANAMERICANA LOTE C,-53000000.00,5,2023,PROYECTADO,2. COSTO MATERIALES
3,P-1621,RENAULT,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES


In [120]:
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,1. INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,2. COSTO MATERIALES


In [121]:
df_proyectado4.to_csv('archivos_respaldo/df_proyectado4.csv', mode='a')
df_proyectado4_1M.to_csv('archivos_respaldo/df_proyectado4_1M.csv', mode='a')

### 3.5 TRATAMIENTO LIBRO MAYOR AÑOS ANTERIORES

#### 3.5.1 - REVISIÓN DE LOS DATOS

In [122]:
df_libro_mayor_old.shape

(17368, 13)

In [123]:
df_libro_mayor_old.describe()

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
count,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368
unique,10,2,12,531,5,1312,3453,134,1500,5119,8462,1714,2
top,4101010-000,2022,9,12/31/22 0:00,CIN,1252,REMUNERACIONES DIRECTAS PROYECTO,,76525290-3,,0,0,
freq,12133,9797,1775,252,11773,251,1359,699,406,1958,3199,15265,17359


#### 3.5.2 - Estandarización campo "Debito" y Crédito

In [124]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor_old['Debito_2'] = pd.to_numeric(df_libro_mayor_old['Debito'], errors='coerce')
df_libro_mayor_old['Credito_2'] = pd.to_numeric(df_libro_mayor_old['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor_old['Credito_2'] = df_libro_mayor_old['Credito_2'].fillna(0)

df_libro_mayor_old.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.3 - Creación campo Codigo_proyecto

In [125]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['Primer_Analisis']

cols = df_libro_mayor_old.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('CODIGO_PROYECTO'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor_old = df_libro_mayor_old.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['CODIGO_PROYECTO'].str.upper()

df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.4 - Creación de campo total

In [126]:
df_libro_mayor_old['TOTAL'] = df_libro_mayor_old['Credito_2'] - df_libro_mayor_old['Debito_2']
df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00,0.00


#### 3.5.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [127]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return '1. INGRESO'
    elif cta_contable == "4101010-000":
        return '2. COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return '3. COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return '4. COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor_old['Tipo_cuenta'] = df_libro_mayor_old['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor_old.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
11649,P-1704,4101010-000,2022,7,7/5/22 0:00,CIN,495,Broca Guia p/porta Sierra Copa tomas A04,P-1704,H21582716,46098,12270,0,,12270.00,0.00,-12270.00,2. COSTO MATERIALES
7860,P-1691,3101030-000,2022,5,5/26/22 0:00,CVT,386,EP N° 5 - OBRA: ROSARIO ROSALES,P-1691,76009867-1,12521,0,2178750,,0.00,2178750.00,2178750.00,1. INGRESO
13866,P-1682,4101010-000,2022,10,10/7/22 0:00,CIN,157,"Tornillo Framer 7 x 7/16"" Negro",P-1682,I17440300,47525,1600,0,,1600.00,0.00,-1600.00,2. COSTO MATERIALES
7746,P-1689,3101030-000,2022,3,3/29/22 0:00,CVT,509,Adicional carretes y bases.Obra Casino Buenave...,P-1689,76525290-3,12342,0,1198700,,0.00,1198700.00,1198700.00,1. INGRESO
5905,P-1672,4101010-000,2021,10,10/25/21 0:00,CIN,717,Amarras plasticas 300mm,P-1672,I16431048,43525,4700,0,,4700.00,0.00,-4700.00,2. COSTO MATERIALES


In [128]:
df_libro_mayor_old['Tipo_cuenta'].unique()

array(['1. INGRESO', '2. COSTO MATERIALES', 'OTRA',
       '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'], dtype=object)

In [129]:
# Revisión de los datos creados

df_libro_mayor_old_piv = pd.pivot_table(df_libro_mayor_old, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old_piv = pd.DataFrame(df_libro_mayor_old_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_old_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
83,4. COSTO SUPERVISIÓN,2021,9,0,2758892,-2758892
107,OTRA,2021,9,0,9550423,-9550423
70,3. COSTO MANO DE OBRA,2022,8,0,70301037,-70301037
76,4. COSTO SUPERVISIÓN,2021,2,0,2610884,-2610884
68,3. COSTO MANO DE OBRA,2022,6,0,51888874,-51888874
25,2. COSTO MATERIALES,2021,10,1846393,21300701,-19454308
75,4. COSTO SUPERVISIÓN,2021,12,0,2799714,-2799714
23,1. INGRESO,2022,9,456650724,178539015,278111709
59,3. COSTO MANO DE OBRA,2021,9,0,0,0
16,1. INGRESO,2022,2,81398021,38892002,42506019


#### 3.5.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [130]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor_old2 = df_libro_mayor_old.copy()

# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)


Proyectos disponibles antes de la reducción : 
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANALISIS PROYE' 

##### 3.5.6.1 - Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [131]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor_old2_2 = df_libro_mayor_old2.copy()

# Filtrar el DataFrame
df_libro_mayor_old2_2 = df_libro_mayor_old2_2[(df_libro_mayor_old2_2['CODIGO_PROYECTO'] == 'P-OFICINA') & (df_libro_mayor_old2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor_old2_2


,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN


In [132]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor_old2 = df_libro_mayor_old2.dropna(subset=['CODIGO_PROYECTO'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE', '13056802-5', '16411600-K', '18800250-1','P-MANTENCIÓN','18800250','13056802','16411600']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask = df_libro_mayor_old2['CODIGO_PROYECTO'].isin(valores_a_eliminar)

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor_old2 = df_libro_mayor_old2[~mask]


In [133]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old.shape)
print(df_libro_mayor_old["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())


Proyectos disponibles antes de la reducción : 
(17368, 18)
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANAL

#### 3.5.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [134]:
df_libro_mayor_old2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2['Año'].astype(str) + '-' + df_libro_mayor_old2['Mes'].astype(str) + '-1')
df_libro_mayor_old2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
3274,P-1670,4101010-000,2021,4,4/14/21 0:00,CIN,419,Cinta Aisladora 3M Temflex 1500,P-1670,I16430107,41652,5985,0,,5985.00,0.00,-5985.00,2. COSTO MATERIALES,2021-04-01
6133,P-1661,4101010-000,2021,11,11/17/21 0:00,CIN,372,Citofono IG 0TD/6H/A Aiphone 6 llamadas,P-1661,C04090522,43723,39173,0,,39173.00,0.00,-39173.00,2. COSTO MATERIALES,2021-11-01
16431,P-1692,4150002-000,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES DIRECTAS PROYECTO,P-1692,20148177,,180584,0,,180584.00,0.00,-180584.00,3. COSTO MANO DE OBRA,2022-10-01
2094,P-1636,4101010-000,2021,2,2/17/21 0:00,CIN,794,"Magnetico Sobrepuesto 7/8"" Cafe",P-1636,S11290200,40771,7410,0,,7410.00,0.00,-7410.00,2. COSTO MATERIALES,2021-02-01
6443,P-1685,4101010-000,2021,12,12/14/21 0:00,CIN,516,"Tapa Tornillo 1/4"" NPT HE FE 150#",P-1685,P08201651,43988,1090,0,,1090.00,0.00,-1090.00,2. COSTO MATERIALES,2021-12-01


##### 3.5.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [135]:
df_libro_mayor_old2_2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2_2['Año'].astype(str) + '-' + df_libro_mayor_old2_2['Mes'].astype(str) + '-1')
df_libro_mayor_old2_2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN,2022-11-01
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN,2022-11-01
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN,2022-12-01
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN,2022-10-01
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN,2022-09-01


In [136]:
df_libro_mayor_old2_2['Tipo_cuenta'].unique()

array(['4. COSTO SUPERVISIÓN'], dtype=object)

In [137]:
df_libro_mayor_old2_2[df_libro_mayor_old2_2["Primer_Analisis"] == "P-1673"]

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2


#### 3.5.8 - Se realiza pivote final del dataframe

In [138]:
# Se exploran las columnas:
df_libro_mayor_old2.columns

Index(['CODIGO_PROYECTO', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Num_Referencia',
       'Debito', 'Credito', 'Comentario ', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object', name=0)

In [139]:
# crear pivot table sin margenes

df_libro_mayor_old2_piv = pd.pivot_table(df_libro_mayor_old2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_piv = pd.DataFrame(df_libro_mayor_old2_piv.to_records())

df_libro_mayor_old2_piv.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
708,P-1667,3. COSTO MANO DE OBRA,2021-04-01 00:00:00,2021,4,0,0,0
628,P-1664,3. COSTO MANO DE OBRA,2021-01-01 00:00:00,2021,1,0,0,0
1314,P-1691,3. COSTO MANO DE OBRA,2022-02-01 00:00:00,2022,2,0,797482,-797482
873,P-1672,2. COSTO MATERIALES,2021-11-01 00:00:00,2021,11,583740,2546716,-1962976
366,P-1649,2. COSTO MATERIALES,2021-07-01 00:00:00,2021,7,282025,80528,201497


In [140]:
df_libro_mayor_old2_piv[df_libro_mayor_old2_piv["CODIGO_PROYECTO"] == "P-1704"]

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1523,P-1704,1. INGRESO,2022-07-01 00:00:00,2022,7,179643612,0,179643612
1524,P-1704,1. INGRESO,2022-08-01 00:00:00,2022,8,84292194,0,84292194
1525,P-1704,1. INGRESO,2022-09-01 00:00:00,2022,9,97802473,34754970,63047503
1526,P-1704,1. INGRESO,2022-10-01 00:00:00,2022,10,49425495,0,49425495
1527,P-1704,1. INGRESO,2022-11-01 00:00:00,2022,11,42952357,0,42952357
1528,P-1704,1. INGRESO,2022-12-01 00:00:00,2022,12,89246807,42952357,46294450
1529,P-1704,2. COSTO MATERIALES,2022-05-01 00:00:00,2022,5,0,273115,-273115
1530,P-1704,2. COSTO MATERIALES,2022-06-01 00:00:00,2022,6,518865,27455060,-26936195
1531,P-1704,2. COSTO MATERIALES,2022-07-01 00:00:00,2022,7,0,24099085,-24099085
1532,P-1704,2. COSTO MATERIALES,2022-08-01 00:00:00,2022,8,0,26228061,-26228061


In [141]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor_old2_piv.shape

(1632, 8)

##### 3.5.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [142]:
# crear pivot table sin margenes

df_libro_mayor_old2_2_piv = pd.pivot_table(df_libro_mayor_old2_2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_2_piv = pd.DataFrame(df_libro_mayor_old2_2_piv.to_records())

df_libro_mayor_old2_2_piv = df_libro_mayor_old2_2_piv[df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] = df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'].replace('P-OFICINA', 'TOTAL')

df_libro_mayor_old2_2_piv.sample()

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
3,TOTAL,4. COSTO SUPERVISIÓN,2022-12-01 00:00:00,2022,12,0,4656619,-4656619


#### 3.5.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta


In [143]:
df_libro_mayor_old2_piv2 = pd.pivot_table(df_libro_mayor_old2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_old2_piv2 = pd.DataFrame(df_libro_mayor_old2_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_old2_piv2.insert(loc=0, column='CODIGO_PROYECTO', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != '']

df_libro_mayor_old2_piv2.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
24,TOTAL,1. INGRESO,2022-12-01 00:00:00,2022,12,346131023,100849557,245281466
19,TOTAL,1. INGRESO,2022-07-01 00:00:00,2022,7,463367517,112941494,350426023
62,TOTAL,3. COSTO MANO DE OBRA,2022-02-01 00:00:00,2022,2,0,14497984,-14497984
79,TOTAL,4. COSTO SUPERVISIÓN,2021-07-01 00:00:00,2021,7,0,2784855,-2784855
86,TOTAL,OTRA,2021-02-01 00:00:00,2021,2,0,13905678,-13905678


##### 3.5.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [144]:
df_libro_mayor_old2_piv_fin = pd.concat([df_libro_mayor_old2_piv2, df_libro_mayor_old2_2_piv], ignore_index=True)
df_libro_mayor_old2_piv_fin.sample(10)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
81,TOTAL,4. COSTO SUPERVISIÓN,2021-10-01 00:00:00,2021,10,0,3047422,-3047422
18,TOTAL,1. INGRESO,2022-07-01 00:00:00,2022,7,463367517,112941494,350426023
50,TOTAL,3. COSTO MANO DE OBRA,2021-03-01 00:00:00,2021,3,0,527823,-527823
80,TOTAL,4. COSTO SUPERVISIÓN,2021-09-01 00:00:00,2021,9,0,2758892,-2758892
21,TOTAL,1. INGRESO,2022-10-01 00:00:00,2022,10,368845820,102127936,266717884
3,TOTAL,1. INGRESO,2021-04-01 00:00:00,2021,4,165417555,36206601,129210954
82,TOTAL,4. COSTO SUPERVISIÓN,2021-11-01 00:00:00,2021,11,0,2862818,-2862818
9,TOTAL,1. INGRESO,2021-10-01 00:00:00,2021,10,188858961,118021388,70837573
107,TOTAL,4. COSTO SUPERVISIÓN,2022-11-01 00:00:00,2022,11,0,5089343,-5089343
103,TOTAL,OTRA,2022-10-01 00:00:00,2022,10,0,9743100,-9743100


#### 3.5.10 - Se concatenan los 2 dataframes resultantes

In [145]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor_old2_piv_fin.shape)

Dimensiones del dataframe antes de la concatenación : 
(109, 8)


In [146]:
df_libro_mayor_old3 = pd.concat([df_libro_mayor_old2_piv, df_libro_mayor_old2_piv_fin])
df_libro_mayor_old3.sort_values(by=['CODIGO_PROYECTO']).sample(5)
print(df_libro_mayor_old3.shape)

(1741, 8)


#### 3.5.11 - Estandarización de campos 

In [147]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor_old3 = df_libro_mayor_old3.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor_old3['AÑO'] = pd.to_numeric(df_libro_mayor_old3['AÑO'], downcast='integer')
df_libro_mayor_old3['MES'] = pd.to_numeric(df_libro_mayor_old3['MES'], downcast='integer')

df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
1391,P-1695,3. COSTO MANO DE OBRA,2022-10-01 00:00:00,2022.00,10.00,0,3433480,-3433480
715,P-1667,3. COSTO MANO DE OBRA,2021-11-01 00:00:00,2021.00,11.00,0,0,0
478,P-1656,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,0,15000,-15000
927,P-1674,1. INGRESO,2021-08-01 00:00:00,2021.00,8.00,689526,0,689526
62,TOTAL,3. COSTO MANO DE OBRA,2022-03-01 00:00:00,2022.00,3.00,0,21750062,-21750062
46,P-1592,2. COSTO MATERIALES,2021-11-01 00:00:00,2021.00,11.00,24389,0,24389
616,P-1664,2. COSTO MATERIALES,2021-05-01 00:00:00,2021.00,5.00,0,1135380,-1135380
774,P-1669,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,0,2564575,-2564575
1187,P-1685,3. COSTO MANO DE OBRA,2022-05-01 00:00:00,2022.00,5.00,0,1013320,-1013320
1089,P-1680,2. COSTO MATERIALES,2022-11-01 00:00:00,2022.00,11.00,52901,8928577,-8875676


#### 3.5.14 Creación de columna MARGEN

In [148]:
df_libro_mayor_old3_2 = df_libro_mayor_old3.copy()

#df_libro_mayor4_2['TOTAL'] = pd.to_numeric(df_libro_mayor4_2['AÑO'], downcast='float')

df_margen = df_libro_mayor_old3_2.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2035/3531166699.py:6: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,MES,MARGEN
456,P-1681,9.00,0.00
394,P-1674,6.00,0.00
165,P-1645,5.00,0.00
570,P-1694,10.00,0.00
167,P-1645,7.00,0.00
569,P-1694,9.00,0.00
110,P-1638,3.00,0.00
561,P-1693,6.00,0.00
524,P-1690,1.00,0.00
472,P-1684,3.00,0.00


In [149]:
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_libro_mayor_old3.shape)
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_margen.shape)

Dimensiones del dataframe antes del merge : 
(1741, 8)
Dimensiones del dataframe antes del merge : 
(694, 3)


In [150]:
df_libro_mayor_old3 = pd.merge(df_libro_mayor_old3, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')
#EJEMPLO
#df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor_old3[df_libro_mayor_old3["CODIGO_PROYECTO"]=="TOTAL"]

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor_old3['CODIGO_PROYECTO'] = df_libro_mayor_old3['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

In [151]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor_old3.shape)

Dimensiones del dataframe después de la concatenación : 
(1741, 9)


In [152]:
df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
229,P-1639,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,0,10000,-10000,0.00
1085,P-1680,2. COSTO MATERIALES,2022-07-01 00:00:00,2022.00,7.00,0,4718455,-4718455,0.00
357,P-1649,1. INGRESO,2021-04-01 00:00:00,2021.00,4.00,10124374,0,10124374,0.00
501,P-1658,2. COSTO MATERIALES,2021-02-01 00:00:00,2021.00,2.00,0,540000,-540000,0.00
1700,1. TOTAL,3. COSTO MANO DE OBRA,2022-09-01 00:00:00,2022.00,9.00,0,74019234,-74019234,0.00
539,P-1660,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,0,822286,-822286,0.00
350,P-1648,2. COSTO MATERIALES,2021-09-01 00:00:00,2021.00,9.00,51634,0,51634,0.00
1304,P-1691,2. COSTO MATERIALES,2022-03-01 00:00:00,2022.00,3.00,209296,891210,-681914,0.00
1230,P-1688,1. INGRESO,2022-12-01 00:00:00,2022.00,12.00,2918133,0,2918133,0.00
830,P-1670,3. COSTO MANO DE OBRA,2022-04-01 00:00:00,2022.00,4.00,0,95979,-95979,0.00


In [153]:
df_libro_mayor_old3.to_csv('archivos_respaldo/df_libro_mayor_old3.csv', mode='a')

### 3.6 TRATAMIENTO DE "PLANILLA PRESUPUESTO"

In [154]:
# Se crea una copia del dataframe llamado "presupuesto"
df_presupuesto2 = df_presupuesto.copy()

In [155]:
# Eliminar primera fila completa
df_presupuesto2 = df_presupuesto2.iloc[1:]

# Filtrar y mantener solo las columnas deseadas
df_presupuesto2 = df_presupuesto2[['PROYECTO', 'MONTO EN $']]

df_presupuesto2 = df_presupuesto2.rename(columns={'PROYECTO': 'CODIGO_PROYECTO', 'MONTO EN $': 'VALOR'})

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR
5,P-1654,23880352.49
6,P-1655,51593018.53
23,P-1679,169118040.00
34,P-1690,64872885.78
58,P-1716,121165036.91
63,P-1721,58149422.75
59,P-1717,170727176.42
49,P-1705,65889598.10
39,P-1695,94918400.00
12,P-1668,37160752.60


In [156]:
# Se agregan nuevas columnas tipo de cuenta y Origen.

df_presupuesto2["TIPO_CUENTA"]= "1. INGRESO"
df_presupuesto2["ORIGEN"]= "PLANILLA PRESUPUESTO"

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR,TIPO_CUENTA,ORIGEN
4,P-1647,29482131.00,1. INGRESO,PLANILLA PRESUPUESTO
1,P-1632,138107706.03,1. INGRESO,PLANILLA PRESUPUESTO
56,P-1714,61454685.57,1. INGRESO,PLANILLA PRESUPUESTO
34,P-1690,64872885.78,1. INGRESO,PLANILLA PRESUPUESTO
20,P-1676,153113097.14,1. INGRESO,PLANILLA PRESUPUESTO
65,P-1723,328203511.36,1. INGRESO,PLANILLA PRESUPUESTO
47,P-1703,56316406.63,1. INGRESO,PLANILLA PRESUPUESTO
46,P-1702,0.00,1. INGRESO,PLANILLA PRESUPUESTO
59,P-1717,170727176.42,1. INGRESO,PLANILLA PRESUPUESTO
51,P-1707,67945807.16,1. INGRESO,PLANILLA PRESUPUESTO


### 3.7 TRATAMIENTO DE "TABLA PRESUPUESTO"

In [157]:
# Se crea una copia del dataframe para realizar el tratamiento.

df_tabla_presupuesto2 = df_tabla_prespuesto.copy()
print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.sample(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13831, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
9655,P-1696,1093,TC-SOPORTE MASTIL L ZINC+CPR ATORNILLABLE 300MM,,2,5226,10452
3446,P-1633,1075,"""CCTV IP-Monitor 21"""" """,,1,60000,60000
5993,P-1657,1051,"Sala de Bomba-Valvula OS&Y 6""""""",,4,200000,800000
12237,P1718,1109,"""Manifold-Copla 2"""" """,UNIDA,12,2603,31236
10881,P-1708,1013,Detec-Otros materiales,Gl,1,508933,508933


In [158]:
# Se estandarizan los tipos de campo y se reemplazan valores NaN

df_tabla_presupuesto2['COSTO_UNITARIO'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_UNITARIO'], errors='coerce')
df_tabla_presupuesto2['COSTO_TOTAL'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_TOTAL'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_tabla_presupuesto2['COSTO_UNITARIO'] = df_tabla_presupuesto2['COSTO_UNITARIO'].fillna(0)
df_tabla_presupuesto2['COSTO_TOTAL'] = df_tabla_presupuesto2['COSTO_TOTAL'].fillna(0)

print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.head(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13831, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00


In [159]:
#### Creación de campo tipo_costo basado en el valor del a columna "PRODUCTO"

def clasificar_tipo_costo(producto):
    if   producto.upper().startswith("MANO DE OBRA"):
        return "3. COSTO MANO DE OBRA"
    elif producto.upper().startswith("SUPER"):
        return "4. COSTO SUPERVISIÓN"
    else:
        return "2. COSTO MATERIALES"

df_tabla_presupuesto2['TIPO_COSTO'] = df_tabla_presupuesto2['PRODUCTO'].apply(clasificar_tipo_costo)

df_tabla_presupuesto2.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL,TIPO_COSTO
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00,2. COSTO MATERIALES
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00,2. COSTO MATERIALES
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00,2. COSTO MATERIALES
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00,2. COSTO MATERIALES
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00,2. COSTO MATERIALES


In [160]:
# crear pivot table sin margenes

df_tabla_presupuesto3 = pd.pivot_table(df_tabla_presupuesto2, values=['COSTO_TOTAL'], index=['CODIGO_PROYECTO','TIPO_COSTO'], aggfunc='sum', fill_value=0)

df_tabla_presupuesto3 = pd.DataFrame(df_tabla_presupuesto3.to_records())
df_tabla_presupuesto3.sample(10)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL
202,P-1669,3. COSTO MANO DE OBRA,9423692.00
3,P-1603,2. COSTO MATERIALES,13744197.00
91,P-1632,3. COSTO MANO DE OBRA,43457804.00
145,P-1650,3. COSTO MANO DE OBRA,0.00
151,P-1652,3. COSTO MANO DE OBRA,7363407.00
316,P-1709,3. COSTO MANO DE OBRA,8592182.00
33,P-1613,2. COSTO MATERIALES,21496374.00
15,P-1607,2. COSTO MATERIALES,2785933.00
134,P-1646,4. COSTO SUPERVISIÓN,1226580.00
333,P-1715,2. COSTO MATERIALES,180244214.00


In [161]:

# Se crea columna VALOR y se considera el valor en negativo.
df_tabla_presupuesto3['TOTAL'] = df_tabla_presupuesto3['COSTO_TOTAL'].apply(lambda x: x * -1 if x != 0 else 0)

# Se crea la columna de ORIGEN del archivo
df_tabla_presupuesto3["ORIGEN"] = "TABLA_PRESUPUESTO"

print("DIMENSIÓN DE LA TABLA ES", df_tabla_presupuesto3.shape)
df_tabla_presupuesto3.sample(5)

DIMENSIÓN DE LA TABLA ES (388, 5)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL,TOTAL,ORIGEN
140,P-1648,4. COSTO SUPERVISIÓN,8171095.00,-8171095.00,TABLA_PRESUPUESTO
130,P-1645,3. COSTO MANO DE OBRA,2547080.00,-2547080.00,TABLA_PRESUPUESTO
282,P-1697,2. COSTO MATERIALES,43150692.00,-43150692.00,TABLA_PRESUPUESTO
268,P-1691,4. COSTO SUPERVISIÓN,2099697.00,-2099697.00,TABLA_PRESUPUESTO
253,P-1686,3. COSTO MANO DE OBRA,19743989.00,-19743989.00,TABLA_PRESUPUESTO


In [162]:
# INSPECCIÓN VISUAL
df_tabla_presupuesto_testing = pd.pivot_table(df_tabla_presupuesto3, values=['TOTAL'], index=['TIPO_COSTO'], aggfunc='sum', fill_value=0)
df_tabla_presupuesto_testing

,TOTAL
TIPO_COSTO,
2. COSTO MATERIALES,-5745413332.04
3. COSTO MANO DE OBRA,-1482885459.00
4. COSTO SUPERVISIÓN,-311477854.00


In [163]:
df_tabla_presupuesto3.to_csv('archivos_respaldo/df_tabla_presupuesto3.csv', mode='a')

## 4.- **MERGE Y TRATAMIENTO DE DATAFRAMES 

### 4.1 - **MERGE PARA -> CUADRO FINANCIERO PROYECTOS 
Merge entre Dataframes: df_libro_mayor5 y df_cod_proyecto

In [164]:
print("Dimensiones antes del merge:")
print(df_libro_mayor5.shape)

df_libro_mayor6 = pd.merge(df_libro_mayor5, df_cod_proyecto, on=['CODIGO_PROYECTO'], how='left')

print("Dimensiones después del merge:")
print(df_libro_mayor6.shape)

Dimensiones antes del merge:
(2621, 9)
Dimensiones después del merge:
(2621, 11)


In [165]:
# Reemplazar los valores NaN por "TOTAL" cuando el CODIGO_PROYECTO sea "TOTAL"

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_libro_mayor6[cols_to_replace] = df_libro_mayor6[cols_to_replace].replace('TOTAL', '1. TOTAL')

# Quizás el código anterior sea redundante.
mask = df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'
df_libro_mayor6.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Se elimina la columna MERGE la que es innecesaria tener hasta ahora
df_libro_mayor6 = df_libro_mayor6.drop('MARGEN', axis=1)

df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]== "1. TOTAL"].head(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
558,1. TOTAL,,,NaN,NaN,2293294225,1935963462,357330763,1. TOTAL,1. TOTAL
559,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,1. TOTAL,1. TOTAL
560,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,1. TOTAL,1. TOTAL
561,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,1. TOTAL,1. TOTAL
562,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,1. TOTAL,1. TOTAL


#### 4.1.1 - AGREGACIÓN DE NUEVA COLUMNA DE TOTAL PARA AGREGAR EN GRÁFICO.
Se agrega nueva columna de total pero considerando el valor de los tipos de costo con valores positivos.

In [166]:
# Crear la nueva columna 'TOTAL_2' con los valores de 'TOTAL' positivos
df_libro_mayor6['TOTAL_2'] = np.where(df_libro_mayor6['TIPO_CUENTA'] != 'INGRESO', np.abs(df_libro_mayor6['TOTAL']), df_libro_mayor6['TOTAL'])
df_libro_mayor6.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
1700,P-1689,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Casino Buenaventura,EBCO S.A.,0
960,P-1645,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada,0
961,P-1645,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada,0
566,1. TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88061320,-85296908,1. TOTAL,1. TOTAL,85296908
2615,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,0,0,1. TOTAL,1. TOTAL,0
2586,P-1729,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0,NaN,NaN,0
643,P-1592,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Nueva Costanera 3300,EBCO S.A.,0
694,P-1611,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,Edificio Plaza Egaña,EBCO S.A.,0
1051,P-1656,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Mirador Club Hípico,Constructora Vital Edificacion SPA,0
2583,P-1729,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,NaN,NaN,0


#### 4.1.2 - AJUSTES DE COLUMNAS 

In [167]:
# Renombrar los valores de la columna CODIGO_PROYECTO
df_libro_mayor6['TIPO_CUENTA'] = df_libro_mayor6['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor6['CODIGO_PROYECTO'] = df_libro_mayor6['CODIGO_PROYECTO'].replace({
    'TOTAL': '1. TOTAL'})

In [168]:
# Ordenar el DataFrame por la columna CODIGO_PROYECTO
df_libro_mayor6 = df_libro_mayor6.sort_values('CODIGO_PROYECTO')

# Mover la fila con CODIGO_PROYECTO = '1. TOTAL' al primer lugar
df_libro_mayor6 = pd.concat([df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'],
                            df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] != '1. TOTAL']])

df_libro_mayor6['NOMBRE_OBRA'] = df_libro_mayor6['NOMBRE_OBRA'].str.upper()
df_libro_mayor6['NOMBRE_CLIENTE'] = df_libro_mayor6['NOMBRE_CLIENTE'].str.upper()

# Reiniciar los índices del DataFrame
df_libro_mayor6 = df_libro_mayor6.reset_index(drop=True)
df_libro_mayor6.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
705,P-1666,1. INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD AXIONLONG,EBCO S.A.,0
113,P-1592,2. COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,47815,0,47815,NUEVA COSTANERA 3300,EBCO S.A.,47815
778,P-1669,3. COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0
1711,P-1700,2. COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,5162689,-5162689,CD ORIZON,EBCO S.A.,5162689
1909,P-1706,3. COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0


#### 4.1.3 - CREACIÓN DE COLUMNA MARGEN

In [169]:
df_margen = df_libro_mayor6.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

#warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

# mostrar el DataFrame resultante
df_margen.sample(10)

df_libro_mayor6 = pd.merge(df_libro_mayor6, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# INSPECCIÓN VISUAL 1
df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]=="P-1704"]


/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2035/1823500159.py:2: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2035/1823500159.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2,MARGEN
1808,P-1704,2. COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1809,P-1704,1. INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1810,P-1704,4. COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.66
1811,P-1704,4. COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1812,P-1704,3. COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1813,P-1704,4. COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1814,P-1704,3. COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1815,P-1704,2. COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1816,P-1704,1. INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1817,P-1704,4. COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04


In [170]:
df_libro_mayor6.to_csv('archivos_respaldo/df_libro_mayor6.csv', mode='a')

### 4.2 - **MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO
MERGE ENTRE df_libro_mayor_ytd_closed_piv y df_cod_proyecto

In [171]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_piv.shape

Dimensiones antes del merge:


(340, 3)

In [172]:
df_ytd_cerrado = df_libro_mayor_ytd_closed_piv.copy()

In [173]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones mes_en_curso antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado = df_ytd_cerrado.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones mes_en_curso2 antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

Dimensiones mes_en_curso antes de cambios:  (340, 3)
Dimensiones mes_en_curso2 antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
94,P-1661,COSTO SUPERVISIÓN,0,San Vicente de Paul,DANACORP S.A
269,P-1712,COSTO MATERIALES,-18624205,Centro Comercial Viña Outlet Park,EBCO S.A.
95,P-1661,INGRESO,0,San Vicente de Paul,DANACORP S.A
247,P-1704,INGRESO,220776196,CD Mercado Libre Mezzanina,EBCO S.A.
135,P-1674,INGRESO,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.
0,1. TOTAL,COSTO MANO DE OBRA,-478147965,NaN,NaN
215,P-1695,INGRESO,28719770,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
258,P-1707,COSTO SUPERVISIÓN,0,Las Tranqueras,Constructora Santolaya Ltda.
121,P-1670,COSTO MATERIALES,6945,MK II Seguridad y Redes,Comercial K SpA.
216,P-1696,COSTO MANO DE OBRA,-15747680,Santa Victoria - Terracorp,Constructora Terracorp S.A.


In [174]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado['TIPO_CUENTA'] = df_ytd_cerrado['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
59,P-1643,1. INGRESO,0,Chorrillos M4,Constructora Suksa S.A.
105,P-1666,2. COSTO MATERIALES,-89550,CD Axionlong,EBCO S.A.
111,P-1667,1. INGRESO,0,Eco Placilla,Constructora ICF SPA
25,P-1612,2. COSTO MATERIALES,30258,Barrio Parque Santiago 7,CONSTRUCTORA ALTURAS DEL SUR S.A
236,P-1701,3. COSTO MANO DE OBRA,-891528,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.
237,P-1701,2. COSTO MATERIALES,-1111637,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.
73,P-1649,2. COSTO MATERIALES,45255,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
31,P-1620,1. INGRESO,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.
325,P-1726,2. COSTO MATERIALES,-23539593,NaN,NaN
257,P-1707,2. COSTO MATERIALES,-8495556,Las Tranqueras,Constructora Santolaya Ltda.


In [175]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-605032841,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,NaN,NaN
3,1. TOTAL,1. INGRESO,1579362409,NaN,NaN


In [176]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado[cols_to_replace] = df_ytd_cerrado[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
154,P-1680,4. COSTO SUPERVISIÓN,0,Nueva Uno,Constructora Tecsa S.A.
285,P-1716,2. COSTO MATERIALES,-26152479,CD Astara,Astara Logística Chile SpA.
64,P-1645,3. COSTO MANO DE OBRA,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada
298,P-1719,4. COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.


In [177]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605032841,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL


#### 4.2.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [178]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   : ", df_ytd_cerrado.shape)

df_ytd_cerrado2 = df_ytd_cerrado.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
7,P-1557,1. INGRESO,0
18,P-1603,4. COSTO SUPERVISIÓN,0
259,P-1707,1. INGRESO,11263421
100,P-1664,3. COSTO MANO DE OBRA,0
53,P-1642,2. COSTO MATERIALES,8880


In [179]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
130,P-1714,3. COSTO MANO DE OBRA,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
293,P-1695,4. COSTO SUPERVISIÓN,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
92,P-1698,2. COSTO MATERIALES,0,CD ANDES,EBCO S.A.
216,P-1648,3. COSTO MANO DE OBRA,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.
281,P-1686,4. COSTO SUPERVISIÓN,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.


In [180]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado3 = df_libro_mayor5_base2.merge(df_ytd_cerrado2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
290,P-1693,1. INGRESO,0,VOLTA V0800,ECOBIO SPA,0
214,P-1645,4. COSTO SUPERVISIÓN,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0
333,P-1729,1. INGRESO,0,NaN,NaN,0
90,P-1697,2. COSTO MATERIALES,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-17660281
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0


In [181]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605032841
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965


In [182]:
# INSPECCIÓN VISUAL 2

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066
154,P-1724,2. COSTO MATERIALES,0,NaN,NaN,-7660814
155,P-1724,1. INGRESO,0,NaN,NaN,28935273
327,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN,0


#### 4.2.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [183]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
51,P-1693,0.00
23,P-1661,0.00
70,P-1715,0.24
6,P-1612,0.00
11,P-1639,0.00
31,P-1672,0.98
68,P-1713,-0.38
62,P-1705,0.05
20,P-1656,0.00
13,P-1642,0.00


In [184]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [185]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3 = df_ytd_cerrado3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,220776196,0.41
310,P-1714,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0,0.28
201,P-1641,1. INGRESO,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0,0.00
249,P-1668,4. COSTO SUPERVISIÓN,0,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,0,0.00
199,P-1639,3. COSTO MANO DE OBRA,0,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0,0.00


In [186]:
df_ytd_cerrado3.to_csv('archivos_respaldo/df_ytd_cerrado3.csv', index = False)

### 4.2-1 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO (PERSPECTIVA MES ANTERIOR)
MERGE ENTRE df_libro_mayor_ytd_closed_2M y df_cod_proyecto

In [187]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_2M.shape

Dimensiones antes del merge:


(340, 3)

In [188]:
df_ytd_cerrado_2M = df_libro_mayor_ytd_closed_2M.copy()

In [189]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M = df_ytd_cerrado_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 3)
Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,COSTO MANO DE OBRA,-4714680,NaN,NaN
211,P-1694,INGRESO,36274900,Eco Huemul,Constructora ICF SPA
324,P-1726,COSTO MANO DE OBRA,-2654140,NaN,NaN
333,P-1728,COSTO MATERIALES,-1822779,NaN,NaN
260,P-1708,COSTO MANO DE OBRA,-1471220,Eco España,Constructora ICF SPA
43,P-1636,INGRESO,0,Santo Domingo 1410,DANACORP S.A
259,P-1707,INGRESO,4178952,Las Tranqueras,Constructora Santolaya Ltda.
258,P-1707,COSTO SUPERVISIÓN,0,Las Tranqueras,Constructora Santolaya Ltda.
284,P-1716,COSTO MANO DE OBRA,-20322771,CD Astara,Astara Logística Chile SpA.
308,P-1722,COSTO MANO DE OBRA,0,Eco Cerrillos,Constructora ICF SPA


In [190]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado_2M['TIPO_CUENTA'] = df_ytd_cerrado_2M['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado_2M.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
198,P-1691,4. COSTO SUPERVISIÓN,0,Rosario Rosales,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.
336,P-1729,3. COSTO MANO DE OBRA,0,NaN,NaN
328,P-1727,3. COSTO MANO DE OBRA,-4714680,NaN,NaN
156,P-1681,3. COSTO MANO DE OBRA,0,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC
189,P-1689,2. COSTO MATERIALES,26522,Casino Buenaventura,EBCO S.A.
157,P-1681,2. COSTO MATERIALES,0,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC
269,P-1712,2. COSTO MATERIALES,-13499992,Centro Comercial Viña Outlet Park,EBCO S.A.
295,P-1718,1. INGRESO,88900450,Patio Comidas,EBCO S.A.
326,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN
134,P-1674,4. COSTO SUPERVISIÓN,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.


In [191]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-435497453,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,NaN,NaN
3,1. TOTAL,1. INGRESO,1118563896,NaN,NaN


In [192]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado_2M[cols_to_replace] = df_ytd_cerrado_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado_2M.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
61,P-1644,2. COSTO MATERIALES,204032,Mirador de San Cristobal,EBCO S.A.
69,P-1648,2. COSTO MATERIALES,200959,Etapa 2 CD Peralillo TW,Inmobiliaria y Rentas Logística S.A.
54,P-1642,4. COSTO SUPERVISIÓN,0,Corinto,EMP. CONSTRUCTORA INGENIEROS S.A.
259,P-1707,1. INGRESO,4178952,Las Tranqueras,Constructora Santolaya Ltda.


In [193]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435497453,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896,1. TOTAL,1. TOTAL


#### 4.2-1.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [194]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   : ", df_ytd_cerrado_2M.shape)

df_ytd_cerrado_2M_2 = df_ytd_cerrado_2M.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 : ", df_ytd_cerrado_2M_2.shape)

df_ytd_cerrado_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
315,P-1723,1. INGRESO,18134971
280,P-1715,3. COSTO MANO DE OBRA,-12758158
131,P-1673,1. INGRESO,0
285,P-1716,2. COSTO MATERIALES,-23165972
168,P-1684,3. COSTO MANO DE OBRA,-4658669


In [195]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [196]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
176,P-1592,4. COSTO SUPERVISIÓN,0,NUEVA COSTANERA 3300,EBCO S.A.
131,P-1714,2. COSTO MATERIALES,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
24,P-1663,2. COSTO MATERIALES,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN


In [197]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado_2M_3 = df_libro_mayor5_base2.merge(df_ytd_cerrado_2M_2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
152,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,18134971
26,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.,0
286,P-1689,4. COSTO SUPERVISIÓN,0,CASINO BUENAVENTURA,EBCO S.A.,0
99,P-1700,1. INGRESO,0,CD ORIZON,EBCO S.A.,16747965
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066


In [198]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado_2M_3[df_ytd_cerrado_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250


#### 4.2-1.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [199]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado_2M_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
19,P-1653,0.00
38,P-1680,0.21
71,P-1716,0.58
6,P-1612,0.00
9,P-1632,0.00
13,P-1642,0.00
68,P-1713,0.06
49,P-1691,0.32
31,P-1672,1.03
12,P-1641,0.00


In [200]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [201]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3 = df_ytd_cerrado_2M_3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
194,P-1632,4. COSTO SUPERVISIÓN,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0,0.00
80,P-1694,3. COSTO MANO DE OBRA,0,ECO HUEMUL,CONSTRUCTORA ICF SPA,-12624130,0.26
58,P-1685,2. COSTO MATERIALES,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,88900,0.00
334,P-1729,3. COSTO MANO DE OBRA,0,NaN,NaN,0,0.00
266,P-1677,4. COSTO SUPERVISIÓN,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0,-0.66


In [202]:
df_ytd_cerrado_2M_3.to_csv('archivos_respaldo/df_ytd_cerrado_2M_3.csv', index = False)

### 4.3 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO 
(DATAFRAMES: df_libro_mayor5_mec2 y df_proyectado4)

#### 4.3.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec2" Y AJUSTES INICIALES

In [203]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec2.sample(5)
df_libro_mayor5_mec3 = df_libro_mayor5_mec2.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
2364,P-1720,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
510,P-1723,COSTO MATERIALES,2023-06-01,2023.00,6.00,-28471458,LIBRO MAYOR
1716,P-1690,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1476,P-1680,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
462,P-1716,COSTO MATERIALES,2023-06-01,2023.00,6.00,-2308672,LIBRO MAYOR


In [204]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  : ", df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3[df_libro_mayor5_mec3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  :  (426, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
325,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883,LIBRO MAYOR
331,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,-24160596,LIBRO MAYOR
2027,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2028,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2041,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR
2042,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR


In [205]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec3['DESTINO'] = df_libro_mayor5_mec3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3.sample(10)


(426, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
108,P-1682,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-423674,LIBRO MAYOR,REAL Y PROYECTADO
1524,P-1682,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1322,P-1672,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1609,P-1685,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
2555,P-1728,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
697,P-1611,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
2234,P-1714,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
2172,P-1712,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
794,P-1632,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
1706,P-1689,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL


In [206]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec3['TIPO_CUENTA'] = df_libro_mayor5_mec3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3["TIPO_CUENTA"].unique()

(426, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES

In [207]:
# Ajuste del dataframe proveniente de PROYECTADO

df_proyectado4.sample(5)
df_proyectado5 = df_proyectado4.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5['FECHA'] = pd.to_datetime(df_proyectado5['MES'].astype(str) + '-1-' + df_proyectado5['AÑO'].astype(str))

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
67,P-1725,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01
61,P-1719,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01
22,P-1675,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES,2023-06-01
61,P-1719,0.00,6,2023,PROYECTADO,1. INGRESO,2023-06-01
66,P-1724,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01


In [208]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5['DESTINO'] = df_proyectado5.apply(determinar_origen, axis=1)
df_proyectado5 = df_proyectado5[df_proyectado5['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5 = df_proyectado5.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
44,P-1700,2. COSTO MATERIALES,2023-06-01,2023,6,0.00,PROYECTADO,PROYECTADO
64,P-1722,2. COSTO MATERIALES,2023-06-01,2023,6,0.00,PROYECTADO,PROYECTADO
62,P-1720,1. INGRESO,2023-06-01,2023,6,0.00,PROYECTADO,PROYECTADO
34,P-1690,1. INGRESO,2023-06-01,2023,6,5000000.00,PROYECTADO,PROYECTADO
29,P-1684,1. INGRESO,2023-06-01,2023,6,25380000.00,PROYECTADO,PROYECTADO


#### 4.3.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO

In [209]:
# Se realiza concatenación entre ambos dataframes

print("Dimensiones df_libro_mayor5_mec3 antes de cambios: ", df_libro_mayor5_mec3.shape)
print("Dimensiones df_proyectado5 antes de cambios      : ", df_proyectado5.shape)

mes_en_curso = pd.concat([df_libro_mayor5_mec3, df_proyectado5])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso.shape)
mes_en_curso.sample(10)

Dimensiones df_libro_mayor5_mec3 antes de cambios:  (426, 8)
Dimensiones df_proyectado5 antes de cambios      :  (146, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (572, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
1163,P-1664,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1571,P-1684,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
2425,P-1722,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
38,P-1694,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-8000000.00,PROYECTADO,PROYECTADO
1753,P-1691,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
1499,P-1681,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1907,P-1698,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1777,P-1692,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
2388,P-1721,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1129,P-1661,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL


#### 4.3.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [210]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso.shape)
mes_en_curso2 = mes_en_curso.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso2 = mes_en_curso2[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso2 antes de cambios: ", mes_en_curso2.shape)
mes_en_curso2.sample(10)

Dimensiones mes_en_curso antes de cambios:  (572, 8)
Dimensiones mes_en_curso2 antes de cambios:  (572, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
81,P-1729,NaN,NaN,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-114402.00,LIBRO MAYOR,REAL
248,P-1683,CD BP-A05,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
118,P-1632,Chorrillos Clorox,Constructora Suksa S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
48,P-1707,Las Tranqueras,Constructora Santolaya Ltda.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-3557418.00,LIBRO MAYOR,REAL
145,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
130,P-1641,Inacap Temuco,Desarrollos Constructivos Axis Sociedad Anonima,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
26,P-1690,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-474474.00,LIBRO MAYOR,REAL
129,P-1639,Lyon 2546,Moller y Perez - Cotapos Construcciones Indust...,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
478,P-1708,Eco España,Constructora ICF SPA,1. INGRESO,2023-06-01,2023.00,6.00,0.00,PROYECTADO,PROYECTADO
403,P-1725,Lo Cañas,Constructora Brisas de Batuco,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO


In [211]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso2.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
518,P-1671,Barrio Parque Stgo 8-9,CONSTRUCTORA ALTURAS DEL SUR S.A,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,PROYECTADO,PROYECTADO
174,P-1661,San Vicente de Paul,DANACORP S.A,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
310,P-1699,Villa Panamericana Lote B,Empresa Constructora DLP S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
348,P-1709,Centro Comercial Paseo San Pedro,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
241,P-1681,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL


In [212]:
# INSPECCIÓN VISUAL 1

mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
40,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883.00,LIBRO MAYOR,REAL Y PROYECTADO
41,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-24160596.00,LIBRO MAYOR,REAL
326,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
327,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
328,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
329,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
474,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,64764000.00,PROYECTADO,PROYECTADO
547,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-27000000.00,PROYECTADO,PROYECTADO


#### 4.3.5 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [213]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso2.shape)

# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso2.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

# concatenar nuevas filas al DataFrame original
mes_en_curso2 = pd.concat([mes_en_curso2, pd.DataFrame(df_rows_to_add)], ignore_index=True)

# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso2['CODIGO_PROYECTO'] = mes_en_curso2['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso2[cols_to_replace] = mes_en_curso2[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso2.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Cambiar los valores de la columna TIPO_CUENTA

print("Dimensiones Dataframe después de cambios", mes_en_curso2.shape)


Dimensiones Dataframe antes de cambios (572, 10)
Dimensiones Dataframe después de cambios (1361, 10)


In [214]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [215]:
# INSPECCIÓN VISUAL 1
mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1644"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
142,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
143,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
144,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
145,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,LIBRO MAYOR,REAL
670,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL Y PROYECTADO
671,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL Y PROYECTADO
672,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL Y PROYECTADO
673,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,AGREGACION,REAL
674,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0.00,AGREGACION,PROYECTADO
675,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0.00,AGREGACION,REAL


In [216]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.6 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [217]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_proy = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy = pd.DataFrame(mes_en_curso_proy.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  : ", mes_en_curso_proy.shape)

mes_en_curso_proy.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1361, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (874, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (874, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
63,P-1641,4. COSTO SUPERVISIÓN,0.00
254,P-1700,1. INGRESO,0.00
146,P-1672,3. COSTO MANO DE OBRA,0.00
36,P-1620,1. INGRESO,0.00
29,P-1611,4. COSTO SUPERVISIÓN,0.00
26,P-1611,1. INGRESO,0.00
54,P-1638,1. INGRESO,0.00
330,P-1721,3. COSTO MANO DE OBRA,-3509940.00
352,P-1727,1. INGRESO,29520000.00
40,P-1621,1. INGRESO,0.00


In [218]:
# TESTING VISUAL 1
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
292,P-1712,1. INGRESO,15228000.00
293,P-1712,2. COSTO MATERIALES,-7700000.00
294,P-1712,3. COSTO MANO DE OBRA,-5436047.00
295,P-1712,4. COSTO SUPERVISIÓN,0.00


In [219]:
# TESTING VISUAL 2
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,432312383.88
3,1. TOTAL,2. COSTO MATERIALES,-176535806.00
4,1. TOTAL,3. COSTO MANO DE OBRA,-114373715.00
5,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261.00


##### 4.3.6.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [220]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   : ", mes_en_curso_proy.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy2 = df_libro_mayor5_base2.merge(mes_en_curso_proy, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       : ", mes_en_curso_proy2.shape)

mes_en_curso_proy2['TOTAL'] = mes_en_curso_proy2['TOTAL'].fillna(0)

mes_en_curso_proy2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
137,P-1716,2. COSTO MATERIALES,0,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0.00
150,P-1723,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904.00
169,P-1557,2. COSTO MATERIALES,0,ROSARIO ROSALES,EBCO S.A.,0.00
22,P-1660,2. COSTO MATERIALES,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0.00
38,P-1675,2. COSTO MATERIALES,0,PALENA II,CONSTRUCTORA MYA SPA.,0.00


In [221]:
# TESTING VISUAL 1

mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047.00
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7700000.00
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,15228000.00
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0.00


In [222]:
# TESTING VISUAL 2
mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,432312383.88
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-176535806.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715.00


##### 4.3.6.2 - SE RECALCULAN LOS TOTALES

In [223]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy2 = mes_en_curso_proy2.drop(mes_en_curso_proy2[(mes_en_curso_proy2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 : ", mes_en_curso_proy2.shape)

df_total = mes_en_curso_proy2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy3 = pd.concat([mes_en_curso_proy2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy3.loc[mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
244,P-1666,4. COSTO SUPERVISIÓN,0.00,CD AXIONLONG,EBCO S.A.,0.00
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,0.00
322,P-1721,4. COSTO SUPERVISIÓN,0.00,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,0.00
143,P-1719,1. INGRESO,0.00,CD METALPAR,EBCO S.A.,0.00
95,P-1699,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-22000000.00
59,P-1686,3. COSTO MANO DE OBRA,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-3733792.00
115,P-1707,3. COSTO MANO DE OBRA,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-3715026.00
315,P-1717,4. COSTO SUPERVISIÓN,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00
176,P-1592,4. COSTO SUPERVISIÓN,0.00,NUEVA COSTANERA 3300,EBCO S.A.,0.00
260,P-1674,3. COSTO MANO DE OBRA,0.00,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0.00


In [224]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,420312383.88
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-174535806.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261.00


In [225]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3 = mes_en_curso_proy3.drop(mes_en_curso_proy3[(mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,420312383.88
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-174535806.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715.00


In [226]:
# TESTING VISUAL 1
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904.00
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000.00
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,85000000.00
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00


In [227]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,420312383.88
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-174535806.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715.00


##### 4.3.6.3 - CREACIÓN DE MARGEN

In [228]:
#CREACIÓN DE MARGENES

mes_en_curso_proy4 = mes_en_curso_proy3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
44,P-1686,0.23
79,P-1724,0.00
47,P-1689,0.00
0,1. TOTAL,0.30
8,P-1626,0.00
33,P-1674,0.00
56,P-1698,0.00
39,P-1681,0.00
38,P-1680,0.00
32,P-1673,0.00


In [229]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 : ", mes_en_curso_proy4.shape)

mes_en_curso_proy5 = pd.merge(mes_en_curso_proy4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 : ", mes_en_curso_proy5.shape)
mes_en_curso_proy5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
194,P-1632,4. COSTO SUPERVISIÓN,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,0.00
12,P-1642,2. COSTO MATERIALES,0.00,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00
45,P-1679,3. COSTO MANO DE OBRA,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,0.00,0.00
223,P-1653,4. COSTO SUPERVISIÓN,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,0.00
34,P-1672,2. COSTO MATERIALES,0.00,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,0.00,0.00
315,P-1717,4. COSTO SUPERVISIÓN,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00,0.79
276,P-1683,4. COSTO SUPERVISIÓN,0.00,CD BP-A05,EBCO S.A.,0.00,0.00
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-1666000.00,0.00
29,P-1668,2. COSTO MATERIALES,0.00,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00,0.00
49,P-1680,1. INGRESO,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,0.00,0.00


In [230]:
# TESTING VISUAL 1
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904.00,0.45
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000.00,0.45
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,85000000.00,0.45
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,0.45


In [231]:
# TESTING VISUAL 2
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261.00,0.30
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,420312383.88,0.30
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-174535806.00,0.30
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715.00,0.30


In [232]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy5.to_csv('archivos_respaldo/mes_en_curso_proy5.csv', mode='a')

#### 4.3.7 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO

##### 4.3.7.1 - TRATAMIENTO mes_en_curso2

In [233]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('REAL')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_real = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real = pd.DataFrame(mes_en_curso_real.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  : ", mes_en_curso_real.shape)

mes_en_curso_real.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1361, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (948, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (948, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
293,P-1712,4. COSTO SUPERVISIÓN,0
9,P-1591,2. COSTO MATERIALES,0
286,P-1709,1. INGRESO,0
87,P-1652,2. COSTO MATERIALES,0
36,P-1620,3. COSTO MANO DE OBRA,0
178,P-1681,3. COSTO MANO DE OBRA,0
45,P-1632,2. COSTO MATERIALES,840
298,P-1714,1. INGRESO,0
343,P-1725,2. COSTO MATERIALES,-4319747
265,P-1703,4. COSTO SUPERVISIÓN,0


In [234]:
# TESTING VISUAL 1

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
290,P-1712,1. INGRESO,-3823676
291,P-1712,2. COSTO MATERIALES,-7219006
292,P-1712,3. COSTO MANO DE OBRA,-5436047
293,P-1712,4. COSTO SUPERVISIÓN,0


In [235]:
# TESTING VISUAL 2

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,1. INGRESO,6762521
1,1. TOTAL,2. COSTO MATERIALES,-145613361
2,1. TOTAL,3. COSTO MANO DE OBRA,-114373715
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261


##### 4.3.7.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [236]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   : ", mes_en_curso_real.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real2 = df_libro_mayor5_base2.merge(mes_en_curso_real, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       : ", mes_en_curso_real.shape)

mes_en_curso_real2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   :  (362, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
211,P-1644,3. COSTO MANO DE OBRA,0,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,0
157,P-1725,2. COSTO MATERIALES,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4319747
265,P-1676,4. COSTO SUPERVISIÓN,0,LO CAMPINO,EBCO S.A.,0
200,P-1639,4. COSTO SUPERVISIÓN,0,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


In [237]:
# TESTING VISUAL 1

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7219006
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-3823676
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [238]:
# TESTING VISUAL 2

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,6762521
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-145613361
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.3 - SE RECALCULAN LOS TOTALES

In [239]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real2 = mes_en_curso_real2.drop(mes_en_curso_real2[(mes_en_curso_real2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 : ", mes_en_curso_real2.shape)

df_total = mes_en_curso_real2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real3 = pd.concat([mes_en_curso_real2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real3.loc[mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3.sample(10)   

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
214,P-1645,4. COSTO SUPERVISIÓN,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0
199,P-1639,3. COSTO MANO DE OBRA,0.00,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0
132,P-1714,1. INGRESO,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0
227,P-1658,1. INGRESO,0.00,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0
233,P-1661,1. INGRESO,0.00,SAN VICENTE DE PAUL,DANACORP S.A,0
204,P-1642,1. INGRESO,0.00,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0
260,P-1674,3. COSTO MANO DE OBRA,0.00,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0
285,P-1689,3. COSTO MANO DE OBRA,0.00,CASINO BUENAVENTURA,EBCO S.A.,0
226,P-1656,4. COSTO SUPERVISIÓN,0.00,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0


In [240]:
# TESTING VISUAL 1

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,6762521
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-145613361
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261


In [241]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3 = mes_en_curso_real3.drop(mes_en_curso_real3[(mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,6762521
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-145613361
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.4 - CREACIÓN DE MARGEN

In [242]:
#CREACIÓN DE MARGENES

mes_en_curso_real4 = mes_en_curso_real3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
15,P-1644,0.00
76,P-1721,0.00
7,P-1620,0.00
56,P-1698,0.00
5,P-1611,0.00
14,P-1643,0.00
44,P-1686,0.00
29,P-1669,0.00
57,P-1699,0.00
20,P-1656,0.00


In [243]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", mes_en_curso_real4.shape)

mes_en_curso_real5 = pd.merge(mes_en_curso_real4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 : ", mes_en_curso_real5.shape)
mes_en_curso_real5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
174,P-1592,1. INGRESO,0.00,NUEVA COSTANERA 3300,EBCO S.A.,0,0.00
287,P-1690,4. COSTO SUPERVISIÓN,0.00,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0,0.00
321,P-1720,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0,0.00
285,P-1689,3. COSTO MANO DE OBRA,0.00,CASINO BUENAVENTURA,EBCO S.A.,0,0.00
223,P-1653,4. COSTO SUPERVISIÓN,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0,0.00
129,P-1713,1. INGRESO,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0,0.00
309,P-1713,4. COSTO SUPERVISIÓN,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0,0.00
267,P-1679,1. INGRESO,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,0,0.00
306,P-1708,4. COSTO SUPERVISIÓN,0.00,ECO ESPAÑA,CONSTRUCTORA ICF SPA,0,0.00
87,P-1696,2. COSTO MATERIALES,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0,0.00


In [244]:
# TESTING VISUAL 1

mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-1666000,0.00
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2308672,0.00
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00


In [245]:
# TESTING VISUAL 3
mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-12890883,0.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-24160596,0.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00


In [246]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real5.to_csv('archivos_respaldo/mes_en_curso_real5.csv', mode='a')

### 4.4 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO (PERSPECTIVA MES ANTERIOR)
(DATAFRAMES: df_libro_mayor5_mec_2M y df_proyectado4) 


#### 4.4.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec_2M" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de LIBRO MAYOR

In [247]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
1495,P-1681,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1328,P-1673,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1135,P-1663,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
968,P-1648,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2266,P-1716,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [248]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
1041,P-1656,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1616,P-1686,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
727,P-1620,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1857,P-1696,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2049,P-1705,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [249]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
324,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535,LIBRO MAYOR
330,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551,LIBRO MAYOR
336,P-1704,INGRESO,2023-05-01,2023.00,5.00,79059997,LIBRO MAYOR
2023,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2024,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2025,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2026,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [250]:
# INSPECCIÓN VISUAL INICIAL

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  : ", df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec_2M_3['DESTINO'] = df_libro_mayor5_mec_2M_3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3.sample(10)


DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  :  (456, 7)
(456, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
897,P-1643,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
776,P-1632,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
461,P-1716,COSTO MATERIALES,2023-05-01,2023.00,5.00,-2986507,LIBRO MAYOR,REAL
316,P-1703,COSTO MATERIALES,2023-05-01,2023.00,5.00,-200940,LIBRO MAYOR,REAL
1472,P-1680,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
509,P-1723,COSTO MATERIALES,2023-05-01,2023.00,5.00,-34038712,LIBRO MAYOR,REAL
2098,P-1707,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
20,P-1643,COSTO MATERIALES,2023-05-01,2023.00,5.00,-330968,LIBRO MAYOR,REAL
2145,P-1709,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
2481,P-1725,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL


In [251]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec_2M_3['TIPO_CUENTA'] = df_libro_mayor5_mec_2M_3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3["TIPO_CUENTA"].unique()

(456, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.4.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de PROYECTADO

In [252]:
df_proyectado4_1M.sample(5)
df_proyectado5_1M = df_proyectado4_1M.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5_1M['FECHA'] = pd.to_datetime(df_proyectado5_1M['MES'].astype(str) + '-1-' + df_proyectado5_1M['AÑO'].astype(str))

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
55,P-1709,11127770.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01
8,P-1645,0.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01
71,P-1727,-9500000.00,5,2023,PROYECTADO,2. COSTO MATERIALES,2023-05-01
48,P-1702,0.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01
22,P-1675,0.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01


In [253]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5_1M['DESTINO'] = df_proyectado5_1M.apply(determinar_origen, axis=1)
df_proyectado5_1M = df_proyectado5_1M[df_proyectado5_1M['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5_1M = df_proyectado5_1M.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
44,P-1698,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
43,P-1697,2. COSTO MATERIALES,2023-05-01,2023,5,-2500000.00,PROYECTADO,PROYECTADO
5,P-1632,2. COSTO MATERIALES,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
4,P-1652,2. COSTO MATERIALES,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
34,P-1688,1. INGRESO,2023-05-01,2023,5,4529751.00,PROYECTADO,PROYECTADO


#### 4.4.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO
Se realiza concatenación entre ambos dataframes

In [254]:
print("Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios: ", df_libro_mayor5_mec_2M_3.shape)
print("Dimensiones df_proyectado5_1M antes de cambios       : ", df_proyectado5_1M.shape)

mes_en_curso_2M = pd.concat([df_libro_mayor5_mec_2M_3, df_proyectado5_1M])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)


Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios:  (456, 8)
Dimensiones df_proyectado5_1M antes de cambios       :  (148, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (604, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
54,P-1708,1. INGRESO,2023-05-01,2023.00,5.00,1568763.00,PROYECTADO,PROYECTADO
943,P-1645,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
920,P-1644,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
25,P-1678,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
518,P-1724,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-26558.00,LIBRO MAYOR,REAL
10,P-1655,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
69,P-1725,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-6000000.00,PROYECTADO,PROYECTADO
324,P-1704,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535.00,LIBRO MAYOR,REAL Y PROYECTADO
50,P-1704,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-24000000.00,PROYECTADO,PROYECTADO
407,P-1712,1. INGRESO,2023-05-01,2023.00,5.00,15000000.00,LIBRO MAYOR,REAL


#### 4.4.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [255]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M = mes_en_curso_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso_2M = mes_en_curso_2M[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso_2M antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)

Dimensiones mes_en_curso antes de cambios:  (604, 8)
Dimensiones mes_en_curso_2M antes de cambios:  (604, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
457,P-1601,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
478,P-1675,Palena II,CONSTRUCTORA MYA SpA.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
205,P-1661,San Vicente de Paul,DANACORP S.A,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
247,P-1674,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
171,P-1643,Chorrillos M4,Constructora Suksa S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
314,P-1692,José Perez Cotapos,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
249,P-1675,Palena II,CONSTRUCTORA MYA SpA.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
0,P-1557,Rosario Rosales,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-945949.00,LIBRO MAYOR,REAL Y PROYECTADO
594,P-1720,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
517,P-1717,Eco Quilín I,Constructora ICF SPA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO


In [256]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso_2M.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
478,P-1675,Palena II,CONSTRUCTORA MYA SpA.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
531,P-1601,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
343,P-1699,Villa Panamericana Lote B,Empresa Constructora DLP S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
177,P-1645,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
237,P-1672,Sta Clara (ccdd-telecom),Constructora UPC S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO


In [257]:
# INSPECCIÓN VISUAL 1

mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
58,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535.00,LIBRO MAYOR,REAL Y PROYECTADO
59,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551.00,LIBRO MAYOR,REAL
60,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,79059997.00,LIBRO MAYOR,REAL
356,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
357,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
358,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
359,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
506,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,68606659.28,PROYECTADO,PROYECTADO
580,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-24000000.00,PROYECTADO,PROYECTADO


#### 4.3.4 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [258]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso_2M.shape)

Dimensiones Dataframe antes de cambios (604, 10)


In [259]:
# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso_2M.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

In [260]:
# concatenar nuevas filas al DataFrame original
mes_en_curso_2M = pd.concat([mes_en_curso_2M, pd.DataFrame(df_rows_to_add)], ignore_index=True)


In [261]:
# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso_2M['CODIGO_PROYECTO'] = mes_en_curso_2M['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso_2M[cols_to_replace] = mes_en_curso_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("Dimensiones Dataframe después de cambios", mes_en_curso_2M.shape)

mes_en_curso_2M.sample(10)

Dimensiones Dataframe después de cambios (1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
634,P-1603,Farellones,Empresa Constructora DLP S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL
55,P-1701,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,175639.00,LIBRO MAYOR,REAL
1023,P-1703,Santa Victoria Danacorp,DANACORP S.A,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO
471,P-1667,Eco Placilla,Constructora ICF SPA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
2,P-1592,Nueva Costanera 3300,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,51537.00,LIBRO MAYOR,REAL
79,P-1713,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-1088874.00,LIBRO MAYOR,REAL
675,P-1636,Santo Domingo 1410,DANACORP S.A,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO
1295,P-1722,Eco Cerrillos,Constructora ICF SPA,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,REAL
1278,P-1698,CD Andes,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO
1214,P-1652,Abdon Cifuentes,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO


In [262]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1726"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
105,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-2654140.00,LIBRO MAYOR,REAL Y PROYECTADO
106,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-9165547.00,LIBRO MAYOR,REAL
107,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,LIBRO MAYOR,REAL
436,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
437,P-1726,NaN,NaN,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
438,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
439,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
526,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,PROYECTADO,PROYECTADO
600,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-10000000.00,PROYECTADO,PROYECTADO
1170,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO


In [263]:
mes_en_curso_2M["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [264]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="1. TOTAL"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-111885468.00,LIBRO MAYOR,REAL Y PROYECTADO
113,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-169535388.00,LIBRO MAYOR,REAL
114,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,460798513.00,LIBRO MAYOR,REAL
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,-5096121.00,LIBRO MAYOR,REAL Y PROYECTADO
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
454,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
455,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,455821573.22,PROYECTADO,PROYECTADO
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-212300000.00,PROYECTADO,PROYECTADO


#### 4.4.7 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [265]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   : ", mes_en_curso_2M_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_proy_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy_2M = pd.DataFrame(mes_en_curso_proy_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  : ", mes_en_curso_proy_2M.shape)

mes_en_curso_proy_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   :  (878, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (878, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
259,P-1701,2. COSTO MATERIALES,0.00
284,P-1708,1. INGRESO,1568763.00
143,P-1671,2. COSTO MATERIALES,0.00
203,P-1687,2. COSTO MATERIALES,0.00
59,P-1639,4. COSTO SUPERVISIÓN,0.00
358,P-1728,3. COSTO MANO DE OBRA,-541472.00
334,P-1722,3. COSTO MANO DE OBRA,0.00
360,P-1729,1. INGRESO,0.00
21,P-1601,2. COSTO MATERIALES,0.00
184,P-1682,3. COSTO MANO DE OBRA,-1086051.00


In [266]:
# TESTING VISUAL 1
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
348,P-1726,1. INGRESO,25429159.00
349,P-1726,2. COSTO MATERIALES,-10000000.00
350,P-1726,3. COSTO MANO DE OBRA,-2654140.00
351,P-1726,4. COSTO SUPERVISIÓN,0.00


In [267]:
# TESTING VISUAL 2
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,455821573.22
3,1. TOTAL,2. COSTO MATERIALES,-212300000.00
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00


##### 4.4.7.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [268]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     : ", mes_en_curso_proy_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy_2M_2 = df_libro_mayor5_base2.merge(mes_en_curso_proy_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  : ", mes_en_curso_proy_2M_2.shape)

mes_en_curso_proy_2M_2['TOTAL'] = mes_en_curso_proy_2M_2['TOTAL'].fillna(0)

mes_en_curso_proy_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
18,P-1649,2. COSTO MATERIALES,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0.00
313,P-1717,1. INGRESO,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00
34,P-1672,2. COSTO MATERIALES,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,0.00
326,P-1723,4. COSTO SUPERVISIÓN,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00
284,P-1689,1. INGRESO,0,CASINO BUENAVENTURA,EBCO S.A.,0.00


In [269]:
# TESTING VISUAL 1

mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00


In [270]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,455821573.22
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.2 - SE RECALCULAN LOS TOTALES

In [271]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy_2M_2 = mes_en_curso_proy_2M_2.drop(mes_en_curso_proy_2M_2[(mes_en_curso_proy_2M_2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 : ", mes_en_curso_proy_2M_2.shape)

df_total = mes_en_curso_proy_2M_2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy_2M_3 = pd.concat([mes_en_curso_proy_2M_2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy_2M_3.loc[mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
218,P-1649,1. INGRESO,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0.00
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-945949.00
23,P-1661,2. COSTO MATERIALES,0.00,SAN VICENTE DE PAUL,DANACORP S.A,0.00
40,P-1676,3. COSTO MANO DE OBRA,0.00,LO CAMPINO,EBCO S.A.,0.00
74,P-1691,1. INGRESO,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0.00
114,P-1706,1. INGRESO,0.00,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,17637537.00
186,P-1620,1. INGRESO,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00
124,P-1712,3. COSTO MANO DE OBRA,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-2791400.00
126,P-1712,1. INGRESO,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,14901765.87
91,P-1697,1. INGRESO,0.00,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,16105752.69


In [272]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,455821573.22
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121.00


In [273]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3 = mes_en_curso_proy_2M_3.drop(mes_en_curso_proy_2M_3[(mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,455821573.22
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


In [274]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00


In [275]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,455821573.22
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.3 - CREACIÓN DE MARGEN

In [276]:
#CREACIÓN DE MARGENES

mes_en_curso_proy_2M_4 = mes_en_curso_proy_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
44,P-1686,0.19
54,P-1696,0.00
1,P-1557,0.00
67,P-1712,0.24
71,P-1716,0.91
84,P-1729,0.00
52,P-1694,0.31
38,P-1680,-1.97
36,P-1677,0.00
73,P-1718,-0.10


In [277]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 : ", mes_en_curso_proy_2M_4.shape)

mes_en_curso_proy_2M_5 = pd.merge(mes_en_curso_proy_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 : ", mes_en_curso_proy_2M_5.shape)
mes_en_curso_proy_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
100,P-1701,3. COSTO MANO DE OBRA,0.00,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,0.00,0.00
57,P-1684,1. INGRESO,0.00,EDIFICIO GOYCOLEA,EBCO S.A.,3335671.64,0.55
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00,0.50
321,P-1720,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00
258,P-1673,4. COSTO SUPERVISIÓN,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,0.00
132,P-1714,1. INGRESO,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,2436155.00,0.70
257,P-1673,3. COSTO MANO DE OBRA,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,0.00
260,P-1674,3. COSTO MANO DE OBRA,0.00,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0.00,0.00
224,P-1656,1. INGRESO,0.00,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0.00,0.00
135,P-1715,1. INGRESO,0.00,CD LO ECHEVERS,QUEZADA Y BOETSCH,18230799.58,0.47


In [278]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-7889395.00,0.11
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-53000000.00,0.11
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,68515939.00,0.11
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,0.11


In [279]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00,0.28
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,455821573.22,0.28
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00,0.28
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00,0.28


In [280]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy_2M_5.to_csv('archivos_respaldo/mes_en_curso_proy_2M_5.csv', mode='a')

In [281]:
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00,0.50
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00,0.50
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00,0.50
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.50


#### 4.4.8 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO (PERSPECTIVA MES ANTERIOR)

##### 4.4.8.1 - TRATAMIENTO mes_en_curso2

In [282]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('REAL')]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_real_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real_2M = pd.DataFrame(mes_en_curso_real_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (979, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
10,P-1591,1. INGRESO,0
130,P-1668,1. INGRESO,0
90,P-1653,1. INGRESO,0
120,P-1664,3. COSTO MANO DE OBRA,0
216,P-1690,3. COSTO MANO DE OBRA,0
173,P-1679,4. COSTO SUPERVISIÓN,0
289,P-1709,2. COSTO MATERIALES,-3154246
180,P-1681,3. COSTO MANO DE OBRA,0
195,P-1685,2. COSTO MATERIALES,0
154,P-1674,3. COSTO MANO DE OBRA,0


In [283]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
292,P-1712,1. INGRESO,15000000
293,P-1712,2. COSTO MATERIALES,-5124213
294,P-1712,3. COSTO MANO DE OBRA,-2791400
295,P-1712,4. COSTO SUPERVISIÓN,0


In [284]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,460798513
3,1. TOTAL,2. COSTO MATERIALES,-169535388
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121


##### 4.4.8.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [285]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    : ", mes_en_curso_real_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real_2M = df_libro_mayor5_base2.merge(mes_en_curso_real_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
81,P-1694,2. COSTO MATERIALES,0,ECO HUEMUL,CONSTRUCTORA ICF SPA,-7641345
24,P-1663,2. COSTO MATERIALES,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0
222,P-1653,3. COSTO MANO DE OBRA,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0
127,P-1713,3. COSTO MANO DE OBRA,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-172081
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,-3205300


In [286]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-2791400
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5124213
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,15000000
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [287]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.3 - SE RECALCULAN LOS TOTALES

In [288]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real_2M = mes_en_curso_real_2M.drop(mes_en_curso_real_2M[(mes_en_curso_real_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M : ", mes_en_curso_real_2M.shape)

df_total = mes_en_curso_real_2M.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real_2M_3 = pd.concat([mes_en_curso_real_2M, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real_2M_3.loc[mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3.sample(10)  

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
146,P-1721,3. COSTO MANO DE OBRA,0.00,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,-2706033
314,P-1717,3. COSTO MANO DE OBRA,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0
11,P-1641,2. COSTO MATERIALES,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0
38,P-1675,2. COSTO MATERIALES,0.00,PALENA II,CONSTRUCTORA MYA SPA.,0
60,P-1686,2. COSTO MATERIALES,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-2268468
93,P-1698,1. INGRESO,0.00,CD ANDES,EBCO S.A.,0
202,P-1641,3. COSTO MANO DE OBRA,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0
53,P-1682,1. INGRESO,0.00,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,18808581
196,P-1636,3. COSTO MANO DE OBRA,0.00,SANTO DOMINGO 1410,DANACORP S.A,0
211,P-1644,3. COSTO MANO DE OBRA,0.00,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,0


In [289]:
# TESTING VISUAL 1

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121


In [290]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3 = mes_en_curso_real_2M_3.drop(mes_en_curso_real_2M_3[(mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169535388
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.4 - CREACIÓN DE MARGEN

In [291]:
#CREACIÓN DE MARGENES

mes_en_curso_real_2M_4 = mes_en_curso_real_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
39,P-1681,0.00
29,P-1669,0.00
23,P-1661,0.00
0,1. TOTAL,0.38
66,P-1709,0.51
80,P-1725,0.34
27,P-1667,0.00
41,P-1683,0.00
65,P-1708,-1.35
70,P-1715,0.47


In [292]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 : ", mes_en_curso_real_2M_4.shape)

mes_en_curso_real_2M_5 = pd.merge(mes_en_curso_real_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 : ", mes_en_curso_real_2M_5.shape)
mes_en_curso_real_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
81,P-1694,2. COSTO MATERIALES,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,-7641345,0.32
69,P-1690,3. COSTO MANO DE OBRA,0.00,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0,1.07
215,P-1648,1. INGRESO,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0,0.00
131,P-1714,2. COSTO MATERIALES,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,-234124,0.60
173,P-1591,4. COSTO SUPERVISIÓN,0.00,EDIFICIO ESTORIL,EBCO S.A.,0,0.00
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-9165547,0.54
170,P-1557,4. COSTO SUPERVISIÓN,0.00,ROSARIO ROSALES,EBCO S.A.,0,0.00
313,P-1717,1. INGRESO,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,0.00
95,P-1699,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-8687386,0.21
40,P-1676,3. COSTO MANO DE OBRA,0.00,LO CAMPINO,EBCO S.A.,0,0.00


In [293]:
# TESTING VISUAL 1
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-895428,0.61
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2986507,0.61
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,10082518,0.61
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.61


In [294]:
# TESTING VISUAL 2
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-14242535,0.64
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-13967551,0.64
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,79059997,0.64
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.64


In [295]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real_2M_5.to_csv('archivos_respaldo/mes_en_curso_real_2M_5.csv', mode='a')

### 4.5 - **MERGE PARA -> YTD PROYECTADO
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.5.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [296]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
235,P-1700,1. INGRESO,16747965,CD Orizon,EBCO S.A.
160,P-1682,3. COSTO MANO DE OBRA,-10329193,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.
293,P-1718,2. COSTO MATERIALES,-13961366,Patio Comidas,EBCO S.A.
318,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN
285,P-1716,2. COSTO MATERIALES,-26152479,CD Astara,Astara Logística Chile SpA.
75,P-1649,1. INGRESO,0,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
50,P-1641,4. COSTO SUPERVISIÓN,0,Inacap Temuco,Desarrollos Constructivos Axis Sociedad Anonima
237,P-1701,2. COSTO MATERIALES,-1111637,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.
175,P-1685,1. INGRESO,0,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
331,P-1727,1. INGRESO,28570952,NaN,NaN


In [297]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso2.shape)
mes_en_curso2.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso2_1 = mes_en_curso2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso2_1.sample(5)

(1361, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
849,P-1682,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.,1. INGRESO,0.00
1047,P-1713,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,2. COSTO MATERIALES,0.00
1194,P-1661,San Vicente de Paul,DANACORP S.A,1. INGRESO,0.00
1202,P-1669,Departamental,EMP. CONSTRUCTORA INGENIEROS S.A.,1. INGRESO,0.00
718,P-1658,VIVIPRA,Vicherat y Pradenas Ltda.,2. COSTO MATERIALES,0.00


#### 4.5.2 - Concatenación de Dataframes

In [298]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado   ", df_ytd_cerrado.shape)
print("Dataframes antes de concatenar   : mes_en_curso2_1  ", mes_en_curso2_1.shape)

df_ytd_proyectado = pd.concat([df_ytd_cerrado, mes_en_curso2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado  ", df_ytd_proyectado.shape)

df_ytd_proyectado.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado    (340, 5)
Dataframes antes de concatenar   : mes_en_curso2_1   (874, 5)
Dataframes después de concatenar : df_ytd_proyectado   (1214, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
1260,P-1722,1. INGRESO,0.00,Eco Cerrillos,Constructora ICF SPA
586,P-1592,3. COSTO MANO DE OBRA,0.00,Nueva Costanera 3300,EBCO S.A.
971,P-1700,3. COSTO MANO DE OBRA,0.00,CD Orizon,EBCO S.A.
940,P-1695,1. INGRESO,0.00,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
1262,P-1724,1. INGRESO,0.00,NaN,NaN


In [299]:
df_ytd_proyectado["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [300]:
# TESTING VISUAL 1 

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-8169893.00,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-13655419.00,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
331,P-1727,1. INGRESO,28570952.00,NaN,NaN
79,P-1727,3. COSTO MANO DE OBRA,-3455213.00,NaN,NaN
410,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN
411,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
495,P-1727,1. INGRESO,29520000.00,NaN,NaN
568,P-1727,2. COSTO MATERIALES,-10000000.00,NaN,NaN
1139,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN


In [301]:
# TESTING VISUAL 2

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965.00,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605032841.00,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666.00,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409.00,1. TOTAL,1. TOTAL
82,1. TOTAL,3. COSTO MANO DE OBRA,-114373715.00,1. TOTAL,1. TOTAL
85,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261.00,1. TOTAL,1. TOTAL
422,1. TOTAL,3. COSTO MANO DE OBRA,0.00,1. TOTAL,1. TOTAL
423,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL
498,1. TOTAL,1. INGRESO,432312383.88,1. TOTAL,1. TOTAL
571,1. TOTAL,2. COSTO MATERIALES,-176535806.00,1. TOTAL,1. TOTAL


#### 4.5.3 - AGREGACIÓN COLUMNA MARGEN

In [302]:
#CREACIÓN DE MARGENES

df_ytd_proyectado2 = df_ytd_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", df_ytd_proyectado2.shape)

df_ytd_proyectado2 = pd.merge(df_ytd_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 : ", df_ytd_proyectado2.shape)
df_ytd_proyectado2.sample(10)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (1214, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 :  (1214, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
667,P-1704,2. COSTO MATERIALES,-27000000.00,CD Mercado Libre Mezzanina,EBCO S.A.,0.41
1026,P-1712,2. COSTO MATERIALES,0.00,Centro Comercial Viña Outlet Park,EBCO S.A.,0.02
589,P-1699,1. INGRESO,40000000.00,Villa Panamericana Lote B,Empresa Constructora DLP S.A.,0.42
982,P-1701,3. COSTO MANO DE OBRA,0.00,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,-0.44
530,P-1723,3. COSTO MANO DE OBRA,0.00,Villa Panamericana Lote C,Constructora Mena y Ovalle S.A.,0.32
1142,P-1678,1. INGRESO,0.00,Habilitación Oficinas Pirer SPA,SOC. CONSTRUCTORA MANSILLA Y SALGADO LIMITADA,0.00
862,P-1676,3. COSTO MANO DE OBRA,0.00,Lo Campino,EBCO S.A.,0.00
866,P-1676,2. COSTO MATERIALES,0.00,Lo Campino,EBCO S.A.,0.00
1191,P-1677,2. COSTO MATERIALES,0.00,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...,-0.42
702,P-1592,3. COSTO MANO DE OBRA,0.00,Nueva Costanera 3300,EBCO S.A.,0.00


In [303]:
# TESTING VISUAL 1 

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-8169893.00,NaN,NaN,0.39
329,P-1727,2. COSTO MATERIALES,-13655419.00,NaN,NaN,0.39
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.39
331,P-1727,1. INGRESO,28570952.00,NaN,NaN,0.39
373,P-1727,3. COSTO MANO DE OBRA,-3455213.00,NaN,NaN,0.39
538,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.39
539,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.39
615,P-1727,1. INGRESO,29520000.00,NaN,NaN,0.39
688,P-1727,2. COSTO MATERIALES,-10000000.00,NaN,NaN,0.39
1097,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.39


In [304]:
# TESTING VISUAL 2

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965.00,1. TOTAL,1. TOTAL,0.30
1,1. TOTAL,2. COSTO MATERIALES,-605032841.00,1. TOTAL,1. TOTAL,0.30
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666.00,1. TOTAL,1. TOTAL,0.30
3,1. TOTAL,1. INGRESO,1579362409.00,1. TOTAL,1. TOTAL,0.30
374,1. TOTAL,3. COSTO MANO DE OBRA,-114373715.00,1. TOTAL,1. TOTAL,0.30
375,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261.00,1. TOTAL,1. TOTAL,0.30
544,1. TOTAL,3. COSTO MANO DE OBRA,0.00,1. TOTAL,1. TOTAL,0.30
545,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,0.30
618,1. TOTAL,1. INGRESO,432312383.88,1. TOTAL,1. TOTAL,0.30
691,1. TOTAL,2. COSTO MATERIALES,-176535806.00,1. TOTAL,1. TOTAL,0.30


In [305]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado2.to_csv('archivos_respaldo/df_ytd_proyectado2.csv', mode='a')

### 4.6 - **MERGE PARA -> YTD PROYECTADO (PERSPECTIVA MES ANTERIOR)
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.6.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [306]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
101,P-1664,2. COSTO MATERIALES,207670,MK II PCI,Comercial K SpA.
11,P-1591,1. INGRESO,0,Edificio Estoril,EBCO S.A.
83,P-1656,1. INGRESO,0,Mirador Club Hípico,Constructora Vital Edificacion SPA
148,P-1679,3. COSTO MANO DE OBRA,-187507,Juan Sabaj,Constructora Grevia
275,P-1713,1. INGRESO,4823540,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...
231,P-1699,1. INGRESO,73159443,Villa Panamericana Lote B,Empresa Constructora DLP S.A.
126,P-1672,4. COSTO SUPERVISIÓN,0,Sta Clara (ccdd-telecom),Constructora UPC S.A.
215,P-1695,1. INGRESO,15860326,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
74,P-1649,4. COSTO SUPERVISIÓN,0,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
186,P-1688,4. COSTO SUPERVISIÓN,0,Suiza,Moller y Perez - Cotapos Construcciones Indust...


In [307]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso_2M.shape)
mes_en_curso_2M.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)#
mes_en_curso_proy_2M_2_1 = mes_en_curso_2M[mes_en_curso_2M['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso_proy_2M_2_1 = mes_en_curso_proy_2M_2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso_proy_2M_2_1.sample(5)

(1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
639,P-1611,Edificio Plaza Egaña,EBCO S.A.,4. COSTO SUPERVISIÓN,0.00
801,P-1668,Quillayes Plaza,E.MOLINA MOREL CONSTRUCTORA S.A.,3. COSTO MANO DE OBRA,0.00
1386,P-1719,CD METALPAR,EBCO S.A.,2. COSTO MATERIALES,0.00
765,P-1661,San Vicente de Paul,DANACORP S.A,4. COSTO SUPERVISIÓN,0.00
1065,P-1709,Centro Comercial Paseo San Pedro,EBCO S.A.,3. COSTO MANO DE OBRA,0.00


In [308]:
# TESTING VISUAL 1 

mes_en_curso_proy_2M_2_1[mes_en_curso_proy_2M_2_1["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,455821573.22
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,-212300000.00
1198,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1199,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
1200,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1202,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,0.00


#### 4.6.2 - Concatenación de Dataframes

In [309]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado_2M       :", df_ytd_cerrado_2M.shape)
print("Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1:", mes_en_curso_proy_2M_2_1.shape)

df_ytd_proyectado_2M = pd.concat([df_ytd_cerrado_2M, mes_en_curso_proy_2M_2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado_2M  ", df_ytd_proyectado_2M.shape)

df_ytd_proyectado_2M.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado_2M       : (340, 5)
Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1: (878, 5)
Dataframes después de concatenar : df_ytd_proyectado_2M   (1218, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
964,P-1693,1. INGRESO,0.00,Volta V0800,ECOBIO SPA
1236,P-1667,1. INGRESO,0.00,Eco Placilla,Constructora ICF SPA
768,P-1661,1. INGRESO,0.00,San Vicente de Paul,DANACORP S.A
47,P-1697,3. COSTO MANO DE OBRA,-4075065.00,Estocolmo,Constructora Fortaleza S.A
761,P-1660,1. INGRESO,0.00,Coronel Alvarado,Larrain Prieto Risopatron S.A.


In [310]:
df_ytd_proyectado_2M["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [311]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
331,P-1727,1. INGRESO,11005000.00,NaN,NaN
108,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN
440,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN
441,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
527,P-1727,1. INGRESO,17255460.00,NaN,NaN
601,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN
1177,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN


In [312]:
# TESTING VISUAL 2

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250.00,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435497453.00,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405.00,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896.00,1. TOTAL,1. TOTAL
112,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00,1. TOTAL,1. TOTAL
115,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00,1. TOTAL,1. TOTAL
452,1. TOTAL,3. COSTO MANO DE OBRA,0.00,1. TOTAL,1. TOTAL
453,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL
529,1. TOTAL,1. INGRESO,455821573.22,1. TOTAL,1. TOTAL
603,1. TOTAL,2. COSTO MATERIALES,-212300000.00,1. TOTAL,1. TOTAL


#### 4.6.3 - AGREGACIÓN COLUMNA MARGEN

In [313]:
#CREACIÓN DE MARGENES

df_ytd_proyectado_2M_2 = df_ytd_proyectado_2M.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado_2M_2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)

df_ytd_proyectado_2M_2 = pd.merge(df_ytd_proyectado_2M_2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)
df_ytd_proyectado_2M_2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
141,P-1676,2. COSTO MATERIALES,-2031638.00,Lo Campino,EBCO S.A.,0.00
527,P-1721,3. COSTO MANO DE OBRA,0.00,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.,0.35
1033,P-1713,3. COSTO MANO DE OBRA,0.00,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0.02
477,P-1693,3. COSTO MANO DE OBRA,0.00,Volta V0800,ECOBIO SPA,0.00
708,P-1592,3. COSTO MANO DE OBRA,0.00,Nueva Costanera 3300,EBCO S.A.,0.00
825,P-1666,3. COSTO MANO DE OBRA,0.00,CD Axionlong,EBCO S.A.,0.00
444,P-1675,4. COSTO SUPERVISIÓN,0.00,Palena II,CONSTRUCTORA MYA SpA.,1.00
782,P-1648,2. COSTO MATERIALES,0.00,Etapa 2 CD Peralillo TW,Inmobiliaria y Rentas Logística S.A.,0.00
1080,P-1722,3. COSTO MANO DE OBRA,0.00,Eco Cerrillos,Constructora ICF SPA,0.00
361,P-1708,3. COSTO MANO DE OBRA,-921206.00,Eco España,Constructora ICF SPA,-0.60


In [314]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M_2[df_ytd_proyectado_2M_2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN,0.25
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN,0.25
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
331,P-1727,1. INGRESO,11005000.00,NaN,NaN,0.25
373,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN,0.25
539,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25
540,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
618,P-1727,1. INGRESO,17255460.00,NaN,NaN,0.25
692,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN,0.25
1103,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25


In [315]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado_2M_2.to_csv('archivos_respaldo/df_ytd_proyectado_2M_2.csv', mode='a')

### 4.7 - **MERGE PARA -> PTD CERRADO

Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.7.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [316]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.2 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado_1 = df_ytd_cerrado2.copy()
df_ytd_cerrado_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
81,P-1656,2. COSTO MATERIALES,11250
206,P-1693,4. COSTO SUPERVISIÓN,0
82,P-1656,4. COSTO SUPERVISIÓN,0
320,P-1725,3. COSTO MANO DE OBRA,-5242316
141,P-1676,2. COSTO MATERIALES,-1518340


In [317]:
# TESTING VISUAL 1

print(df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]

473614937


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965
1,1. TOTAL,2. COSTO MATERIALES,-605032841
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666
3,1. TOTAL,1. INGRESO,1579362409


In [318]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 : ", df_ytd_cerrado_1.shape)
df_ytd_cerrado_2 = pd.pivot_table(df_ytd_cerrado_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 : ", df_ytd_cerrado_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2 = df_ytd_cerrado_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2 = pd.DataFrame(df_ytd_cerrado_2.to_records())
df_ytd_cerrado_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
51,P-1641,4. COSTO SUPERVISIÓN,0
213,P-1695,2. COSTO MATERIALES,-18940946
16,P-1603,1. INGRESO,0
11,P-1591,4. COSTO SUPERVISIÓN,0
227,P-1698,4. COSTO SUPERVISIÓN,0
186,P-1688,3. COSTO MANO DE OBRA,-10238516
274,P-1713,3. COSTO MANO DE OBRA,-4283233
244,P-1704,1. INGRESO,220776196
328,P-1727,1. INGRESO,28570952
198,P-1691,3. COSTO MANO DE OBRA,-252060


In [319]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3

# Se crea una copia del dataframe 
print("Dimensiones del dataframe df_libro_mayor_old3 : ", df_libro_mayor_old3.shape)
df_libro_mayor_old4 = df_libro_mayor_old3.copy()
print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old3 :  (1741, 9)
Dimensiones del dataframe df_libro_mayor_old4 :  (1741, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
7,,4. COSTO SUPERVISIÓN,2021-05-01 00:00:00,2021.00,5.00,0,2610776,-2610776,0.00
923,P-1673,3. COSTO MANO DE OBRA,2021-11-01 00:00:00,2021.00,11.00,0,0,0,0.00
385,P-1652,1. INGRESO,2021-01-01 00:00:00,2021.00,1.00,2408311,0,2408311,0.00
825,P-1670,3. COSTO MANO DE OBRA,2021-07-01 00:00:00,2021.00,7.00,0,0,0,0.00
1031,P-1677,2. COSTO MATERIALES,2022-02-01 00:00:00,2022.00,2.00,15600,557388,-541788,0.00
379,P-1649,3. COSTO MANO DE OBRA,2021-02-01 00:00:00,2021.00,2.00,0,0,0,0.00
71,P-1615,1. INGRESO,2021-03-01 00:00:00,2021.00,3.00,22771306,7511111,15260195,0.00
1262,P-1690,1. INGRESO,2022-03-01 00:00:00,2022.00,3.00,492456,0,492456,0.00
1117,P-1682,2. COSTO MATERIALES,2022-12-01 00:00:00,2022.00,12.00,0,4702083,-4702083,0.00
1737,1. TOTAL,4. COSTO SUPERVISIÓN,2022-09-01 00:00:00,2022.00,9.00,0,3907465,-3907465,0.00


In [320]:
# Se realiza limpieza del dataframe para obtener solo las columnas utiles.

df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['CODIGO_PROYECTO'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != 'OTRA']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['TIPO_CUENTA'])
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['CODIGO_PROYECTO'])

print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old4 :  (1680, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1570,P-1712,1. INGRESO,2022-09-01 00:00:00,2022.00,9.00,32322323,0,32322323,0.00
1288,P-1690,3. COSTO MANO DE OBRA,2022-09-01 00:00:00,2022.00,9.00,0,2293368,-2293368,0.00
413,P-1652,3. COSTO MANO DE OBRA,2021-01-01 00:00:00,2021.00,1.00,0,0,0,0.00
639,P-1665,2. COSTO MATERIALES,2021-03-01 00:00:00,2021.00,3.00,18630,1525750,-1507120,0.00
740,P-1668,2. COSTO MATERIALES,2021-03-01 00:00:00,2021.00,3.00,0,53262,-53262,0.00
1142,P-1683,3. COSTO MANO DE OBRA,2022-03-01 00:00:00,2022.00,3.00,0,287910,-287910,0.00
640,P-1665,2. COSTO MATERIALES,2021-04-01 00:00:00,2021.00,4.00,0,900,-900,0.00
147,P-1628,1. INGRESO,2021-01-01 00:00:00,2021.00,1.00,415712,0,415712,0.00
508,P-1658,2. COSTO MATERIALES,2021-09-01 00:00:00,2021.00,9.00,189529,73700,115829,0.00
854,P-1672,1. INGRESO,2021-04-01 00:00:00,2021.00,4.00,13474003,13474003,0,0.00


In [321]:
# TESTING VISUAL 1

print(df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1632,1. TOTAL,1. INGRESO,2021-01-01 00:00:00,2021.00,1.00,153082726,51479131,101603595,0.00
1633,1. TOTAL,1. INGRESO,2021-02-01 00:00:00,2021.00,2.00,202772310,71750206,131022104,0.00
1634,1. TOTAL,1. INGRESO,2021-03-01 00:00:00,2021.00,3.00,227194679,39876627,187318052,0.00
1635,1. TOTAL,1. INGRESO,2021-04-01 00:00:00,2021.00,4.00,165417555,36206601,129210954,0.00
1636,1. TOTAL,1. INGRESO,2021-05-01 00:00:00,2021.00,5.00,113310222,10911630,102398592,0.00


In [322]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old5 = pd.pivot_table(df_libro_mayor_old4, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

# Se renombra campo "TOTAL"
df_libro_mayor_old5 = df_libro_mayor_old5.rename(columns={"TOTAL": "TOTAL LB OLD"})

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)

df_libro_mayor_old5 = pd.DataFrame(df_libro_mayor_old5.to_records())
df_libro_mayor_old5.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 :  (1680, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 :  (253, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
207,P-1700,1. INGRESO,109290844
23,P-1623,3. COSTO MANO DE OBRA,0
100,P-1664,1. INGRESO,138414890
96,P-1661,2. COSTO MATERIALES,-14273969
248,P-1718,3. COSTO MANO DE OBRA,-2971700
79,P-1654,2. COSTO MATERIALES,-1142947
91,P-1659,2. COSTO MATERIALES,21197
181,P-1691,2. COSTO MATERIALES,-16324925
125,P-1672,2. COSTO MATERIALES,-45941535
176,P-1689,3. COSTO MANO DE OBRA,-2217581


In [323]:
# TESTING VISUAL 1

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
0,1. TOTAL,1. INGRESO,3185711545
1,1. TOTAL,2. COSTO MATERIALES,-1152612025
2,1. TOTAL,3. COSTO MANO DE OBRA,-611212228
3,1. TOTAL,4. COSTO SUPERVISIÓN,-51949755


In [324]:
# TESTING VISUAL 2

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"].head(5)

31342023


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
14,P-1615,1. INGRESO,56248814
15,P-1615,2. COSTO MATERIALES,-23003962
16,P-1615,3. COSTO MANO DE OBRA,-1902829


#### 4.7.2 - Concatenación de Dataframes

In [325]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)
df_ptd_cerrado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN
151,P-1723,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.
325,P-1722,4. COSTO SUPERVISIÓN,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA
42,P-1677,3. COSTO MANO DE OBRA,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...


In [326]:
# MERGE 1 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_ytd_cerrado_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
175,P-1592,3. COSTO MANO DE OBRA,0,NUEVA COSTANERA 3300,EBCO S.A.,0
43,P-1677,2. COSTO MATERIALES,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-1379557
202,P-1641,3. COSTO MANO DE OBRA,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-369720
309,P-1713,4. COSTO SUPERVISIÓN,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0
141,P-1718,2. COSTO MATERIALES,0,PATIO COMIDAS,EBCO S.A.,-13961366
231,P-1660,3. COSTO MANO DE OBRA,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0
297,P-1698,4. COSTO SUPERVISIÓN,0,CD ANDES,EBCO S.A.,0
1,P-1591,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EBCO S.A.,-4593960
220,P-1649,4. COSTO SUPERVISIÓN,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0


In [327]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
75,P-1692,3. COSTO MANO DE OBRA,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,-27116720,-18580010.00
241,P-1664,4. COSTO SUPERVISIÓN,0,MK II PCI,COMERCIAL K SPA.,0,NaN
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0,0.00
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-252060,-16971354.00
293,P-1695,4. COSTO SUPERVISIÓN,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,0,NaN
213,P-1645,3. COSTO MANO DE OBRA,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0,-3015429.00
60,P-1686,2. COSTO MATERIALES,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-12682908,-22422016.00
185,P-1612,4. COSTO SUPERVISIÓN,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0,NaN
32,P-1670,2. COSTO MATERIALES,0,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,6945,-56749120.00
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,28719770,14973674.00


In [328]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL YTD"] = df_ptd_cerrado["TOTAL YTD"].fillna(0)
df_ptd_cerrado["TOTAL LB OLD"] = df_ptd_cerrado["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL"] = df_ptd_cerrado["TOTAL YTD"] + df_ptd_cerrado["TOTAL LB OLD"]
df_ptd_cerrado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
271,P-1681,3. COSTO MANO DE OBRA,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0,-337550.00,-337550.00
123,P-1709,1. INGRESO,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,76827615,7158358.00,83985973.00
164,P-1727,1. INGRESO,0,NaN,NaN,28570952,0.00,28570952.00
73,P-1691,2. COSTO MATERIALES,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-1214688,-16324925.00,-17539613.00
197,P-1636,4. COSTO SUPERVISIÓN,0,SANTO DOMINGO 1410,DANACORP S.A,0,0.00,0.00


In [329]:
# TESTING VISUAL 1

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]

473614937
1369937537.0
1843552474.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409,3185711545.00,4765073954.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605032841,-1152612025.00,-1757644866.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965,-611212228.00,-1089360193.00


In [330]:
# TESTING VISUAL 2

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]

6745640
0.0
6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,-8169893,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-13655419,0.00,-13655419.00
164,P-1727,1. INGRESO,0,NaN,NaN,28570952,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,0.00,0.00


#### 4.7.3 - SE AGREGAN FILAS DE 1.TOTAL

In [331]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado2 = df_ptd_cerrado[~((df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (337, 8)


In [332]:
# TESTING VISUAL 2


print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]

-22566666
-51949755.0
-74516421.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00


In [333]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_total = df_ptd_cerrado2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado2 = pd.concat([df_ptd_cerrado2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado2.loc[df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_ptd_cerrado2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00
314,P-1717,3. COSTO MANO DE OBRA,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00,0.00,0.00
235,P-1661,4. COSTO SUPERVISIÓN,0.00,SAN VICENTE DE PAUL,DANACORP S.A,0.00,0.00,0.00
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
128,P-1713,2. COSTO MATERIALES,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-2358536.00,-595747.00,-2954283.00


In [334]:
# TESTING VISUAL 1

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1591529635.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-74516421.00


In [335]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado2 = df_ptd_cerrado2[~((df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1666046056.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00


In [336]:
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00


#### 4.7.4 - AGREGACIÓN COLUMNA MARGEN

In [337]:
#CREACIÓN DE MARGENES

df_ptd_cerrado3 = df_ptd_cerrado2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3 = pd.merge(df_ptd_cerrado3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)
df_ptd_cerrado3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
248,P-1668,3. COSTO MANO DE OBRA,0.00,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00,-7927098.00,-7927098.00,0.37
318,P-1719,3. COSTO MANO DE OBRA,0.00,CD METALPAR,EBCO S.A.,0.00,0.00,0.00,1.00
334,P-1729,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.00,0.00,0.00,0.00
215,P-1648,1. INGRESO,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0.00,672750.00,672750.00,2.17
154,P-1724,2. COSTO MATERIALES,0.00,NaN,NaN,-7660814.00,0.00,-7660814.00,0.72


In [338]:
# TESTING VISUAL 1

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1666046056.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37


In [339]:
# TESTING VISUAL 2

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24


In [340]:
df_ptd_cerrado3.to_csv('archivos_respaldo/df_ptd_cerrado3.csv', index = False)

### 4.8 - **MERGE PARA -> PTD CERRADO (PERSPECTIVA MES ANTERIOR)
Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.8.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [341]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.3 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO PERSPECTIVA MES ANTERIOR

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3_1 = df_ytd_cerrado_2M_3.copy()
df_ytd_cerrado_2M_3_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
2,P-1592,2. COSTO MATERIALES,0,NUEVA COSTANERA 3300,EBCO S.A.,-9821,0.00
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-252060,0.32
3,P-1603,2. COSTO MATERIALES,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000,0.00
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591,0.27
232,P-1660,4. COSTO SUPERVISIÓN,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0,0.00


In [342]:
# TESTING VISUAL 1

print(df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]

301368788


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,0.27
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,0.27
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453,0.27
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,0.27


In [343]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   : ", df_ytd_cerrado_2M_3_1.shape)
df_ytd_cerrado_2M_3_2 = pd.pivot_table(df_ytd_cerrado_2M_3_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2M_3_2 = df_ytd_cerrado_2M_3_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2M_3_2 = pd.DataFrame(df_ytd_cerrado_2M_3_2.to_records())
df_ytd_cerrado_2M_3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
278,P-1714,3. COSTO MANO DE OBRA,-8052049
71,P-1648,4. COSTO SUPERVISIÓN,0
78,P-1653,3. COSTO MANO DE OBRA,0
156,P-1681,1. INGRESO,765714
239,P-1701,4. COSTO SUPERVISIÓN,0
188,P-1689,1. INGRESO,0
289,P-1717,2. COSTO MATERIALES,0
45,P-1639,2. COSTO MATERIALES,5166
224,P-1698,1. INGRESO,909040
255,P-1706,4. COSTO SUPERVISIÓN,0


In [344]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3
# Se utiliza el dataframe creado previamente en el punto: 4.7.1 : df_libro_mayor_old5 (NO ES NECESARIO TRATAR NUEVAMENTE LOS DATOS DEL LIBRO MAYOR DE AÑOS ANTRIORES)

print("Dimensiones del dataframe df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)
df_libro_mayor_old5.sample(10)

Dimensiones del dataframe df_libro_mayor_old5 :  (253, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
252,S-1601,3. COSTO MANO DE OBRA,-1233034
179,P-1690,3. COSTO MANO DE OBRA,-16380832
247,P-1718,2. COSTO MATERIALES,-7325767
40,P-1637,1. INGRESO,810940
238,P-1714,1. INGRESO,34349625
45,P-1638,3. COSTO MANO DE OBRA,-1251366
217,P-1703,2. COSTO MATERIALES,-7394307
59,P-1644,3. COSTO MANO DE OBRA,-341190
83,P-1656,2. COSTO MATERIALES,-7997069
161,P-1684,3. COSTO MANO DE OBRA,-5364295


#### 4.8.2 - Concatenación de Dataframes

In [345]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado_anterior = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)
df_ptd_cerrado_anterior.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
58,P-1685,2. COSTO MATERIALES,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.
151,P-1723,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.
161,P-1726,1. INGRESO,0,NaN,NaN
28,P-1667,2. COSTO MATERIALES,0,ECO PLACILLA,CONSTRUCTORA ICF SPA
321,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN


In [346]:
# MERGE 1 - df_ytd_cerrado_2M_3_2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_ytd_cerrado_2M_3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
289,P-1692,4. COSTO SUPERVISIÓN,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,0
57,P-1684,1. INGRESO,0,EDIFICIO GOYCOLEA,EBCO S.A.,6708755
66,P-1688,2. COSTO MATERIALES,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-14620172
317,P-1719,2. COSTO MATERIALES,0,CD METALPAR,EBCO S.A.,0
97,P-1700,3. COSTO MANO DE OBRA,0,CD ORIZON,EBCO S.A.,-4264241
7,P-1626,2. COSTO MATERIALES,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460
103,P-1703,3. COSTO MANO DE OBRA,0,SANTA VICTORIA DANACORP,DANACORP S.A,-8696794
232,P-1660,4. COSTO SUPERVISIÓN,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0
98,P-1700,2. COSTO MATERIALES,0,CD ORIZON,EBCO S.A.,-7728748
240,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,COMERCIAL K SPA.,0


In [347]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
66,P-1688,2. COSTO MATERIALES,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-14620172,-7648784.00
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,0,NaN
226,P-1656,4. COSTO SUPERVISIÓN,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0,NaN
63,P-1687,2. COSTO MATERIALES,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,-6360393,-3323092.00
214,P-1645,4. COSTO SUPERVISIÓN,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0,NaN
318,P-1719,3. COSTO MANO DE OBRA,0,CD METALPAR,EBCO S.A.,0,NaN
250,P-1669,1. INGRESO,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0,19895589.00
144,P-1720,3. COSTO MANO DE OBRA,0,NaN,NaN,0,NaN
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0,NaN
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,NaN


In [348]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL YTD"] = df_ptd_cerrado_anterior["TOTAL YTD"].fillna(0)
df_ptd_cerrado_anterior["TOTAL LB OLD"] = df_ptd_cerrado_anterior["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL"] = df_ptd_cerrado_anterior["TOTAL YTD"] + df_ptd_cerrado_anterior["TOTAL LB OLD"]
df_ptd_cerrado_anterior.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
52,P-1682,2. COSTO MATERIALES,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-11527560,-14706814.00,-26234374.00
237,P-1663,3. COSTO MANO DE OBRA,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0,0.00,0.00
156,P-1725,3. COSTO MANO DE OBRA,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-2705788,0.00,-2705788.00
117,P-1707,1. INGRESO,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,4178952,25887019.00,30065971.00
301,P-1703,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA DANACORP,DANACORP S.A,0,0.00,0.00


In [349]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]

301368788
1369937537.0
1671306325.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,3185711545.00,4304275441.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435497453,-1152612025.00,-1588109478.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,-611212228.00,-974986478.00


In [350]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]

38892837
252119092.0
291011929.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00,0.00


#### 4.8.3 - SE AGREGAN FILAS DE 1.TOTAL

In [351]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior[~((df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado_anterior["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (337, 8)


In [352]:
# TESTING VISUAL 2


print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]

-17923405
-51949755.0
-69873160.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00


In [353]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_total = df_ptd_cerrado_anterior2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado_anterior2 = pd.concat([df_ptd_cerrado_anterior2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado_anterior2.loc[df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_ptd_cerrado_anterior2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
311,P-1715,4. COSTO SUPERVISIÓN,0.00,CD LO ECHEVERS,QUEZADA Y BOETSCH,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,0.00,-2483998.00
13,P-1643,2. COSTO MATERIALES,0.00,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,-102685.00,-102685.00
179,P-1603,4. COSTO SUPERVISIÓN,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0.00,0.00,0.00
187,P-1620,3. COSTO MANO DE OBRA,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00,0.00,0.00


In [354]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1423926747.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1516685549.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-69873160.00


In [355]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior2[~((df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado_anterior2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado_anterior2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "P-1704"].head(5)

1493799907.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


#### 4.8.4 - AGREGACIÓN COLUMNA MARGEN

In [356]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_anterior3 = df_ptd_cerrado_anterior2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_anterior3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado_anterior3 = pd.merge(df_ptd_cerrado_anterior3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)
df_ptd_cerrado_anterior3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
278,P-1685,1. INGRESO,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0.00,48965677.00,48965677.00,0.04
280,P-1685,4. COSTO SUPERVISIÓN,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0.00,0.00,0.00,0.04
215,P-1648,1. INGRESO,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0.00,672750.00,672750.00,2.17
93,P-1698,1. INGRESO,0.00,CD ANDES,EBCO S.A.,909040.00,69364255.00,70273295.00,0.36
167,P-1729,2. COSTO MATERIALES,0.00,NaN,NaN,0.00,0.00,0.00,0.00


In [357]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1493799907.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1516685549.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00,0.37


In [358]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1704"].head(5)

2536455.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48


In [359]:
df_ptd_cerrado_anterior3.to_csv('archivos_respaldo/df_ptd_cerrado_anterior3.csv', index = False)

### 4.9 - MERGE PTD + MES EN CURSO (PROYECTADO)
Cruce entre df_ptd_cerrado3 + mes_en_curso_proy5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.9.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [360]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado3_1 = df_ptd_cerrado3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado3_1 = df_ptd_cerrado3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   : ", df_ptd_cerrado3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado3_1 = df_ptd_cerrado3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
182,P-1611,4. COSTO SUPERVISIÓN,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00
230,P-1660,1. INGRESO,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,26532592.00
313,P-1717,1. INGRESO,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00
153,P-1724,3. COSTO MANO DE OBRA,NaN,NaN,-506066.00
54,P-1683,2. COSTO MATERIALES,CD BP-A05,EBCO S.A.,-23738720.00


In [361]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
mes_en_curso_proy5_1 = mes_en_curso_proy5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy5_1 = mes_en_curso_proy5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  : ", mes_en_curso_proy5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 : ", mes_en_curso_proy5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy5_1 = mes_en_curso_proy5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
118,P-1708,3. COSTO MANO DE OBRA,-208532.00
173,P-1591,4. COSTO SUPERVISIÓN,0.00
81,P-1694,2. COSTO MATERIALES,-8000000.00
79,P-1693,2. COSTO MATERIALES,0.00
210,P-1644,1. INGRESO,0.00


#### 4.9.2 - MERGE DE DATAFRAMES 

In [362]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

df_ptd_cerrado_y_proyectado = pd.merge(df_ptd_cerrado3_1, mes_en_curso_proy5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado : ", df_ptd_cerrado_y_proyectado.shape)

df_ptd_cerrado_y_proyectado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
111,P-1705,1. INGRESO,LOS CLARINES,CONSTRUCTORA INARCO S.A.,30287817.00,12109554.10
70,P-1690,2. COSTO MATERIALES,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,-22241592.00,-2000000.00
10,P-1639,2. COSTO MATERIALES,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-11381381.00,0.00
236,P-1663,1. INGRESO,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0.00,0.00
250,P-1669,1. INGRESO,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,19895589.00,0.00


In [363]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL PTD"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"] + df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
332,P-1728,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,897720.00,0.00,897720.00
252,P-1670,3. COSTO MANO DE OBRA,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,-431689.00,0.00,-431689.00
319,P-1719,4. COSTO SUPERVISIÓN,CD METALPAR,EBCO S.A.,0.00,0.00,0.00
153,P-1724,3. COSTO MANO DE OBRA,NaN,NaN,-506066.00,0.00,-506066.00


In [364]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1792805657.882083


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261.00,-79159682.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,420312383.88,4923504958.88
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686220937.00,-174535806.00,-1860756743.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715.00,-1190782876.00


In [365]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"].head(5)

22810427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213.00,-11625106.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000.00,-23655419.00
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,29520000.00,58090952.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00


#### 4.9.3 - AGREGACIÓN COLUMNA MARGEN

In [366]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado2 = df_ptd_cerrado_y_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2 = pd.merge(df_ptd_cerrado_y_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)
df_ptd_cerrado_y_proyectado2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
223,P-1653,4. COSTO SUPERVISIÓN,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,0.00,0.00,0.59
101,P-1701,2. COSTO MATERIALES,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,-33214441.00,0.00,-33214441.00,0.26
13,P-1643,2. COSTO MATERIALES,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-433653.00,0.00,-433653.00,0.00
273,P-1682,4. COSTO SUPERVISIÓN,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.37
286,P-1689,4. COSTO SUPERVISIÓN,CASINO BUENAVENTURA,EBCO S.A.,0.00,0.00,0.00,0.51
155,P-1724,1. INGRESO,NaN,NaN,28935273.00,0.00,28935273.00,0.72
89,P-1697,3. COSTO MANO DE OBRA,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-20207041.00,-4485402.00,-24692443.00,0.43
14,P-1644,2. COSTO MATERIALES,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,-4583696.00,0.00,-4583696.00,0.71
36,P-1673,2. COSTO MATERIALES,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,-11666275.00,0.00,-11666275.00,0.56
328,P-1725,4. COSTO SUPERVISIÓN,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,0.00,0.00,0.00,0.33


In [367]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1792805657.882083


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261.00,-79159682.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,420312383.88,4923504958.88,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686220937.00,-174535806.00,-1860756743.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715.00,-1190782876.00,0.36


In [368]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

22810427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213.00,-11625106.00,0.39
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000.00,-23655419.00,0.39
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,29520000.00,58090952.00,0.39
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00,0.39


In [369]:
df_ptd_cerrado_y_proyectado2.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado2.csv', index = False)

### 4.10 - MERGE PTD + MES EN CURSO (PROYECTADO) (PERSPECTIVA MES ANTERIOR)
Cruce entre df_ptd_cerrado_anterior3 + mes_en_curso_proy_2M_5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.10.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [370]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   : ", df_ptd_cerrado_anterior3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado_anterior3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
235,P-1661,4. COSTO SUPERVISIÓN,SAN VICENTE DE PAUL,DANACORP S.A,0.00
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00
67,P-1688,1. INGRESO,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,41287291.00
143,P-1719,1. INGRESO,CD METALPAR,EBCO S.A.,131348.00
157,P-1725,2. COSTO MATERIALES,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4180127.00


In [371]:
# TRATAMIENTO DATAFRAME mes en curso proyectado

# Se realiza una copia del dataframe 
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   : ", mes_en_curso_proy_2M_5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 : ", mes_en_curso_proy_2M_5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy_2M_5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
255,P-1672,4. COSTO SUPERVISIÓN,0.00
265,P-1676,4. COSTO SUPERVISIÓN,0.00
15,P-1645,2. COSTO MATERIALES,0.00
20,P-1656,2. COSTO MATERIALES,0.00
10,P-1639,2. COSTO MATERIALES,0.00


#### 4.10.2 - MERGE DE DATAFRAMES 

In [372]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

df_ptd_cerrado_y_proyectado_ant = pd.merge(df_ptd_cerrado_anterior3_1, mes_en_curso_proy_2M_5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant : ", df_ptd_cerrado_y_proyectado_ant.shape)

df_ptd_cerrado_y_proyectado_ant.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
175,P-1592,3. COSTO MANO DE OBRA,NUEVA COSTANERA 3300,EBCO S.A.,0.00,0.00
227,P-1658,1. INGRESO,VIVIPRA,VICHERAT Y PRADENAS LTDA.,30794481.00,0.00
270,P-1681,2. COSTO MATERIALES,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,-2962572.00,0.00
159,P-1726,3. COSTO MANO DE OBRA,NaN,NaN,-2654140.00,-2654140.00
94,P-1699,3. COSTO MANO DE OBRA,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-19304691.00,-8267470.00


In [373]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] + df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado_ant.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
52,P-1682,2. COSTO MATERIALES,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-26234374.00,-200000.00,-26434374.00
13,P-1643,2. COSTO MATERIALES,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-102685.00,0.00,-102685.00
176,P-1592,4. COSTO SUPERVISIÓN,NUEVA COSTANERA 3300,EBCO S.A.,0.00,0.00,0.00
69,P-1690,3. COSTO MANO DE OBRA,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,-21545891.00,0.00,-21545891.00
288,P-1691,4. COSTO SUPERVISIÓN,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0.00,0.00,0.00


In [374]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1620339891.221817


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,455821573.22,4498215635.22
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1516685549.00,-212300000.00,-1728985549.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00


In [375]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"].head(5)

7086615.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-4714680.00,-3205300.00,-7919980.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-3753865.00,-9500000.00,-13253865.00
164,P-1727,1. INGRESO,NaN,NaN,11005000.00,17255460.00,28260460.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00


#### 4.10.3 - AGREGACIÓN COLUMNA MARGEN

In [376]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado_ant2 = df_ptd_cerrado_y_proyectado_ant.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado_ant2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2 = pd.merge(df_ptd_cerrado_y_proyectado_ant2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)
df_ptd_cerrado_y_proyectado_ant2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
125,P-1712,2. COSTO MATERIALES,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-40112958.00,-8500000.00,-48612958.00,0.53
287,P-1690,4. COSTO SUPERVISIÓN,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0.00,0.00,0.00,0.18
297,P-1698,4. COSTO SUPERVISIÓN,CD ANDES,EBCO S.A.,0.00,0.00,0.00,0.36
243,P-1666,3. COSTO MANO DE OBRA,CD AXIONLONG,EBCO S.A.,-1957926.00,0.00,-1957926.00,0.63
13,P-1643,2. COSTO MATERIALES,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-102685.00,0.00,-102685.00,0.00
75,P-1692,3. COSTO MANO DE OBRA,JOSÉ PEREZ COTAPOS,EBCO S.A.,-40393357.00,-7576783.00,-47970140.00,-0.06
29,P-1668,2. COSTO MATERIALES,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,-14024713.00,0.00,-14024713.00,0.37
189,P-1626,1. INGRESO,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,75533.00,0.00,75533.00,10.03
186,P-1620,1. INGRESO,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00,0.00,0.00,0.00
134,P-1715,2. COSTO MATERIALES,CD LO ECHEVERS,QUEZADA Y BOETSCH,-22024051.00,-6000000.00,-28024051.00,0.24


In [377]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1620339891.221817


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,455821573.22,4498215635.22,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1516685549.00,-212300000.00,-1728985549.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00,0.36


In [378]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"].head(5)

321376053.27884996


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


In [379]:
df_ptd_cerrado_y_proyectado_ant.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado_ant.csv', index = False)

### 4.11 MERGE PRESUPUESTO
Se realiza mergen entre los datos que provienen de la planilla de presupuesto y tabla de presupuestos.

#### 4.11.1 CREACIÓN Y TRATAMIENTO INICIAL DE DATAFRAMES A CONCANTENAR.

In [380]:
print("DIMENSIONES DATAFRAME *df_tabla_presupuesto3* :", df_tabla_presupuesto3.shape)
df_tabla_presupuesto4 = df_tabla_presupuesto3.copy()
print("DIMENSIONES DATAFRAME *df_presupuesto2* :", df_presupuesto2.shape)
df_presupuesto3 = df_presupuesto2.copy()

DIMENSIONES DATAFRAME *df_tabla_presupuesto3* : (388, 5)
DIMENSIONES DATAFRAME *df_presupuesto2* : (70, 4)


In [381]:
df_tabla_presupuesto4 = df_tabla_presupuesto4[['CODIGO_PROYECTO', 'TIPO_COSTO', 'TOTAL', 'ORIGEN']]
df_tabla_presupuesto4.rename(columns={'TIPO_COSTO': 'TIPO_CUENTA'}, inplace=True)

df_tabla_presupuesto4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
93,P-1633,2. COSTO MATERIALES,-21923791.00,TABLA_PRESUPUESTO
33,P-1613,2. COSTO MATERIALES,-21496374.00,TABLA_PRESUPUESTO
356,P-1725,2. COSTO MATERIALES,-46312307.00,TABLA_PRESUPUESTO
139,P-1648,3. COSTO MANO DE OBRA,-33676709.00,TABLA_PRESUPUESTO
252,P-1686,2. COSTO MATERIALES,-33753271.00,TABLA_PRESUPUESTO
282,P-1697,2. COSTO MATERIALES,-43150692.00,TABLA_PRESUPUESTO
77,P-1627,4. COSTO SUPERVISIÓN,-1355789.00,TABLA_PRESUPUESTO
347,P-1721,2. COSTO MATERIALES,-26959401.00,TABLA_PRESUPUESTO
110,P-1638,4. COSTO SUPERVISIÓN,-1226616.00,TABLA_PRESUPUESTO
24,P-1610,2. COSTO MATERIALES,-52360443.00,TABLA_PRESUPUESTO


In [382]:
df_presupuesto3.rename(columns={'VALOR': 'TOTAL'}, inplace=True)
df_presupuesto3.sample(10)

,CODIGO_PROYECTO,TOTAL,TIPO_CUENTA,ORIGEN
43,P-1699,406744605.61,1. INGRESO,PLANILLA PRESUPUESTO
38,P-1694,189774065.87,1. INGRESO,PLANILLA PRESUPUESTO
25,P-1681,11390647.75,1. INGRESO,PLANILLA PRESUPUESTO
23,P-1679,169118040.00,1. INGRESO,PLANILLA PRESUPUESTO
41,P-1697,97580171.14,1. INGRESO,PLANILLA PRESUPUESTO
70,P-1728,7874868.00,1. INGRESO,PLANILLA PRESUPUESTO
57,P-1715,335709882.43,1. INGRESO,PLANILLA PRESUPUESTO
18,S-1601,16486000.00,1. INGRESO,PLANILLA PRESUPUESTO
47,P-1703,56316406.63,1. INGRESO,PLANILLA PRESUPUESTO
30,P-1686,105081270.13,1. INGRESO,PLANILLA PRESUPUESTO


#### 4.11.2 SE CONCATENA LOS DATAFRAMES

In [383]:
# Imprimir las dimensiones de los DataFrames
print("Dimensiones de df_tabla_presupuesto4:", df_tabla_presupuesto4.shape)
print("Dimensiones de df_presupuesto3:", df_presupuesto3.shape)

# Concatenar los DataFrames
df_presupuesto_final = pd.concat([df_tabla_presupuesto4, df_presupuesto3], ignore_index=True)
print("Dimensiones de df_presupuesto_final:", df_presupuesto_final.shape)

df_presupuesto_final.sample(10)

Dimensiones de df_tabla_presupuesto4: (388, 4)
Dimensiones de df_presupuesto3: (70, 4)
Dimensiones de df_presupuesto_final: (458, 4)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
278,P-1695,3. COSTO MANO DE OBRA,-19577383.00,TABLA_PRESUPUESTO
456,P-1727,1. INGRESO,166727122.73,PLANILLA PRESUPUESTO
283,P-1697,3. COSTO MANO DE OBRA,0.00,TABLA_PRESUPUESTO
50,P-1618,4. COSTO SUPERVISIÓN,-685684.00,TABLA_PRESUPUESTO
74,P-1626,4. COSTO SUPERVISIÓN,-8577318.00,TABLA_PRESUPUESTO
204,P-1670,2. COSTO MATERIALES,-77955381.00,TABLA_PRESUPUESTO
7,P-1604,3. COSTO MANO DE OBRA,-1518738.00,TABLA_PRESUPUESTO
325,P-1712,3. COSTO MANO DE OBRA,-30109897.00,TABLA_PRESUPUESTO
262,P-1689,3. COSTO MANO DE OBRA,-2190200.00,TABLA_PRESUPUESTO
148,P-1651,3. COSTO MANO DE OBRA,-858400.00,TABLA_PRESUPUESTO


#### 4.11.3 SE REALIZA MERGE CON CODIGOS DE PROYECTO

In [384]:
# Se hace merge con los codigos de proyecto.

print("Dimensiones de df_libro_mayor5_base2", df_libro_mayor5_base2.shape)

df_presupuesto_final2 = pd.merge(df_libro_mayor5_base2, df_presupuesto_final, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("Dimensiones de df_presupuesto_final2", df_presupuesto_final2.shape)
df_presupuesto_final2.sample(10)

Dimensiones de df_libro_mayor5_base2 (340, 5)
Dimensiones de df_presupuesto_final2 (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,NaN,NaN
240,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,COMERCIAL K SPA.,-343022.00,TABLA_PRESUPUESTO
50,P-1681,1. INGRESO,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,11390647.75,PLANILLA PRESUPUESTO
217,P-1648,4. COSTO SUPERVISIÓN,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,-8171095.00,TABLA_PRESUPUESTO
328,P-1725,4. COSTO SUPERVISIÓN,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4468043.00,TABLA_PRESUPUESTO
12,P-1642,2. COSTO MATERIALES,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,-7385337.00,TABLA_PRESUPUESTO
282,P-1687,4. COSTO SUPERVISIÓN,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,-1083403.00,TABLA_PRESUPUESTO
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-8420659.00,TABLA_PRESUPUESTO
121,P-1709,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,-8592182.00,TABLA_PRESUPUESTO
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,215321946.83,PLANILLA PRESUPUESTO


#### 4.11.4 SE AGREGAN FILAS DE "1.TOTAL"

In [385]:
# Calcular la suma de los valores totales para cada tipo de cuenta


df_total = df_presupuesto_final2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_presupuesto_final2 = pd.concat([df_presupuesto_final2, df_total], ignore_index=True)

df_presupuesto_final2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
230,P-1660,1. INGRESO,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,39366390.00,PLANILLA PRESUPUESTO
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,NaN,NaN
120,P-1708,1. INGRESO,0.00,ECO ESPAÑA,CONSTRUCTORA ICF SPA,213452539.86,PLANILLA PRESUPUESTO
117,P-1707,1. INGRESO,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,67945807.16,PLANILLA PRESUPUESTO
53,P-1682,1. INGRESO,0.00,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,77511681.77,PLANILLA PRESUPUESTO
144,P-1720,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.00,TABLA_PRESUPUESTO
74,P-1691,1. INGRESO,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,5751334.03,PLANILLA PRESUPUESTO
239,P-1664,1. INGRESO,0.00,MK II PCI,COMERCIAL K SPA.,NaN,NaN
142,P-1718,1. INGRESO,0.00,PATIO COMIDAS,EBCO S.A.,184688657.35,PLANILLA PRESUPUESTO
104,P-1703,2. COSTO MATERIALES,0.00,SANTA VICTORIA DANACORP,DANACORP S.A,-22488985.00,TABLA_PRESUPUESTO


In [386]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_presupuesto_final2.loc[df_presupuesto_final2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_presupuesto_final2.sample(10)      

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
27,P-1667,3. COSTO MANO DE OBRA,0.00,ECO PLACILLA,CONSTRUCTORA ICF SPA,-16552739.00,TABLA_PRESUPUESTO
41,P-1676,2. COSTO MATERIALES,0.00,LO CAMPINO,EBCO S.A.,-41750844.00,TABLA_PRESUPUESTO
30,P-1669,3. COSTO MANO DE OBRA,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,-9423692.00,TABLA_PRESUPUESTO
306,P-1708,4. COSTO SUPERVISIÓN,0.00,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-10663380.00,TABLA_PRESUPUESTO
45,P-1679,3. COSTO MANO DE OBRA,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,-28372514.00,TABLA_PRESUPUESTO
29,P-1668,2. COSTO MATERIALES,0.00,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,-14798537.00,TABLA_PRESUPUESTO
17,P-1648,2. COSTO MATERIALES,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,-220215219.50,TABLA_PRESUPUESTO
155,P-1724,1. INGRESO,0.00,NaN,NaN,28935271.00,PLANILLA PRESUPUESTO
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,TABLA_PRESUPUESTO
86,P-1696,3. COSTO MANO DE OBRA,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0.00,TABLA_PRESUPUESTO


#### 4.11.5 AGREGACIÓN COLUMNA MARGEN

In [387]:
#CREACIÓN DE MARGENES

df_presupuesto_final3 = df_presupuesto_final2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_presupuesto_final3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 : ", df_presupuesto_final3.shape)

df_presupuesto_final4 = pd.merge(df_presupuesto_final3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 : ", df_presupuesto_final4.shape)
df_presupuesto_final4.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 :  (344, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 :  (344, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
153,P-1724,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.00,TABLA_PRESUPUESTO,0.66
86,P-1696,3. COSTO MANO DE OBRA,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0.00,TABLA_PRESUPUESTO,0.60
92,P-1698,2. COSTO MATERIALES,0.00,CD ANDES,EBCO S.A.,-27960522.00,TABLA_PRESUPUESTO,0.37
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0.00,TABLA_PRESUPUESTO,0.53
123,P-1709,1. INGRESO,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,105853124.47,PLANILLA PRESUPUESTO,0.48
49,P-1680,1. INGRESO,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,206761360.91,PLANILLA PRESUPUESTO,0.31
127,P-1713,3. COSTO MANO DE OBRA,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-19416318.00,TABLA_PRESUPUESTO,0.36
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,-219387084.00,TABLA_PRESUPUESTO,-0.96
220,P-1649,4. COSTO SUPERVISIÓN,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,-11599543.00,TABLA_PRESUPUESTO,0.00
177,P-1603,1. INGRESO,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,NaN,NaN,0.00


In [388]:
# TESTING VISUAL 1

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[~((df_presupuesto_final4['CODIGO_PROYECTO'] == '1. TOTAL') & df_presupuesto_final4['TOTAL'].isna())]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]

2050798485.5402288


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
340,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,7808050447.84,NaN,0.26
341,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-4375432891.30,NaN,0.26
342,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-1139100986.00,NaN,0.26
343,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-242718085.00,NaN,0.26


In [389]:
# TESTING VISUAL 2

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[(df_presupuesto_final4['CODIGO_PROYECTO'] != '1. TOTAL') | (df_presupuesto_final4['ORIGEN'] != 'AGREGACIÓN TIPO_CUENTA VALOR O')]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]

483525857.8694887


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,TABLA_PRESUPUESTO,0.46
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,TABLA_PRESUPUESTO,0.46
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,PLANILLA PRESUPUESTO,0.46
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,TABLA_PRESUPUESTO,0.46


In [390]:
df_presupuesto_final4.to_csv('archivos_respaldo/df_presupuesto_final4.csv', index = False)

### 4.12 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO)

#### 4.12.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [391]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
190,P-1626,3. COSTO MANO DE OBRA,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.
331,P-1728,1. INGRESO,0,NaN,NaN
149,P-1722,1. INGRESO,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA
114,P-1706,1. INGRESO,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.
76,P-1692,2. COSTO MATERIALES,0,JOSÉ PEREZ COTAPOS,EBCO S.A.


In [392]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1 = df_presupuesto_final4_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1 = pd.DataFrame(df_presupuesto_final4_1.to_records())

df_presupuesto_final4_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 : ", df_presupuesto_final4_1.shape)

df_presupuesto_final4_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
106,P-1666,3. COSTO MANO DE OBRA,-8652090.00
320,P-1725,1. INGRESO,103000000.00
59,P-1643,4. COSTO SUPERVISIÓN,0.00
253,P-1706,2. COSTO MATERIALES,-39538656.00
219,P-1696,4. COSTO SUPERVISIÓN,0.00
6,P-1557,3. COSTO MANO DE OBRA,0.00
224,P-1698,1. INGRESO,57807633.83
56,P-1643,1. INGRESO,0.00
50,P-1641,3. COSTO MANO DE OBRA,-3473560.00
166,P-1683,3. COSTO MANO DE OBRA,-8303894.00


In [393]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_1= df_ptd_cerrado3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_2 = df_ptd_cerrado3_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_2 = pd.DataFrame(df_ptd_cerrado3_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)
df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 : ", df_ptd_cerrado3_2.shape)

df_ptd_cerrado3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
109,P-1667,2. COSTO MATERIALES,-45657105.00
242,P-1703,3. COSTO MANO DE OBRA,-16302124.00
47,P-1639,4. COSTO SUPERVISIÓN,0.00
334,P-1728,3. COSTO MANO DE OBRA,-630869.00
41,P-1636,2. COSTO MATERIALES,-455187.00
72,P-1649,1. INGRESO,54564902.00
130,P-1673,3. COSTO MANO DE OBRA,0.00
320,P-1725,1. INGRESO,22301272.00
253,P-1706,2. COSTO MATERIALES,-13077846.00
65,P-1645,2. COSTO MATERIALES,-8405379.00


#### 4.12.2 MERGE ENTRE DATAFRAMES 

In [394]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    : ", df_presupuesto_final4_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente = df_libro_mayor5_base2.merge(df_presupuesto_final4_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)

df_pendiente.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
325,P-1722,4. COSTO SUPERVISIÓN,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA,0.00
272,P-1681,4. COSTO SUPERVISIÓN,0,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,-532619.00
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,-8577318.00
238,P-1663,4. COSTO SUPERVISIÓN,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,-2318715.00
123,P-1709,1. INGRESO,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,105853124.47


In [395]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente2 = df_pendiente.merge(df_ptd_cerrado3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              : ", df_pendiente2.shape)
df_pendiente2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
38,P-1675,2. COSTO MATERIALES,0,PALENA II,CONSTRUCTORA MYA SPA.,-7361719.00,-7984413.00
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-506066.00
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00
304,P-1706,4. COSTO SUPERVISIÓN,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,-4824811.00,0.00
210,P-1644,1. INGRESO,0,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,0.00,16864978.00


In [396]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-433653.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


In [397]:
# TESTING VISUAL 2
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7808050447.84,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1686220937.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00


#### 4.12.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [398]:
# Modificar los valores negativos a positivos
df_pendiente2['TOTAL_PREP2'] = np.where(df_pendiente2['TOTAL_PREP'] < 0, df_pendiente2['TOTAL_PREP'] * -1, df_pendiente2['TOTAL_PREP'])

df_pendiente2['TOTAL_PDT2'] = np.where(df_pendiente2['TOTAL_PDT'] < 0, df_pendiente2['TOTAL_PDT'] * -1, df_pendiente2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente2['TOTAL_1'] = df_pendiente2['TOTAL_PREP2'] - df_pendiente2['TOTAL_PDT2']

df_pendiente2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
115,P-1707,3. COSTO MANO DE OBRA,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-15679431.00,-15597431.00,15679431.00,15597431.00,82000.00
3,P-1603,2. COSTO MATERIALES,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-13744197.00,-30000.00,13744197.00,30000.00,13714197.00
95,P-1699,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-156685807.00,-31268214.00,156685807.00,31268214.00,125417593.00
62,P-1687,3. COSTO MANO DE OBRA,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,-4344898.00,-7395436.00,4344898.00,7395436.00,-3050538.00
235,P-1661,4. COSTO SUPERVISIÓN,0,SAN VICENTE DE PAUL,DANACORP S.A,-2015626.00,0.00,2015626.00,0.00,2015626.00


In [399]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-145534854.00,156095149.00,145534854.00,10560295.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-197683461.00,387816472.00,197683461.00,190133011.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,686431807.00,1058718831.87,686431807.00,372287024.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [400]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente2['TOTAL'] = df_pendiente2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente2['TOTAL_1'] != 0)
df_pendiente2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente2.loc[df_pendiente2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
172,P-1591,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EBCO S.A.,0.00,0.00,0.00,0.00,0.00,0.00
161,P-1726,1. INGRESO,0,NaN,NaN,76030481.44,46466438.00,76030481.44,46466438.00,29564043.44,29564043.44
3,P-1603,2. COSTO MATERIALES,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-13744197.00,-30000.00,13744197.00,30000.00,13714197.00,-13714197.00
277,P-1684,4. COSTO SUPERVISIÓN,0,EDIFICIO GOYCOLEA,EBCO S.A.,-5137694.00,0.00,5137694.00,0.00,5137694.00,-5137694.00
170,P-1557,4. COSTO SUPERVISIÓN,0,ROSARIO ROSALES,EBCO S.A.,0.00,0.00,0.00,0.00,0.00,0.00
67,P-1688,1. INGRESO,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,50214086.81,45817042.00,50214086.81,45817042.00,4397044.81,4397044.81
219,P-1649,3. COSTO MANO DE OBRA,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,-38928950.00,-220289.00,38928950.00,220289.00,38708661.00,-38708661.00
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,94918400.00,43693444.00,94918400.00,43693444.00,51224956.00,51224956.00
159,P-1726,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-8656540.00,0.00,8656540.00,-8656540.00,-8656540.00
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,-466503.00,0.00,466503.00,0.00,466503.00,-466503.00


In [401]:
df_pendiente3 = df_pendiente2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)

df_pendiente3 = pd.merge(df_pendiente3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)
df_pendiente3.sample(5)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
87,P-1696,2. COSTO MATERIALES,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-30348318.00,-30852143.00,30348318.00,30852143.00,-503825.00,-503825.00,-74.71
313,P-1717,1. INGRESO,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,170727176.42,0.00,170727176.42,0.00,170727176.42,170727176.42,0.59
155,P-1724,1. INGRESO,0,NaN,NaN,28935271.00,28935273.00,28935271.00,28935273.00,-2.00,2.00,-1368235.50
96,P-1699,1. INGRESO,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,406744605.61,94757672.00,406744605.61,94757672.00,311986933.61,311986933.61,0.53
297,P-1698,4. COSTO SUPERVISIÓN,0,CD ANDES,EBCO S.A.,-1552353.00,0.00,1552353.00,0.00,1552353.00,-1552353.00,-0.27


In [402]:
# TESTING VISUAL 1
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-8169893.00,0.00,8169893.00,-8169893.00,-8169893.00,0.37
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92610838.00,-13655419.00,92610838.00,13655419.00,78955419.00,-78955419.00,0.37
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,28570952.00,166727122.73,28570952.00,138156170.73,138156170.73,0.37
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.37


In [403]:
# TESTING VISUAL 2
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00,242718085.00,74516421.00,168201664.00,-168201664.00,0.12
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7808050447.84,4503192575.00,7808050447.84,4503192575.00,3304857872.84,3304857872.84,0.12
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1686220937.00,4375432891.30,1686220937.00,2689211954.30,-2689211954.30,0.12
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00,1139100986.00,1076409161.00,62691825.00,-62691825.00,0.12


In [404]:
df_pendiente3.to_csv('archivos_respaldo/df_pendiente3.csv', index = False)

### 4.13 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO) (PERSPECTIVA MES ANTERIOR)

#### 4.13.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [405]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
44,P-1677,1. INGRESO,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...
328,P-1725,4. COSTO SUPERVISIÓN,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO
258,P-1673,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP
185,P-1612,4. COSTO SUPERVISIÓN,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A
26,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.


In [406]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1_1 = df_presupuesto_final4_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1_1 = pd.DataFrame(df_presupuesto_final4_1_1.to_records())

df_presupuesto_final4_1_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 : ", df_presupuesto_final4_1_1.shape)

df_presupuesto_final4_1_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
27,P-1612,4. COSTO SUPERVISIÓN,-2259746.00
332,P-1728,1. INGRESO,7874868.00
266,P-1709,3. COSTO MANO DE OBRA,-8592182.00
301,P-1720,2. COSTO MATERIALES,-21939079.00
164,P-1683,1. INGRESO,57098333.68
163,P-1682,4. COSTO SUPERVISIÓN,0.00
72,P-1649,1. INGRESO,0.00
121,P-1670,2. COSTO MATERIALES,-77955381.00
188,P-1689,1. INGRESO,20986779.50
94,P-1661,3. COSTO MANO DE OBRA,-8083499.00


In [407]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado3_1_1 = df_ptd_cerrado_anterior3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_1_2 = df_ptd_cerrado3_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_1_2 = pd.DataFrame(df_ptd_cerrado3_1_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)
df_ptd_cerrado3_1_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)

df_ptd_cerrado3_1_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
73,P-1649,2. COSTO MATERIALES,-23854079.00
229,P-1699,2. COSTO MATERIALES,-22580828.00
329,P-1727,2. COSTO MATERIALES,-3753865.00
20,P-1611,1. INGRESO,4173520.00
247,P-1704,4. COSTO SUPERVISIÓN,0.00
316,P-1724,1. INGRESO,28844553.00
11,P-1591,4. COSTO SUPERVISIÓN,0.00
89,P-1660,2. COSTO MATERIALES,-12060331.00
213,P-1695,2. COSTO MATERIALES,-17009933.00
50,P-1641,3. COSTO MANO DE OBRA,0.00


#### 4.13.2 MERGE ENTRE DATAFRAMES 

In [408]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  : ", df_presupuesto_final4_1_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_libro_mayor5_base2.merge(df_presupuesto_final4_1_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)

df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
281,P-1686,4. COSTO SUPERVISIÓN,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-4212051.00
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-8420659.00
332,P-1728,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,-466503.00
115,P-1707,3. COSTO MANO DE OBRA,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-15679431.00


In [409]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_pendiente_2.merge(df_ptd_cerrado3_1_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              : ", df_pendiente_2.shape)
df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
311,P-1715,4. COSTO SUPERVISIÓN,0,CD LO ECHEVERS,QUEZADA Y BOETSCH,-7424606.00,0.00
326,P-1723,4. COSTO SUPERVISIÓN,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,0.00
87,P-1696,2. COSTO MATERIALES,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-30348318.00,-28547703.00
205,P-1642,3. COSTO MANO DE OBRA,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,-2799066.00,0.00
49,P-1680,1. INGRESO,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.,206761360.91,174843492.00


In [410]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-102685.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


#### 4.13.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [411]:
# Modificar los valores negativos a positivos
df_pendiente_2['TOTAL_PREP2'] = np.where(df_pendiente_2['TOTAL_PREP'] < 0, df_pendiente_2['TOTAL_PREP'] * -1, df_pendiente_2['TOTAL_PREP'])

df_pendiente_2['TOTAL_PDT2'] = np.where(df_pendiente_2['TOTAL_PDT'] < 0, df_pendiente_2['TOTAL_PDT'] * -1, df_pendiente_2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente_2['TOTAL_1'] = df_pendiente_2['TOTAL_PREP2'] - df_pendiente_2['TOTAL_PDT2']

df_pendiente_2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
131,P-1714,2. COSTO MATERIALES,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,-29967845.00,-37356272.00,29967845.00,37356272.00,-7388427.00
277,P-1684,4. COSTO SUPERVISIÓN,0,EDIFICIO GOYCOLEA,EBCO S.A.,-5137694.00,0.00,5137694.00,0.00,5137694.00
26,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.,-45639112.40,-25105908.00,45639112.40,25105908.00,20533204.40
71,P-1690,1. INGRESO,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,64872885.78,54714366.00,64872885.78,54714366.00,10158519.78
83,P-1695,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-19577383.00,-13163688.00,19577383.00,13163688.00,6413695.00


In [412]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-132643971.00,156095149.00,132643971.00,23451178.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-183715910.00,387816472.00,183715910.00,204100562.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,607371810.00,1058718831.87,607371810.00,451347021.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [413]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente_2['TOTAL'] = df_pendiente_2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente_2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente_2['TOTAL_1'] != 0)
df_pendiente_2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente_2.loc[df_pendiente_2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente_2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
148,P-1721,1. INGRESO,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,58149422.75,23320880.00,58149422.75,23320880.00,34828542.75,34828542.75
205,P-1642,3. COSTO MANO DE OBRA,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,-2799066.00,0.00,2799066.00,0.00,2799066.00,-2799066.00
174,P-1592,1. INGRESO,0,NUEVA COSTANERA 3300,EBCO S.A.,0.00,17821101.00,0.00,17821101.00,-17821101.00,17821101.00
226,P-1656,4. COSTO SUPERVISIÓN,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,-1293524.00,0.00,1293524.00,0.00,1293524.00,-1293524.00
155,P-1724,1. INGRESO,0,NaN,NaN,28935271.00,28844553.00,28935271.00,28844553.00,90718.00,90718.00
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,94918400.00,30834000.00,94918400.00,30834000.00,64084400.00,64084400.00
23,P-1661,2. COSTO MATERIALES,0,SAN VICENTE DE PAUL,DANACORP S.A,-15916097.00,-14270769.00,15916097.00,14270769.00,1645328.00,-1645328.00
119,P-1708,2. COSTO MATERIALES,0,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-83658973.00,-125100.00,83658973.00,125100.00,83533873.00,-83533873.00
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00,-31281353.00
94,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,0.00,-19304691.00,0.00,19304691.00,-19304691.00,-19304691.00


In [414]:
df_pendiente_3 = df_pendiente_2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)

df_pendiente_3 = pd.merge(df_pendiente_3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)
df_pendiente_3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
215,P-1648,1. INGRESO,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0.00,672750.00,0.00,672750.00,-672750.00,672750.00,-387.37
151,P-1723,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-147892783.00,-10817986.00,147892783.00,10817986.00,137074797.00,-137074797.00,0.52
150,P-1723,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,-10269821.00,0.00,10269821.00,-10269821.00,-10269821.00,0.52
203,P-1641,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-657160.00,0.00,657160.00,0.00,657160.00,-657160.00,0.00
232,P-1660,4. COSTO SUPERVISIÓN,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,-1495636.00,0.00,1495636.00,0.00,1495636.00,-1495636.00,0.76


In [415]:
# TESTING VISUAL 1
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-4714680.00,0.00,4714680.00,-4714680.00,-4714680.00,0.40
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92610838.00,-3753865.00,92610838.00,3753865.00,88856973.00,-88856973.00,0.40
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,11005000.00,166727122.73,11005000.00,155722122.73,155722122.73,0.40
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.40


In [416]:
# TESTING VISUAL 2
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-69873160.00,242718085.00,69873160.00,172844925.00,-172844925.00,0.15
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7808050447.84,4042394062.00,7808050447.84,4042394062.00,3765656385.84,3765656385.84,0.15
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375432891.30,-1516685549.00,4375432891.30,1516685549.00,2858747342.30,-2858747342.30,0.15
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-962035446.00,1139100986.00,962035446.00,177065540.00,-177065540.00,0.15


In [417]:
df_pendiente_3.to_csv('archivos_respaldo/df_pendiente_3.csv', index = False)

### 4.14 MERGE PDT CERRADO VS PRESUPUESTO

#### 4.14.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [418]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2 = df_ptd_cerrado3.copy()

df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
244,P-1666,4. COSTO SUPERVISIÓN,0.00,CD AXIONLONG,EBCO S.A.,0.00,0.00,0.00,0.63
92,P-1698,2. COSTO MATERIALES,0.00,CD ANDES,EBCO S.A.,924.00,-25888301.00,-25887377.00,0.36
157,P-1725,2. COSTO MATERIALES,0.00,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-10888672.00,0.00,-10888672.00,0.28
117,P-1707,1. INGRESO,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,11263421.00,25887019.00,37150440.00,0.15
307,P-1709,4. COSTO SUPERVISIÓN,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,0.00,0.00,0.00,0.44


In [419]:
# Exploración de dataframe presupuesto
df_presupuesto_final4.sample(10)

df_presupuesto_final4_1 = df_presupuesto_final4.copy()
df_presupuesto_final4_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_2 = pd.pivot_table(df_presupuesto_final4_1, values=['TOTAL_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2 = pd.DataFrame(df_presupuesto_final4_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
107,P-1666,4. COSTO SUPERVISIÓN,-1955309.00
259,P-1707,4. COSTO SUPERVISIÓN,-3909676.00
166,P-1683,3. COSTO MANO DE OBRA,-8303894.00
137,P-1675,2. COSTO MATERIALES,-7361719.00
178,P-1686,3. COSTO MANO DE OBRA,-19743989.00


#### 4.14.2 MERGE ENTRE DATAFRAMES

In [420]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  : ", df_ptd_cerrado3_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2            : ", df_presupuesto_final4_2.shape)

df_pdt_vs_prep = df_ptd_cerrado3_2.merge(df_presupuesto_final4_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_prep           : ", df_pdt_vs_prep.shape)

df_pdt_vs_prep.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2            :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_prep           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP
123,P-1709,1. INGRESO,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,76827615.00,7158358.00,83985973.00,0.44,105853124.47
166,P-1728,2. COSTO MATERIALES,0.00,NaN,NaN,-1822779.00,0.00,-1822779.00,0.00,-4358412.00
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-26152479.00,-4855555.00,-31008034.00,0.54,-56794940.00
131,P-1714,2. COSTO MATERIALES,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,-6486288.00,-31104108.00,-37590396.00,0.02,-29967845.00
42,P-1677,3. COSTO MANO DE OBRA,0.00,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-1992711.00,-24439480.00,-26432191.00,0.35,-18678829.00


#### 4.14.3 CREACIÓN DE COLUMNA % DE AVANCE

In [421]:
df_pdt_vs_prep['% AVANCE'] = df_pdt_vs_prep['TOTAL_PDT'] / df_pdt_vs_prep['TOTAL_PREP']

# Reemplazar NaN por cero
df_pdt_vs_prep['% AVANCE'] = df_pdt_vs_prep['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_prep['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_prep['% AVANCE'] = df_pdt_vs_prep['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_prep.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-1247014.00,0.00,-1247014.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,0.00,-4593960.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,41716.00,856004.00,897720.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,75965.00,236593.00,312558.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-11381553.00,0.04


In [422]:
# TESTING VISUAL 1
 
df_pdt_vs_prep[df_pdt_vs_prep["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24,0.00,0.00
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24,-92610838.00,0.15
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24,166727122.73,0.17
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24,0.00,0.00


In [423]:
# TESTING VISUAL 2
 
df_pdt_vs_prep[df_pdt_vs_prep["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37,-242718085.00,0.31
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37,7808050447.84,0.58
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686220937.00,0.37,-4375432891.30,0.39
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37,-1139100986.00,0.94


In [424]:
df_pdt_vs_prep.to_csv('archivos_respaldo/df_pdt_vs_prep.csv', index = False)

### 4.15 MERGE PDT CERRADO VS PRESUPUESTO (PERSPECTIVA MES ANTERIOR)

#### 4.15.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [425]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2_2 = df_ptd_cerrado_anterior3.copy()

df_ptd_cerrado3_2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
27,P-1667,3. COSTO MANO DE OBRA,0.00,ECO PLACILLA,CONSTRUCTORA ICF SPA,-5776759.00,-31446381.00,-37223140.00,0.02
230,P-1660,1. INGRESO,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0.00,26532592.00,26532592.00,0.31
85,P-1695,1. INGRESO,0.00,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,15860326.00,14973674.00,30834000.00,0.02
81,P-1694,2. COSTO MATERIALES,0.00,ECO HUEMUL,CONSTRUCTORA ICF SPA,-14391526.00,-4165442.00,-18556968.00,0.20
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,0.00,-2483998.00,0.00


In [426]:
# Exploración presupuesto, creado en el punto anterior y que se reutiliza en éste.

df_presupuesto_final4_2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
233,P-1700,2. COSTO MATERIALES,-72938784.00
133,P-1674,2. COSTO MATERIALES,-12233776.00
166,P-1683,3. COSTO MANO DE OBRA,-8303894.00
148,P-1679,1. INGRESO,169118040.00
188,P-1689,1. INGRESO,20986779.50


#### 4.15.2 MERGE ENTRE DATAFRAMES

In [427]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_                 : ", df_ptd_cerrado3_2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2            : ", df_presupuesto_final4_2.shape)

df_pdt_vs_prep_2 = df_ptd_cerrado3_2_2.merge(df_presupuesto_final4_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_prep_2           : ", df_pdt_vs_prep_2.shape)

df_pdt_vs_prep_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_                 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2            :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_prep_2           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP
74,P-1691,1. INGRESO,0.00,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,2124157.00,39478959.00,41603116.00,0.16,5751334.03
241,P-1664,4. COSTO SUPERVISIÓN,0.00,MK II PCI,COMERCIAL K SPA.,0.00,0.00,0.00,0.64,-76676.00
230,P-1660,1. INGRESO,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0.00,26532592.00,26532592.00,0.31,39366390.00
11,P-1641,2. COSTO MATERIALES,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-369720.00,-669050.00,-1038770.00,0.00,-30498737.00
103,P-1703,3. COSTO MANO DE OBRA,0.00,SANTA VICTORIA DANACORP,DANACORP S.A,-8696794.00,-7235935.00,-15932729.00,0.31,-10943242.00


#### 4.15.3 CREACIÓN DE COLUMNA % DE AVANCE

In [430]:
df_pdt_vs_prep_2['% AVANCE'] = df_pdt_vs_prep_2['TOTAL_PDT'] / df_pdt_vs_prep_2['TOTAL_PREP']

# Reemplazar NaN por cero
df_pdt_vs_prep_2['% AVANCE'] = df_pdt_vs_prep_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_prep_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_prep_2['% AVANCE'] = df_pdt_vs_prep_2['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_prep_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-945949.00,0.00,-945949.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,0.00,-2483998.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,-9821.00,856004.00,846183.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,236593.00,236593.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-11381553.00,0.04


In [431]:
# TESTING VISUAL 1
 
df_pdt_vs_prep_2[df_pdt_vs_prep_2["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-4714680.00,0.00,-4714680.00,0.23,0.00,0.00
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-3753865.00,0.00,-3753865.00,0.23,-92610838.00,0.04
164,P-1727,1. INGRESO,0.00,NaN,NaN,11005000.00,0.00,11005000.00,0.23,166727122.73,0.07
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.23,0.00,0.00


In [432]:
# TESTING VISUAL 2
 
df_pdt_vs_prep_2[df_pdt_vs_prep_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48,-156095149.00,0.85
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48,-387816472.00,0.47
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48,1058718831.87,0.57
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,-31281353.00,-0.00


In [433]:
df_pdt_vs_prep_2.to_csv('archivos_respaldo/df_pdt_vs_prep_2.csv', index = False)

### 4.16 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO

#### 4.16.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [434]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2_2 = df_ptd_cerrado_y_proyectado2.copy()

df_ptd_cerrado_y_proyectado2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
273,P-1682,4. COSTO SUPERVISIÓN,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.37
54,P-1683,2. COSTO MATERIALES,CD BP-A05,EBCO S.A.,-23738720.00,0.00,-23738720.00,0.55
51,P-1682,3. COSTO MANO DE OBRA,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-21010033.00,-423674.00,-21433707.00,0.37
272,P-1681,4. COSTO SUPERVISIÓN,CD ROSEN (CCTV PROVISORIO),COLCHONES ROSEN SAIC,0.00,0.00,0.00,0.71
284,P-1689,1. INGRESO,CASINO BUENAVENTURA,EBCO S.A.,22795459.00,0.00,22795459.00,0.51


In [435]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2 = df_presupuesto_final4.copy()
df_presupuesto_final4_2.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2 = pd.pivot_table(df_presupuesto_final4_2, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2 = pd.DataFrame(df_presupuesto_final4_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
132,P-1674,1. INGRESO,28602126.00
28,P-1620,1. INGRESO,0.00
266,P-1709,3. COSTO MANO DE OBRA,-8592182.00
186,P-1688,3. COSTO MANO DE OBRA,-7417508.00
80,P-1656,1. INGRESO,36463980.00


#### 4.16.2 MERGE ENTRE DATAFRAMES

In [436]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 : ", df_ptd_cerrado_y_proyectado2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        : ", df_presupuesto_final4_2.shape)

df_pdt_proy_vs_prep = df_ptd_cerrado_y_proyectado2_2.merge(df_presupuesto_final4_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           : ", df_pdt_proy_vs_prep.shape)

df_pdt_proy_vs_prep.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
114,P-1706,1. INGRESO,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,36302208.00,11000000.00,47302208.00,0.41,104786220.54
236,P-1663,1. INGRESO,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0.00,0.00,0.00,0.00,65362150.00
234,P-1661,3. COSTO MANO DE OBRA,SAN VICENTE DE PAUL,DANACORP S.A,-1144426.00,0.00,-1144426.00,0.61,-8083499.00
188,P-1620,4. COSTO SUPERVISIÓN,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00,0.00,0.00,0.00,-417991.00
320,P-1720,2. COSTO MATERIALES,NaN,NaN,0.00,0.00,0.00,0.00,-21939079.00


#### 4.16.3 CREACIÓN DE COLUMNA % DE AVANCE

In [437]:
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['TOTAL_PDT'] / df_pdt_proy_vs_prep['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-1247014.00,-301065.00,-1548079.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,-2109962.00,-6703922.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,897720.00,0.00,897720.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,312558.00,0.00,312558.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0.00,-455187.00,0.77,-11381553.00,0.04


In [438]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883.00,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000.00,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,64764000.00,751195807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.49,-31281353.00,-0.00


In [439]:
# TESTING VISUAL 2
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883.00,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000.00,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,64764000.00,751195807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.49,-31281353.00,-0.00


In [440]:
df_pdt_proy_vs_prep.to_csv('archivos_respaldo/df_pdt_proy_vs_prep.csv', index = False)

### 4.17 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO (PERSPECTIVA MES ANTERIOR)

#### 4.17.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [441]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado_ant2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2_2 = df_ptd_cerrado_y_proyectado_ant2.copy()

df_ptd_cerrado_y_proyectado_ant2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado_ant2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
113,P-1706,2. COSTO MATERIALES,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,-7879330.00,-5000000.00,-12879330.00,0.48
194,P-1632,4. COSTO SUPERVISIÓN,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.00
109,P-1705,3. COSTO MANO DE OBRA,LOS CLARINES,CONSTRUCTORA INARCO S.A.,-8357637.00,-5553518.00,-13911155.00,0.05
280,P-1685,4. COSTO SUPERVISIÓN,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0.00,0.00,0.00,0.04
75,P-1692,3. COSTO MANO DE OBRA,JOSÉ PEREZ COTAPOS,EBCO S.A.,-40393357.00,-7576783.00,-47970140.00,-0.06


In [442]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2_1 = df_presupuesto_final4.copy()
df_presupuesto_final4_2_1.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2_1 = pd.pivot_table(df_presupuesto_final4_2_1, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2_1 = pd.DataFrame(df_presupuesto_final4_2_1.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 : ", df_presupuesto_final4_2_1.shape)

df_presupuesto_final4_2_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
329,P-1727,2. COSTO MATERIALES,-92610838.00
18,P-1603,3. COSTO MANO DE OBRA,-4345810.00
66,P-1645,3. COSTO MANO DE OBRA,-2547080.00
5,P-1557,2. COSTO MATERIALES,0.00
289,P-1717,2. COSTO MATERIALES,-70545144.00


#### 4.17.2 MERGE ENTRE DATAFRAMES

In [443]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          : ", df_presupuesto_final4_2_1.shape)

df_pdt_proy_vs_prep_2 = df_ptd_cerrado_y_proyectado_ant2_2.merge(df_presupuesto_final4_2_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           : ", df_pdt_proy_vs_prep_2.shape)

df_pdt_proy_vs_prep_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
65,P-1688,3. COSTO MANO DE OBRA,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-12243170.00,-2009744.00,-14252914.00,0.17,-7417508.00
173,P-1591,4. COSTO SUPERVISIÓN,EDIFICIO ESTORIL,EBCO S.A.,0.00,0.00,0.00,0.00,0.00
160,P-1726,2. COSTO MATERIALES,NaN,NaN,-14374046.00,-10000000.00,-24374046.00,0.36,-31369256.00
182,P-1611,4. COSTO SUPERVISIÓN,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,0.00,0.00,1.14,-2206785.00
89,P-1697,3. COSTO MANO DE OBRA,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-15721639.00,-4075065.00,-19796704.00,0.47,0.00


#### 4.17.3 CREACIÓN DE COLUMNA % DE AVANCE

In [444]:
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['TOTAL_PDT'] / df_pdt_proy_vs_prep_2['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-945949.00,-945949.00,-1891898.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,-2483998.00,-4967996.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,846183.00,0.00,846183.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,236593.00,0.00,236593.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0.00,-455187.00,0.77,-11381553.00,0.04


In [445]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep_2[df_pdt_proy_vs_prep_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00,0.48,-156095149.00,0.94
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00,0.48,-387816472.00,0.54
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28,0.48,1058718831.87,0.64
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,-31281353.00,-0.00


In [446]:
df_pdt_proy_vs_prep_2.to_csv('archivos_respaldo/df_pdt_proy_vs_prep_2.csv', index = False)

## 5. - EXPORTACIÓN DE ARCHIVOS

In [447]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name('keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

### 5.1 EXPORTACIÓN LIBRO MAYOR PARA CUADRO FINANCIERO DE PROYECTOS

In [448]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1COX0_p3dTPNLZMwoh__0uQdOYfP_AA1mw7lfKx4SMMQ')

# select a work sheet from its name
worksheet1 = gs.worksheet('Hoja 1')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df_libro_mayor6, include_index=False,
include_column_header=True, resize=True)

### 5.2 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO
Dataframe: df_ytd_cerrado3

In [449]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.2.1 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: df_ytd_cerrado_2M_3

In [450]:
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado_2M_3, include_index=False,
include_column_header=True, resize=True)

### 5.3 EXPORTACIÓN MES EN CURSO PROYECTADO
Dataframe: mes_en_curso2

In [451]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy5, include_index=False,
include_column_header=True, resize=True)

### 5.3.1 EXPORTACIÓN MES EN CURSO PROYECTADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: mes_en_curso_2M

In [452]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.4 EXPORTACIÓN MES EN CURSO

In [453]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real5, include_index=False,
include_column_header=True, resize=True)

### 5.4.1 EXPORTACIÓN MES EN CURSO PERSPECTIVA MES ANTERIOR

In [454]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.5 EXPORTACIÓN YTD PROYECTADO

In [455]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.5.1 EXPORTACIÓN YTD PROYECTADO PERSPECTIVA MES ANTERIOR

In [456]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado_2M_2, include_index=False,
include_column_header=True, resize=True)

### 5.6 EXPORTACIÓN PTD CERRADO

In [457]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.6.1 EXPORTACIÓN PTD CERRADO (PERSPECTIVA MES ANTERIOR)

In [458]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_anterior3, include_index=False,
include_column_header=True, resize=True)

### 5.7 EXPORTACIÓN PTD CERRADO + PROYECTADO

In [459]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.7.1 EXPORTACIÓN PTD CERRADO + PROYECTADO (PERSPECTIVA MES ANTERIOR)

In [460]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado_ant2, include_index=False,
include_column_header=True, resize=True)

### 5.8 EXPORTACIÓN PRESUPUESTO

In [461]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yTtFi7x0rc_FU6xb-FrdQGKndttaf0dg3ENDyOO52eM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_presupuesto_final4, include_index=False,
include_column_header=True, resize=True)

### 5.9 EXPORTACIÓN PENDIENTE

In [462]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente3, include_index=False,
include_column_header=True, resize=True)

### 5.10 EXPORTACIÓN PENDIENTE (PERSPECTIVA MES ANTERIOR)

In [463]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente_3, include_index=False,
include_column_header=True, resize=True)

### 5.11 EXPORTACIÓN PTD Cerrado vs Presupuesto

In [464]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_prep, include_index=False,
include_column_header=True, resize=True)

### 5.12 EXPORTACIÓN PTD Cerrado vs Presupuesto (PERSPECTIVA MES ANTERIOR)

In [465]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_prep_2, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto

In [466]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto (PERSPECTIVA ANTERIOR)

In [467]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep_2, include_index=False,
include_column_header=True, resize=True)